*****************************************
# Project Plan Draft V1

Goal:

Save time by automating more of the current test measurement process.

Means:

Remove need for Excel. By automating statistical testing and rebalancing steps.

The new templates primarily accomplish two things:

In Notebook Rebalancing
* Former: A 3 step process utilizing Databricks and Excel.
* New: Entire process performed completely in Databricks.
* Time saved - 30 mins to 1 hour.

In Notebook Statistical Testing (scorecard creation)
* Former: A 2-step process, utilizing Databricks and Excel
* New:  Entire process performed completely in Databricks.
* Time saved - 30 mins

Bonus:
* Ad-hoc analysis is many times faster as it no longer requires waiting for very long SQL code blocks with several cte’s to run.
* Once SQL data at `upm_id` level is loaded into a Python (pandas) data frame, ad-hoc calculations take 30 seconds or fewer.

Outline of the process:

COE group to determine best statistical tests for the Demand and Conversion data.

Currently, the templates apply:
* Mann Whitney U test for Demand (based on my analysis showing Demand data does not follow a normal distribution)
* Chi Square Test for Conversion. (Based on the data point for conversion  - ‘buyer’ - being a binary value)

Once there is alignment from COE on the correct test, I can then create iterations of the new template for 4 types of tests (AJO ANBD, NCP Incremental Demand, etc).

Currently there are 14 different templates in use.

Then, the code for these notebooks must be reviewed by Nike Data Science and Analytics.

The next step is training for the team on Python and how to use the templates:

Python Crash Course

* [Python for Everybody](https://www.py4e.com/lessons) (special attention to: Ch. 5 - Functions, Ch. 6 - Loops & Iterations, Ch. 9 - 11 Lists, Dictionaries and Tuples).
* [Google’s Python Crash Course](https://developers.google.com/edu/python)
OR [also available on Coursera](https://www.coursera.org/learn/python-crash-course?) - plenty of example problems here

Introduction to Pandas (the Python Library for Data Analysis)

  * [Kaggle Learn Pandas](https://www.kaggle.com/learn/pandas)
  * [A Gentle Introduction to Pandas Data Analysis](https://youtu.be/_Eb0utIRdkw?) - Kaggle on Youtube
  * [Google's Pandas Dataframe Ultraquick Tutorial](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/pandas_dataframe_ultraquick_tutorial.ipynb)
  * [Python Pandas for your Grandpa.](https://youtube.com/playlist?list=PL9oKUrtC4VP7ry0um1QOUUfJBXKnkf-dA&feature=shared) “So easy your grandpa could learn it.”

Optional: Introduction to Numpy (the Python library for doing advanced math on large datasets)

  * [Python Numpy for your Grandma.](https://www.youtube.com/playlist?list=PL9oKUrtC4VP6gDp1Vq3BzfViO0TWgR0vR) “So easy, your grandma could learn it!"
  * [Google's Ultra Quick Intro to Numpy](https://colab.research.google.com/github/google/eng-edu/blob/main/ml/cc/exercises/numpy_ultraquick_tutorial.ipynb?)


We should allow a 6-week period where both the new and old templates are run for each test.

Things we should compare:

Outcomes.
* Is there a difference between which tests reach stat sig.
Ease of use.
*  Is this adding complexity from the perspective of the team.
Velocity.
* Does this shrink the overall timing duration of test measurement from start to finish.

Kickoff week 1 with a walkthrough of a new test, on the new template.

Then, throughout this 6 week period, I will make myself available during the Peer Review standup for troubleshooting.

At the conclusion of the 6-week period, we should asses if these new templates are adding value, saving time, etc

It is possible in the end, we use some portion of the template, and not others.



## ANBD Error code

AnalysisException: Reference 'aud.member_id' is ambiguous, could be: aud.member_id, aud.member_id.; line 27 pos 59

In [ ]:
%sql

------------------------------------------------------------------------------------------------------------------------------------------------------------------
-- DEFINE CTEs
------------------------------------------------------------------------------------------------------------------------------------------------------------------
WITH
--- ACTIVE NON BUYING FREQUENCY *EXCLUDING* THE DAYS OF SEND
active_non_buying_excl_send_day AS (
    select anbd.member_id
    , aud.test_control
        ,count(distinct activity_dt) as active_nonbuying
    from(
          select member_agd.member_id,
                  activity_dt,
                  sum(logged_in_purchase_order_count) as purch
          -- from member_agd
          from aud_select_workspace.member_agg_member_growth_daily AS member_agd
          WHERE activity_dt BETWEEN ${hivevar:msmt_start_dt} AND DATE_ADD(${hivevar:msmt_end_dt},14) -- ANBD KPIs for Comms Pod are calculated over a 14 day period
            AND activity_dt NOT IN (${hivevar:msmt_start_dt}
--                                   , ${hivevar:msmt_second_send_dt}
--                                   , ${hivevar:msmt_third_send_dt}
--                                   , ${hivevar:msmt_end_dt}
                                   )
            AND preferred_retail_geo='NA'
            AND experience_name = 'nike_app'
            and member_agd.member_id is not null
          group by 1,2
    ) anbd
    INNER JOIN base_audience_table aud ON anbd.member_id = aud.member_id
    WHERE purch = 0
    GROUP BY 1,2
),

--Additional ANBD CTE to address the inherent bias in activity caused by a Test vs. Holdout setup, where the Holdout group receives no communication.
member_activity_with_dayOfSend as (
  SELECT
    anbde.member_id
    , anbde.test_control
    , AVG(PV_CNT) as num_views
    , CASE WHEN AVG(PV_CNT) > 9 THEN (max(active_nonbuying) + 1) ELSE max(active_nonbuying)
      END AS active_nonbuying
FROM active_non_buying_excl_send_day anbde
INNER JOIN digital_exp.FACT_CLICKSTREAM_SESSION_NIKEAPP cs
  on anbde.member_id = cs.member_id
WHERE SESSION_START_DT IN (${hivevar:msmt_start_dt}
--                           , date_add(${hivevar:msmt_start_dt}, 5) -- Example: second send was 5 days after the first send
                         )
and lower(COUNTRY_SKEY) = 'usa'
and SESSION_SKEY IS NOT NULL
and SESSION_END_GMT_EPOCH-SESSION_START_GMT_EPOCH > 0
and SESSION_END_GMT_EPOCH-SESSION_START_GMT_EPOCH <= 3600
and PURCHASE_CNT =0
GROUP BY 1,2
),

active_non_buying as (
SELECT
  member_id
  , test_control
  , active_nonbuying
FROM member_activity_with_dayOfSend
),


test_control_table AS (
	SELECT
		 aud.upm_id as upm_id
        ,anbd.member_id as member_id
        ,nuid
        ,aud.test_control
		,MAX(CASE WHEN aud.test_control = 'test' THEN 1 ELSE 0 END) AS test_ind
		,MAX(CASE WHEN aud.test_control = 'control'  THEN 1 ELSE 0 END) AS control_ind
    ,COALESCE(SUM(active_nonbuying),0) as active_non_buying
	FROM base_audience_table aud
    LEFT JOIN active_non_buying anbd ON aud.member_id = anbd.member_id
    ANTI JOIN gwan13.bot_master_nike_com bot ON aud.upm_id = bot.upm_id
    ANTI JOIN aud_select_workspace.na_resellers slr ON aud.upm_id = slr.upm_id
	GROUP BY 1,2,3,4
),

 view_dol AS (
  SELECT dol.upm_id
  , order_nbr
  , order_dt
  , origl_ordered_qty
  , grd_amt_excl_tax_usd
  , line_of_business_desc
  , univ_div_desc
  FROM awight.na_digital_order_line_snapshot dol
  INNER JOIN test_control_table tct ON dol.upm_id = tct.upm_id
  WHERE rec_excl_ind = 0
    AND ttl_demand_ind = 1
    AND univ_cat_desc <> 'Converse'
    AND dol.upm_id IS NOT NULL
    AND CAST(order_dt AS DATE) BETWEEN DATE_SUB(${hivevar:msmt_start_dt},730) AND DATE_ADD(${hivevar:msmt_end_dt},7)
    AND (rtn_qty = 0 or rtn_qty IS NULL)
),

member_activities as (
  SELECT
    member_id
    ,SUM(logged_in_visits_count) AS visits
    ,SUM(PDP_FAVORITE_COUNT) AS PDP_FAVORITE_COUNT
    ,SUM(ADD_TO_CART_COUNT) AS ADD_TO_CART_COUNT
    ,SUM(physical_activity_count) AS physical_activity
    ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
    ,SUM(eod_chat_count) AS NEOD_Session
  FROM member_agd
  WHERE activity_dt BETWEEN ${hivevar:msmt_start_dt} AND DATE_ADD(${hivevar:msmt_end_dt},7)
    AND UPPER(preferred_retail_geo) = 'NA'
    AND UPPER(experience_name) <> 'AGNOSTIC'
  GROUP BY 1
),


-- ACTIVE NON BUYING BUCKET FOR REBALANCING ON ACTIVE NON BUYING DAYS
active_non_buying_bucket as (
 SELECT
    upm_id
   , z.member_id
    , z.test_control
    , active_nonbuying
    , CASE WHEN active_nonbuying = 0 OR active_nonbuying is NULL THEN 'non-active'
         WHEN active_nonbuying < 6 THEN '000-005'
         WHEN active_nonbuying < 11 THEN '006-010'
         WHEN active_nonbuying < 16 THEN '011-015'
         WHEN active_nonbuying < 21 THEN '016-020'
         WHEN active_nonbuying < 26 THEN '021-025'
         WHEN active_nonbuying <=31 THEN '026-031'
    END AS anb_bucket
    , percent_rank() over (
    partition by
    z.test_control
    ,CASE WHEN active_nonbuying = 0 OR active_nonbuying is NULL THEN 'non-active'
         WHEN active_nonbuying < 6 THEN '000-005'
         WHEN active_nonbuying < 11 THEN '006-010'
         WHEN active_nonbuying < 16 THEN '011-015'
         WHEN active_nonbuying < 21 THEN '016-020'
         WHEN active_nonbuying < 26 THEN '021-025'
         WHEN active_nonbuying <=31 THEN '026-031'
         END order by RAND(0)) as anb_percent_rank
  from test_control_table z
  left join active_non_buying anb
    on z.member_id = anb.member_id
),

balanced_aud as (
  SELECT DISTINCT a.upm_id
  FROM active_non_buying_bucket a
  -- remove outliers
  WHERE
  -- rebalance on active non-buying days
     ((a.test_control = 'control' AND (active_nonbuying > 0 AND active_nonbuying <= 5)   AND anb_percent_rank <= 1)
      OR (a.test_control = 'control' AND (active_nonbuying > 5 AND active_nonbuying <= 10)  AND anb_percent_rank <= 1)
      OR (a.test_control = 'control' AND (active_nonbuying > 10 AND active_nonbuying <= 15) AND anb_percent_rank <= 1)
      OR (a.test_control = 'control' AND (active_nonbuying > 15 AND active_nonbuying <= 20) AND anb_percent_rank <= 1)
      OR (a.test_control = 'control' AND (active_nonbuying > 20 AND active_nonbuying <= 25) AND anb_percent_rank <= 1)
      OR (a.test_control = 'control' AND (active_nonbuying >                            25) AND anb_percent_rank <= 1)
      OR (a.test_control = 'control' AND (active_nonbuying = 0 OR active_nonbuying IS null) AND anb_percent_rank <= 1)

      OR (a.test_control = 'test' AND (active_nonbuying > 0 AND active_nonbuying <= 5)   AND anb_percent_rank <= 1)
      OR (a.test_control = 'test' AND (active_nonbuying > 5 AND active_nonbuying <= 10)  AND anb_percent_rank <= 1)
      OR (a.test_control = 'test' AND (active_nonbuying > 10 AND active_nonbuying <= 15) AND anb_percent_rank <= 1)
      OR (a.test_control = 'test' AND (active_nonbuying > 15 AND active_nonbuying <= 20) AND anb_percent_rank <=  1)
      OR (a.test_control = 'test' AND (active_nonbuying > 20 AND active_nonbuying <= 25) AND anb_percent_rank <= 1)
      OR (a.test_control = 'test' AND (active_nonbuying >                            25) AND anb_percent_rank <=  1)
      OR (a.test_control = 'test' AND (active_nonbuying = 0 OR active_nonbuying IS null) AND anb_percent_rank <= 1))
),


ab_purchase as (
  SELECT
    upm_id as upm_id_ab_buyer
    , order_dt as order_ts
    , order_nbr
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,SUM(CASE WHEN UPPER(DOL.LINE_OF_BUSINESS_DESC) = 'BASE INLINE' AND Univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
    ,SUM(CASE WHEN UPPER(DOL.LINE_OF_BUSINESS_DESC) = 'BASE INLINE' AND Univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
    ,SUM(CASE WHEN UPPER(DOL.LINE_OF_BUSINESS_DESC) = 'BASE INLINE' AND (Univ_div_desc = 'Equipment' OR Univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
    ,SUM(CASE WHEN UPPER(DOL.LINE_OF_BUSINESS_DESC) = 'CLEARANCE' AND Univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
    ,SUM(CASE WHEN UPPER(DOL.LINE_OF_BUSINESS_DESC) = 'CLEARANCE' AND Univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
    ,SUM(CASE WHEN UPPER(DOL.LINE_OF_BUSINESS_DESC) = 'CLEARANCE' AND (Univ_div_desc = 'Equipment' OR Univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
    ,SUM(CASE WHEN UPPER(DOL.LINE_OF_BUSINESS_DESC) = 'LAUNCH' AND Univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
    ,SUM(CASE WHEN UPPER(DOL.LINE_OF_BUSINESS_DESC) = 'LAUNCH' AND Univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
    ,SUM(CASE WHEN UPPER(DOL.LINE_OF_BUSINESS_DESC) = 'LAUNCH' AND (Univ_div_desc = 'Equipment' OR Univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
    ,SUM(CASE WHEN UPPER(DOL.LINE_OF_BUSINESS_DESC) NOT IN ('BASE INLINE', 'CLEARANCE', 'LAUNCH') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND date_add(${hivevar:msmt_end_dt}, 7)  -- 7 DAY MEASUREMENT FOR DEMAND AND CONVERSION
  GROUP BY 1,2,3
)

--First Usage CTE
/*
, platform_usage as (
  SELECT DISTINCT flt.upm_id
  , CASE WHEN CAST(first_${hivevar:usage_platform}_activity_dt AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND DATE_ADD(${hivevar:msmt_end_dt},7) THEN 1
         ELSE 0
    END AS platform_user
  FROM aud_select_workspace.gck_first_and_last_touchpoint flt
  INNER JOIN test_control_table tc ON flt.upm_id = tc.upm_id
)
*/


------------------------------------------------------------------------------------------------
----------------------------------------- Main Query -------------------------------------------
------------------------------------------------------------------------------------------------


SELECT
  test_control
  ,COUNT(DISTINCT z.upm_id) AS total_members
  -- DEMAND AND CONVERSION KPIs
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) AS buying_members
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) / COUNT(DISTINCT z.upm_id) AS conversion_rate
  ,AVG(demand) AS avg_demand
  ,stddev(demand) AS std_demand
  -- secondary metrics ON order double click
  ,SUM(demand) / SUM(orders) AS AOV
  ,SUM(demand) / SUM(units) AS AUR
  ,SUM(units) / SUM(orders) AS UPT




  -- secondary metrics ON site/app engagement
  ,AVG(visits) AS avg_site_app_visits
  ,AVG(PDP_FAVORITE_COUNT) AS avg_PDP_FAVORITE_COUNT
  ,AVG(ADD_TO_CART_COUNT) AS avg_ADD_TO_CART_COUNT
  ,AVG(physical_activity) AS avg_physical_activity


  -- secondar metrics ON margin
  ,(AVG(base_FW_demand) * 0.38 + AVG(base_AP_demand) * 0.28 + AVG(base_EQ_demand) * 0.32
    + AVG(clr_FW_demand) * 0.23 + AVG(base_AP_demand) * 0.17 + AVG(clr_EQ_demand) * 0.12
    + AVG(launch_FW_demand) * 0.25 + AVG(launch_AP_demand) * 0.08 + AVG(launch_EQ_demand) * 0.19
    + AVG(rest_demand) * 0.3
  ) AS avg_margin
  ,AVG(base_FW_demand) AS base_FW_demand
  ,AVG(base_AP_demand) AS base_AP_demand
  ,AVG(base_EQ_demand) AS base_EQ_demand
  ,AVG(clr_FW_demand) AS clr_FW_demand
  ,AVG(clr_AP_demand) AS clr_AP_demand
  ,AVG(clr_EQ_demand) AS clr_EQ_demand
  ,AVG(launch_FW_demand) AS launch_FW_demand
  ,AVG(launch_AP_demand) AS launch_AP_demand
  ,AVG(launch_EQ_demand) AS launch_EQ_demand
  ,AVG(rest_demand) AS rest_demand


  -- ACTIVE NON BUYING KPIs
  ,SUM(active_non_buying) AS active_non_buying
  ,AVG(active_non_buying) AS avg_active_non_buying
  ,STD(active_non_buying) AS std_active_non_buying

--------------------------------------- ADD AS NECESSARY ---------------------------------------
 /*
  --Optional: FIRST PLATFORM USERS KPI
  ,SUM(platform_user) AS first_usage
  ,SUM(platform_user)/COUNT(DISTINCT z.upm_id) AS first_usage_pct
*/
/*  -- Optional KPI's  --
  ,SUM(NEOD_Session) AS NEOD_Session
*/


FROM test_control_table z

INNER JOIN balanced_aud b
ON z.upm_id = b.upm_id

LEFT JOIN member_activities a
ON z.member_id = a.member_id

LEFT JOIN ab_purchase abp
ON z.upm_id = abp.upm_id_ab_buyer



--------------------------------------- First Usage---------------------------------------
--LEFT JOIN platform_usage plt ON z.upm_id = plt.upm_id

WHERE (demand < 600 OR demand IS NULL)

GROUP BY 1
ORDER BY 1 DESC;

### Original Code with User Defined Stats Functions

In [ ]:
%python

# Execute SQL Query
ab_data = """
WITH
 aa_purchase AS (
  SELECT dol.upm_id
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1

  -- AA period purchase. dynamically based on when the member receives their first communication.
)

, demand_bucket AS (
  SELECT tct.upm_id
  , tct.test_control
  , aa.upm_id as upm_id_aa_buyer
  , aa.demand
  , aa.orders
  , aa.units
  , CASE
    WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
    WHEN aa.demand < 50 THEN '000-050'
    WHEN aa.demand < 100 THEN '050-100'
    WHEN aa.demand < 200 THEN '100-200'
    WHEN aa.demand < 500 THEN '200-500'
    WHEN aa.demand < 1000 THEN '500-1000'
    WHEN aa.demand >=1000 THEN '1000+'
    END AS demand_per_buyer_bucket
  , PERCENT_RANK() OVER (
    PARTITION BY test_control
    , CASE
      WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
      WHEN aa.demand < 50 THEN '000-050'
      WHEN aa.demand < 100 THEN '050-100'
      WHEN aa.demand < 200 THEN '100-200'
      WHEN aa.demand < 500 THEN '200-500'
      WHEN aa.demand < 1000 THEN '500-1000'
      WHEN aa.demand >=1000 THEN '1000+'
      END ORDER BY RANDOM(0)
    ) AS percent_rank
  FROM test_control_table tct
  LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
)

, balanced_aud_aa as (
  SELECT DISTINCT upm_id
  FROM demand_bucket db
  -- remove outliers
  WHERE (demand < 2400 OR demand IS NULL)
-- To rebalance, paste text block from template in place of defaults
AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
	)
)

, mhub as (
 SELECT source_id as upm_id
 , member_id
 , preferred_gender
 FROM MEMBER.MEMBER_HUB
 WHERE ctry_2_cd = 'US'
 AND lower(dpa_status) IN ('activate','reactivate')
)

-- calculate a dynamic member activity window for each communication received by a member
-- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
-- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
, member_activities as (
  SELECT mgd.member_id
  ,SUM(logged_in_visits_count) AS visits
  ,SUM(pdp_favorite_count) AS pdp_favorite_count
  ,SUM(add_to_cart_count) AS add_to_cart_count
  ,SUM(physical_activity_count) AS physical_activity
  ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
  FROM test_control_table tct
  INNER JOIN mhub mh ON tct.upm_id = mh.upm_id
  LEFT JOIN aud_select_workspace.member_agg_member_growth_daily mgd ON mh.member_id = mgd.member_id
  WHERE mgd.activity_dt >= tct.campaign_send_dt
    AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
    AND mgd.preferred_retail_geo = 'NA'
    AND mgd.experience_name != 'agnostic'
  GROUP BY 1
)

-- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
, ab_purchase as (
  SELECT dol.upm_id as upm_id_ab_buyer
  , order_dt as order_ts
  , order_nbr
  ,SUM(grd_amt_excl_tax_usd) AS demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
  ,COUNT(DISTINCT order_nbr) AS orders
  ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(first_send_dt,7)
  GROUP BY 1,2,3
)

-- For each member, rank each order by how near it was to the last campaign send
-- purchases must be after the campaign send and within 7 days of the send to be attributed
, aud_order_join as (
SELECT a.*
, b.*
,(bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 as order_send_diff_minutes
,RANK() OVER(PARTITION BY a.upm_id, b.order_ts ORDER BY (bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 ASC) as nearest_order
FROM test_control_table a
LEFT JOIN ab_purchase b ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(a.campaign_send_ts,7))
ORDER BY a.upm_id,a.campaign_send_dt, nearest_order asc
)

-- Aggregate each distinct send, attributing only the nearest order.
-- This prevent double counting of orders when a members receives multiple sends within the measurement window.
, aud_order_agg as (
SELECT upm_id AS upm_id_ab_buyer
  ,SUM(demand) AS demand
  ,SUM(base_FW_demand) AS base_FW_demand
  ,SUM(base_AP_demand) AS base_AP_demand
  ,SUM(base_EQ_demand) AS base_EQ_demand
  ,SUM(clr_FW_demand) AS clr_FW_demand
  ,SUM(clr_AP_demand) AS clr_AP_demand
  ,SUM(clr_EQ_demand) AS clr_EQ_demand
  ,SUM(launch_FW_demand) AS launch_FW_demand
  ,SUM(launch_AP_demand) AS launch_AP_demand
  ,SUM(launch_EQ_demand) AS launch_EQ_demand
  ,SUM(rest_demand) AS rest_demand
  ,SUM(orders) AS orders
  ,SUM(units) AS units
FROM aud_order_join a
WHERE nearest_order = 1
AND demand IS NOT NULL -- additional filter for extraneous records
GROUP BY 1
ORDER BY 1
)

, historical_purchase AS (
  SELECT aoa.upm_id_ab_buyer as upm_id
  , COUNT(DISTINCT order_nbr) as histOrders
  FROM aud_order_agg aoa  -- eligible audience
  INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

-- due to how messages are set up on Bluecore, the campaign_cd is not unique to a particular day. We concat with the message_sent_ts
-- only using the date ignores cases where multiple sends occur on the same day, potentially undercounting
,email_sent AS (
  SELECT fed.upm_id
  , COUNT(DISTINCT concat(campaign_cd, LEFT(message_sent_ts,21))) as email_circ
  FROM COMMS.FACT_EMAIL_DELIVERY fed
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(message_sent_ts AS DATE) between ${hivevar:msmt_start_dt} and ${hivevar:msmt_end_dt}
  AND retail_geo_name = 'NA'
  AND fed.upm_id IS NOT NULL
  GROUP BY upm_id
)

-- returns all the instances of the campaign codes for each member along with the response types and revenue if applicable
, email_response as (
  SELECT DISTINCT fer.upm_id
  , LEFT(response_ts,10) AS response_ts
  , response_type
  , revenue_dollar
  FROM comms.fact_email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(LEFT(response_ts,10) AS DATE) between first_send_dt and DATE_ADD(last_send_dt,7)
  AND retail_geo_name = 'NA'
  AND fer.upm_id IS NOT NULL
 )

-- calculates the the total number of opens, clicks, purchases and last touch attributed demand for each member for the campaign
-- responses are only considered if they occur between when the message was sent and 7 days later.
, email_calc AS (
  SELECT fer.upm_id
  , COUNT(CASE WHEN response_type= 'email open' THEN fer.upm_id ELSE NULL END) AS email_opens
  , COUNT(CASE WHEN response_type='link click' THEN fer.upm_id ELSE NULL END) AS email_clicks
  , COUNT(CASE WHEN response_type='ecommerce-purchase' THEN fer.upm_id ELSE NULL END) AS email_purch
  , SUM(CASE WHEN response_type='ecommerce-purchase' THEN revenue_dollar ELSE 0 END) AS email_demand
  FROM email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE CAST(fer.response_ts as DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7)
  GROUP BY 1
)
-- total number of total messages send for each member during the campaign.
-- For a singular campaign, this should be one (1), for triggers it could be more depending on how the trigger is set up
, email_delivery_agg as (
  SELECT upm_id
  ,SUM(email_circ) AS email_circ
  FROM email_sent
  GROUP BY 1
)

, email_response_agg as (
  SELECT upm_id
  ,SUM(email_opens) AS email_opens
  ,SUM(email_clicks) AS email_clicks
  ,SUM(email_purch) AS email_purch
  ,SUM(email_demand) AS email_demand
  FROM email_calc
  GROUP BY 1
)

, platform_usage as (
  SELECT tct.upm_id
  , CASE WHEN CAST(first_${hivevar:usage_platform}_activity_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7) THEN 1
         ELSE 0
    END AS platform_user
  FROM aud_select_workspace.gck_first_and_last_touchpoint flt
  INNER JOIN test_control_table tct ON flt.upm_id = tct.upm_id  -- Need first/last_send values as activity window
)

---------------------
-- MAIN QUERY
----------------------
SELECT z.test_control
  -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
  ,COUNT(DISTINCT z.upm_id) AS total_members
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) AS buying_members
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) / COUNT(DISTINCT z.upm_id) AS conversion_rate
  ,AVG(demand) AS avg_demand
  ,stddev(demand) AS std_demand
  -- secondary metrics ON order double click
  ,SUM(demand) / SUM(orders) AS AOV
  ,SUM(demand) / SUM(units) AS AUR
  ,SUM(units) / SUM(orders) AS UPT

  -- secondary metrics ON emails
  ,SUM(email_circ) AS email_sent
  ,SUM(email_opens) AS email_opens
  ,SUM(email_clicks) AS email_clicks
  ,SUM(email_purch) AS email_purch
  ,SUM(email_demand) AS email_demand
  ,SUM(email_opens) / SUM(email_circ) AS email_open_rate
  ,SUM(email_clicks) / SUM(email_circ) AS email_click_rate --against sends
  --,SUM(email_clicks) / SUM(email_opens) AS email_click_rate --against clicks
  ,SUM(email_purch) / SUM(email_circ) AS email_conversion_rate

  -- secondary metrics ON site/app engagement
  ,AVG(visits) AS avg_site_app_visits
  ,AVG(PDP_FAVORITE_COUNT) AS avg_PDP_FAVORITE_COUNT
  ,AVG(ADD_TO_CART_COUNT) AS avg_ADD_TO_CART_COUNT
  ,AVG(physical_activity) AS avg_physical_activity

  -- secondary metrics ON margin
  ,(AVG(base_FW_demand) * 0.38 + AVG(base_AP_demand) * 0.28 + AVG(base_EQ_demand) * 0.32
    + AVG(clr_FW_demand) * 0.23 + AVG(base_AP_demand) * 0.17 + AVG(clr_EQ_demand) * 0.12
    + AVG(launch_FW_demand) * 0.25 + AVG(launch_AP_demand) * 0.08 + AVG(launch_EQ_demand) * 0.19
    + AVG(rest_demand) * 0.3
  ) AS avg_margin
  ,AVG(base_FW_demand) AS base_FW_demand
  ,AVG(base_AP_demand) AS base_AP_demand
  ,AVG(base_EQ_demand) AS base_EQ_demand
  ,AVG(clr_FW_demand) AS clr_FW_demand
  ,AVG(clr_AP_demand) AS clr_AP_demand
  ,AVG(clr_EQ_demand) AS clr_EQ_demand
  ,AVG(launch_FW_demand) AS launch_FW_demand
  ,AVG(launch_AP_demand) AS launch_AP_demand
  ,AVG(launch_EQ_demand) AS launch_EQ_demand
  ,AVG(rest_demand) AS rest_demand
    -- secondary metrics ON order history
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS DPB_1_prior
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS DPB_2_prior
    -- optional first usage kpi
  ,SUM(platform_user) AS first_usage

FROM test_control_table z
INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

INNER JOIN mhub mh ON z.upm_id = mh.upm_id
LEFT JOIN member_activities a ON mh.member_id = a.member_id

LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

LEFT JOIN email_delivery_agg cda ON z.upm_id = cda.upm_id
LEFT JOIN email_response_agg cra ON z.upm_id = cra.upm_id

LEFT JOIN platform_usage plt ON z.upm_id = plt.upm_id

GROUP BY 1
ORDER BY 1 DESC;
"""


# Load the data into a PySpark DataFrame
df_spark = spark.sql(ab_data)

# Convert to Pandas DataFrame for further analysis
df = df_spark.toPandas()

# Separate control and test group data
control = df[df['test_control'] == 'control']
test = df[df['test_control'] == 'test']

# Extract input parameters for Demand
control_avg_sales = control['avg_demand'].values[0]
test_avg_sales = test['avg_demand'].values[0]
control_std_dev = control['std_demand'].values[0]
test_std_dev = test['std_demand'].values[0]
control_sample_size = control['buying_members'].values[0]
test_sample_size = test['buying_members'].values[0]

# Perform Statistical Analysis for Demand

# Calculate delta
delta = abs(test_avg_sales - control_avg_sales)

# Calculate standard error
standard_error = np.sqrt((control_std_dev**2 / control_sample_size) +
                         (test_std_dev**2 / test_sample_size))
# Calculate z-value
z_value = delta / standard_error

# Calculate p-value and confidence level
p_value = 2 * (1 - stats.norm.cdf(abs(z_value)))  # Two-tailed test
confidence_level = 1 - p_value
type_i_error = p_value # In a two-tailed test, the p-value is the Type I error

# Extracting input parameters for Conversion
control_conv_rate = control['conversion_rate'].values[0]
test_conv_rate = test['conversion_rate'].values[0]
conv_control_sample_size = control['total_members'].values[0]
conv_test_sample_size = test['total_members'].values[0]
lift = abs(test_conv_rate - control_conv_rate)  # (Abs. % point)

# Perform Statistical Analysis for Conversion

difference_of_proportions = test_conv_rate - control_conv_rate
# Calculate standard error
conv_standard_error = np.sqrt((control_conv_rate * (1 - control_conv_rate) / control_sample_size) +
                         (test_conv_rate * (1 - test_conv_rate) / test_sample_size))
# Calculate z value
conv_z_value = difference_of_proportions / conv_standard_error
# Calculate p-value and confidence level
conv_p_value = 2 * (1 - stats.norm.cdf(abs(conv_z_value)))  # Two-tailed test
conv_confidence_level = 1 - conv_p_value
conv_type_i_error = conv_p_value

# Demand Metrics DataFrame
data_demand = {
    'Metric': ['Avg Demand', 'Std Deviation of Demand', 'Sample Size',
               'Absolute Difference Between Means (Delta)', 'Standard Error', 'Z-Value'],
    'Control Group': [control_avg_sales, control_std_dev, control_sample_size, '', '', ''],
    'Test Group': [test_avg_sales, test_std_dev, test_sample_size, '', '', ''],
    'Value': ['', '', '', delta, standard_error, z_value]
}

demand_metrics_df = pd.DataFrame(data_demand)

# Update rows for Delta, Standard Error, and Z-Value, as they are not group-specific
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Absolute Difference Between Means (Delta)', ['Control Group', 'Test Group', 'Value']] = ['', '', delta]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Standard Error', ['Control Group', 'Test Group', 'Value']] = ['', '', standard_error]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Z-Value', ['Control Group', 'Test Group', 'Value']] = ['', '', z_value]

# Demand Significance DataFrame
data_demand_significance = {
    'Significance Metric': ['Achieved Confidence Level', 'Type I Error (α)'],
    'Value': [confidence_level, type_i_error]
}

demand_significance_df = pd.DataFrame(data_demand_significance)





# Demand Metrics DataFrame
data_demand = {
    'Metric': ['Avg Demand', 'Std Deviation of Demand', 'Sample Size',
               'Absolute Difference Between Means (Delta)', 'Standard Error', 'Z-Value','Achieved Confidence Level', 'Type I Error (α)'],
    'Control Group': [control_avg_sales, control_std_dev, control_sample_size, '', '', '', '', ''],
    'Test Group': [test_avg_sales, test_std_dev, test_sample_size, '', '', '','', ''],
    'Value': ['', '', '', delta, standard_error, z_value, confidence_level, type_i_error]
}

demand_metrics_df = pd.DataFrame(data_demand)

# Update rows for Delta, Standard Error, and Z-Value, as they are in dataframe calcuations
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Absolute Difference Between Means (Delta)', ['Control Group', 'Test Group', 'Value']] = ['', '', delta,]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Standard Error', ['Control Group', 'Test Group', 'Value']] = ['', '', standard_error]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Z-Value', ['Control Group', 'Test Group', 'Value']] = ['', '', z_value]

print(demand_metrics_df)


# Conversion Metrics DataFrame
data_conversion = {
    'Metric': ['Conversion Rate', 'Lift (Absolute % Point Difference)', 'Difference of Proportions',
               'Standard Error', 'Z-Value', 'Achieved Confidence Level', 'Type I Error (α)'],
    'Control Group': [control_conv_rate, '', '', '', '', '', ''],
    'Test Group': [test_conv_rate, '', '', '', '','', ''],
    'Value': ['', lift, difference_of_proportions, conv_standard_error, conv_z_value, conv_confidence_level, conv_type_i_error]
}

conversion_metrics_df = pd.DataFrame(data_conversion)

# Update rows for Delta, Standard Error, and Z-Value, as they are in dataframe calcuations
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Lift (Absolute % Point Difference)', ['Control Group', 'Test Group', 'Value']] = ['', '', lift]
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Difference of Proportions', ['Control Group', 'Test Group', 'Value']] = ['', '', difference_of_proportions]
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Standard Error', ['Control Group', 'Test Group', 'Value']] = ['', '', conv_standard_error]
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Z-Value', ['Control Group', 'Test Group', 'Value']] = ['', '', conv_z_value]

print(conversion_metrics_df)



"""
Update with Stats calcs to convert to Scipy

# Calculate delta
# Calculate standard error
# Calculate z value
# Calculate p-value
# Calculate type I error
# Calculate confidence level
# Calculate statistical significance


import numpy as np
from scipy import stats
import pandas as pd

# Assuming control and test groups are already defined as DataFrames

# Assume 'avg_demand' and 'std_demand' are columns in the DataFrame

# Perform t-test for Avg Demand
t_stat, p_value_demand = stats.ttest_ind(test['avg_demand'], control['avg_demand'], equal_var=False)
confidence_level_demand = 1 - p_value_demand
type_i_error_demand = p_value_demand

# Perform t-test for Conversion Rate (Replace 'conversion_rate' with the actual column names in your DataFrame)
t_stat_conv, p_value_conv = stats.ttest_ind(test['conversion_rate'], control['conversion_rate'], equal_var=False)
confidence_level_conv = 1 - p_value_conv
type_i_error_conv = p_value_conv

# Calculate lift for demonstration (you'll adjust this based on your metric of interest)
# Lift calculation example for conversion rate
lift_conversion_rate = ((test['conversion_rate'].mean() - control['conversion_rate'].mean()) / control['conversion_rate'].mean()) * 100

# The above examples show how to use SciPy for p-value and confidence level calculations.
# For delta, standard error, and z-value, these are part of the t-test calculation, and you typically wouldn't calculate these separately when using a t-test.
# If you need to report these, they're more relevant in the context of manual calculations or when not using a t-test directly.


"""

### Turn Word **Wrap**

In [ ]:
# turn on word wrap

import textwrap

wrapper = textwrap.TextWrapper(width=40,
    initial_indent=" " * 4,
    subsequent_indent=" " * 4,
    break_long_words=False,
    break_on_hyphens=False)

### Updated with SciPy Stats calculations, rather than user defined functions

In [ ]:

%python

# Execute SQL Query
ab_data = """
WITH
 aa_purchase AS (
  SELECT dol.upm_id
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1

  -- AA period purchase. dynamically based on when the member receives their first communication.
)

, demand_bucket AS (
  SELECT tct.upm_id
  , tct.test_control
  , aa.upm_id as upm_id_aa_buyer
  , aa.demand
  , aa.orders
  , aa.units
  , CASE
    WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
    WHEN aa.demand < 50 THEN '000-050'
    WHEN aa.demand < 100 THEN '050-100'
    WHEN aa.demand < 200 THEN '100-200'
    WHEN aa.demand < 500 THEN '200-500'
    WHEN aa.demand < 1000 THEN '500-1000'
    WHEN aa.demand >=1000 THEN '1000+'
    END AS demand_per_buyer_bucket
  , PERCENT_RANK() OVER (
    PARTITION BY test_control
    , CASE
      WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
      WHEN aa.demand < 50 THEN '000-050'
      WHEN aa.demand < 100 THEN '050-100'
      WHEN aa.demand < 200 THEN '100-200'
      WHEN aa.demand < 500 THEN '200-500'
      WHEN aa.demand < 1000 THEN '500-1000'
      WHEN aa.demand >=1000 THEN '1000+'
      END ORDER BY RANDOM(0)
    ) AS percent_rank
  FROM test_control_table tct
  LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
)

, balanced_aud_aa as (
  SELECT DISTINCT upm_id
  FROM demand_bucket db
  -- remove outliers
  WHERE (demand < 2400 OR demand IS NULL)
-- To rebalance, paste text block from template in place of defaults
AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
	)
)

, mhub as (
 SELECT source_id as upm_id
 , member_id
 , preferred_gender
 FROM MEMBER.MEMBER_HUB
 WHERE ctry_2_cd = 'US'
 AND lower(dpa_status) IN ('activate','reactivate')
)

-- calculate a dynamic member activity window for each communication received by a member
-- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
-- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
, member_activities as (
  SELECT mgd.member_id
  ,SUM(logged_in_visits_count) AS visits
  ,SUM(pdp_favorite_count) AS pdp_favorite_count
  ,SUM(add_to_cart_count) AS add_to_cart_count
  ,SUM(physical_activity_count) AS physical_activity
  ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
  FROM test_control_table tct
  INNER JOIN mhub mh ON tct.upm_id = mh.upm_id
  LEFT JOIN aud_select_workspace.member_agg_member_growth_daily mgd ON mh.member_id = mgd.member_id
  WHERE mgd.activity_dt >= tct.campaign_send_dt
    AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
    AND mgd.preferred_retail_geo = 'NA'
    AND mgd.experience_name != 'agnostic'
  GROUP BY 1
)

-- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
, ab_purchase as (
  SELECT dol.upm_id as upm_id_ab_buyer
  , order_dt as order_ts
  , order_nbr
  ,SUM(grd_amt_excl_tax_usd) AS demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
  ,COUNT(DISTINCT order_nbr) AS orders
  ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(first_send_dt,7)
  GROUP BY 1,2,3
)

-- For each member, rank each order by how near it was to the last campaign send
-- purchases must be after the campaign send and within 7 days of the send to be attributed
, aud_order_join as (
SELECT a.*
, b.*
,(bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 as order_send_diff_minutes
,RANK() OVER(PARTITION BY a.upm_id, b.order_ts ORDER BY (bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 ASC) as nearest_order
FROM test_control_table a
LEFT JOIN ab_purchase b ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(a.campaign_send_ts,7))
ORDER BY a.upm_id,a.campaign_send_dt, nearest_order asc
)

-- Aggregate each distinct send, attributing only the nearest order.
-- This prevent double counting of orders when a members receives multiple sends within the measurement window.
, aud_order_agg as (
SELECT upm_id AS upm_id_ab_buyer
  ,SUM(demand) AS demand
  ,SUM(base_FW_demand) AS base_FW_demand
  ,SUM(base_AP_demand) AS base_AP_demand
  ,SUM(base_EQ_demand) AS base_EQ_demand
  ,SUM(clr_FW_demand) AS clr_FW_demand
  ,SUM(clr_AP_demand) AS clr_AP_demand
  ,SUM(clr_EQ_demand) AS clr_EQ_demand
  ,SUM(launch_FW_demand) AS launch_FW_demand
  ,SUM(launch_AP_demand) AS launch_AP_demand
  ,SUM(launch_EQ_demand) AS launch_EQ_demand
  ,SUM(rest_demand) AS rest_demand
  ,SUM(orders) AS orders
  ,SUM(units) AS units
FROM aud_order_join a
WHERE nearest_order = 1
AND demand IS NOT NULL -- additional filter for extraneous records
GROUP BY 1
ORDER BY 1
)

, historical_purchase AS (
  SELECT aoa.upm_id_ab_buyer as upm_id
  , COUNT(DISTINCT order_nbr) as histOrders
  FROM aud_order_agg aoa  -- eligible audience
  INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

-- due to how messages are set up on Bluecore, the campaign_cd is not unique to a particular day. We concat with the message_sent_ts
-- only using the date ignores cases where multiple sends occur on the same day, potentially undercounting
,email_sent AS (
  SELECT fed.upm_id
  , COUNT(DISTINCT concat(campaign_cd, LEFT(message_sent_ts,21))) as email_circ
  FROM COMMS.FACT_EMAIL_DELIVERY fed
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(message_sent_ts AS DATE) between ${hivevar:msmt_start_dt} and ${hivevar:msmt_end_dt}
  AND retail_geo_name = 'NA'
  AND fed.upm_id IS NOT NULL
  GROUP BY upm_id
)

-- returns all the instances of the campaign codes for each member along with the response types and revenue if applicable
, email_response as (
  SELECT DISTINCT fer.upm_id
  , LEFT(response_ts,10) AS response_ts
  , response_type
  , revenue_dollar
  FROM comms.fact_email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(LEFT(response_ts,10) AS DATE) between first_send_dt and DATE_ADD(last_send_dt,7)
  AND retail_geo_name = 'NA'
  AND fer.upm_id IS NOT NULL
 )

-- calculates the the total number of opens, clicks, purchases and last touch attributed demand for each member for the campaign
-- responses are only considered if they occur between when the message was sent and 7 days later.
, email_calc AS (
  SELECT fer.upm_id
  , COUNT(CASE WHEN response_type= 'email open' THEN fer.upm_id ELSE NULL END) AS email_opens
  , COUNT(CASE WHEN response_type='link click' THEN fer.upm_id ELSE NULL END) AS email_clicks
  , COUNT(CASE WHEN response_type='ecommerce-purchase' THEN fer.upm_id ELSE NULL END) AS email_purch
  , SUM(CASE WHEN response_type='ecommerce-purchase' THEN revenue_dollar ELSE 0 END) AS email_demand
  FROM email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE CAST(fer.response_ts as DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7)
  GROUP BY 1
)
-- total number of total messages send for each member during the campaign.
-- For a singular campaign, this should be one (1), for triggers it could be more depending on how the trigger is set up
, email_delivery_agg as (
  SELECT upm_id
  ,SUM(email_circ) AS email_circ
  FROM email_sent
  GROUP BY 1
)

, email_response_agg as (
  SELECT upm_id
  ,SUM(email_opens) AS email_opens
  ,SUM(email_clicks) AS email_clicks
  ,SUM(email_purch) AS email_purch
  ,SUM(email_demand) AS email_demand
  FROM email_calc
  GROUP BY 1
)

, platform_usage as (
  SELECT tct.upm_id
  , CASE WHEN CAST(first_${hivevar:usage_platform}_activity_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7) THEN 1
         ELSE 0
    END AS platform_user
  FROM aud_select_workspace.gck_first_and_last_touchpoint flt
  INNER JOIN test_control_table tct ON flt.upm_id = tct.upm_id  -- Need first/last_send values as activity window
)

---------------------
-- MAIN QUERY
----------------------
SELECT z.test_control
  -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
  ,COUNT(DISTINCT z.upm_id) AS total_members
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) AS buying_members
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) / COUNT(DISTINCT z.upm_id) AS conversion_rate
  ,AVG(demand) AS avg_demand
  ,stddev(demand) AS std_demand
  -- secondary metrics ON order double click
  ,SUM(demand) / SUM(orders) AS AOV
  ,SUM(demand) / SUM(units) AS AUR
  ,SUM(units) / SUM(orders) AS UPT

  -- secondary metrics ON emails
  ,SUM(email_circ) AS email_sent
  ,SUM(email_opens) AS email_opens
  ,SUM(email_clicks) AS email_clicks
  ,SUM(email_purch) AS email_purch
  ,SUM(email_demand) AS email_demand
  ,SUM(email_opens) / SUM(email_circ) AS email_open_rate
  ,SUM(email_clicks) / SUM(email_circ) AS email_click_rate --against sends
  --,SUM(email_clicks) / SUM(email_opens) AS email_click_rate --against clicks
  ,SUM(email_purch) / SUM(email_circ) AS email_conversion_rate

  -- secondary metrics ON site/app engagement
  ,AVG(visits) AS avg_site_app_visits
  ,AVG(PDP_FAVORITE_COUNT) AS avg_PDP_FAVORITE_COUNT
  ,AVG(ADD_TO_CART_COUNT) AS avg_ADD_TO_CART_COUNT
  ,AVG(physical_activity) AS avg_physical_activity

  -- secondary metrics ON margin
  ,(AVG(base_FW_demand) * 0.38 + AVG(base_AP_demand) * 0.28 + AVG(base_EQ_demand) * 0.32
    + AVG(clr_FW_demand) * 0.23 + AVG(base_AP_demand) * 0.17 + AVG(clr_EQ_demand) * 0.12
    + AVG(launch_FW_demand) * 0.25 + AVG(launch_AP_demand) * 0.08 + AVG(launch_EQ_demand) * 0.19
    + AVG(rest_demand) * 0.3
  ) AS avg_margin
  ,AVG(base_FW_demand) AS base_FW_demand
  ,AVG(base_AP_demand) AS base_AP_demand
  ,AVG(base_EQ_demand) AS base_EQ_demand
  ,AVG(clr_FW_demand) AS clr_FW_demand
  ,AVG(clr_AP_demand) AS clr_AP_demand
  ,AVG(clr_EQ_demand) AS clr_EQ_demand
  ,AVG(launch_FW_demand) AS launch_FW_demand
  ,AVG(launch_AP_demand) AS launch_AP_demand
  ,AVG(launch_EQ_demand) AS launch_EQ_demand
  ,AVG(rest_demand) AS rest_demand
    -- secondary metrics ON order history
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS DPB_1_prior
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS DPB_2_prior
    -- optional first usage kpi
  ,SUM(platform_user) AS first_usage

FROM test_control_table z
INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

INNER JOIN mhub mh ON z.upm_id = mh.upm_id
LEFT JOIN member_activities a ON mh.member_id = a.member_id

LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

LEFT JOIN email_delivery_agg cda ON z.upm_id = cda.upm_id
LEFT JOIN email_response_agg cra ON z.upm_id = cra.upm_id

LEFT JOIN platform_usage plt ON z.upm_id = plt.upm_id

GROUP BY 1
ORDER BY 1 DESC;
"""


# Load the data into a PySpark DataFrame
df_spark = spark.sql(ab_data)

# Convert to Pandas DataFrame for further analysis
df = df_spark.toPandas()

# Separate control and test group data
control = df[df['test_control'] == 'control']
test = df[df['test_control'] == 'test']

# Extract input parameters for Demand

# Assuming 'avg_demand' and 'std_demand' were calculated in SQL and are columns in the DataFrame
control_avg_sales = control['avg_demand'].values[0]
test_avg_sales = test['avg_demand'].values[0]
control_std_dev = control['std_demand'].values[0]
test_std_dev = test['std_demand'].values[0]
control_sample_size = control['buying_members'].values[0]
test_sample_size = test['buying_members'].values[0]

# Perform Statistical Analysis for Demand


# Calculate delta and z-value
    """
    The delta, standard error, and z-value, are part of the t-test calculation.

    You typically wouldn't calculate these separately when using a t-test.

    If you need to see these, they're more relevant in the context of manual calculations or when not using a t-test directly.
    """

# Calculate standard error
    """
    A note about standard error:

    * When using stats.ttest_ind with equal_var=False, the test internally computes a version of standard error adjusted for the Welch's t-test.
    * Welch's t-test doesn't assume equal population variances.
    * For educational purposes, you can still calculate a simple standard error of the mean (SEM) for each group.
    * Or calculate the standard error of the difference between two means if you're comparing them directly.
    * Code to do so follows below...

    # Calculate the standard error of the mean for each group
    sem_control = control['avg_demand'].sem()
    sem_test = test['avg_demand'].sem()

    # Calculate the standard error for the difference between two means
    # Again, this formula is for illustrative purposes; the t-test already accounts for this in its calculation
    n_control = len(control['avg_demand'])
    n_test = len(test['avg_demand'])
    std_error_diff = np.sqrt(sem_control**2 / n_control + sem_test**2 / n_test)

    print(f"Standard Error (Control): {sem_control}")
    print(f"Standard Error (Test): {sem_test}")
    print(f"Standard Error of Difference: {std_error_diff}")
    """

# Perform t-test for Avg Demand
t_stat, p_value_demand = stats.ttest_ind(test['avg_demand'], control['avg_demand'], equal_var=False)
confidence_level_demand = 1 - p_value_demand
type_i_error_demand = p_value_demand

# Perform t-test for Conversion Rate
t_stat_conv, p_value_conv = stats.ttest_ind(test['conversion_rate'], control['conversion_rate'], equal_var=False)
confidence_level_conv = 1 - p_value_conv
type_i_error_conv = p_value_conv

# Perform Statistical Analysis for Conversion

# Extracting input parameters for Conversion
control_conv_rate = control['conversion_rate'].values[0]
test_conv_rate = test['conversion_rate'].values[0]
conv_control_sample_size = control['total_members'].values[0]
conv_test_sample_size = test['total_members'].values[0]
lift_conversion_rate = ((test['conversion_rate'].mean() - control['conversion_rate'].mean()) / control['conversion_rate'].mean()) * 100
difference_of_proportions = test_conv_rate - control_conv_rate

# Populate the scorecard DataFrame
scorecard_df = pd.DataFrame({
    'Metric': ['Avg Demand', 'Conversion Rate', 'Lift - Conversion Rate'],
    'P-Value': [p_value_demand, p_value_conv, '-'],
    'Confidence Level': [f"{confidence_level_demand:.2f}", f"{confidence_level_conv:.2f}", '-'],
    'Lift': ['-', '-', f"{lift_conversion_rate:.2f}%"]
})

scorecard_df

"""

# Demand Metrics DataFrame
data_demand = {
    'Metric': ['Avg Demand', 'Std Deviation of Demand', 'Sample Size',
               'Absolute Difference Between Means (Delta)', 'Standard Error', 'Z-Value'],
    'Control Group': [control_avg_sales, control_std_dev, control_sample_size, '', '', ''],
    'Test Group': [test_avg_sales, test_std_dev, test_sample_size, '', '', ''],
    'Value': ['', '', '', delta, standard_error, z_value]
}

demand_metrics_df = pd.DataFrame(data_demand)

# Update rows for Delta, Standard Error, and Z-Value, as they are not group-specific
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Absolute Difference Between Means (Delta)', ['Control Group', 'Test Group', 'Value']] = ['', '', delta]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Standard Error', ['Control Group', 'Test Group', 'Value']] = ['', '', standard_error]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Z-Value', ['Control Group', 'Test Group', 'Value']] = ['', '', z_value]

# Demand Significance DataFrame
data_demand_significance = {
    'Significance Metric': ['Achieved Confidence Level', 'Type I Error (α)'],
    'Value': [confidence_level, type_i_error]
}

demand_significance_df = pd.DataFrame(data_demand_significance)





# Demand Metrics DataFrame
data_demand = {
    'Metric': ['Avg Demand', 'Std Deviation of Demand', 'Sample Size',
               'Absolute Difference Between Means (Delta)', 'Standard Error', 'Z-Value','Achieved Confidence Level', 'Type I Error (α)'],
    'Control Group': [control_avg_sales, control_std_dev, control_sample_size, '', '', '', '', ''],
    'Test Group': [test_avg_sales, test_std_dev, test_sample_size, '', '', '','', ''],
    'Value': ['', '', '', delta, standard_error, z_value, confidence_level, type_i_error]
}

demand_metrics_df = pd.DataFrame(data_demand)

# Update rows for Delta, Standard Error, and Z-Value, as they are in dataframe calcuations
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Absolute Difference Between Means (Delta)', ['Control Group', 'Test Group', 'Value']] = ['', '', delta,]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Standard Error', ['Control Group', 'Test Group', 'Value']] = ['', '', standard_error]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Z-Value', ['Control Group', 'Test Group', 'Value']] = ['', '', z_value]

print(demand_metrics_df)


# Conversion Metrics DataFrame
data_conversion = {
    'Metric': ['Conversion Rate', 'Lift (Absolute % Point Difference)', 'Difference of Proportions',
               'Standard Error', 'Z-Value', 'Achieved Confidence Level', 'Type I Error (α)'],
    'Control Group': [control_conv_rate, '', '', '', '', '', ''],
    'Test Group': [test_conv_rate, '', '', '', '','', ''],
    'Value': ['', lift, difference_of_proportions, conv_standard_error, conv_z_value, conv_confidence_level, conv_type_i_error]
}

conversion_metrics_df = pd.DataFrame(data_conversion)

# Update rows for Delta, Standard Error, and Z-Value, as they are in dataframe calcuations
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Lift (Absolute % Point Difference)', ['Control Group', 'Test Group', 'Value']] = ['', '', lift]
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Difference of Proportions', ['Control Group', 'Test Group', 'Value']] = ['', '', difference_of_proportions]
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Standard Error', ['Control Group', 'Test Group', 'Value']] = ['', '', conv_standard_error]
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Z-Value', ['Control Group', 'Test Group', 'Value']] = ['', '', conv_z_value]

print(conversion_metrics_df)



"""
Update with Stats calcs to convert to Scipy

# Calculate delta
# Calculate standard error
# Calculate z value
# Calculate p-value
# Calculate type I error
# Calculate confidence level
# Calculate statistical significance


import numpy as np
from scipy import stats
import pandas as pd

# Assuming control and test groups are already defined as DataFrames

# Assume 'avg_demand' and 'std_demand' are columns in the DataFrame

# Perform t-test for Avg Demand
t_stat, p_value_demand = stats.ttest_ind(test['avg_demand'], control['avg_demand'], equal_var=False)
confidence_level_demand = 1 - p_value_demand
type_i_error_demand = p_value_demand

# Perform t-test for Conversion Rate (Replace 'conversion_rate' with the actual column names in your DataFrame)
t_stat_conv, p_value_conv = stats.ttest_ind(test['conversion_rate'], control['conversion_rate'], equal_var=False)
confidence_level_conv = 1 - p_value_conv
type_i_error_conv = p_value_conv

# Calculate lift for conversion
lift_conversion_rate = ((test['conversion_rate'].mean() - control['conversion_rate'].mean()) / control['conversion_rate'].mean()) * 100

# The above examples show how to use SciPy for p-value and confidence level calculations.
# For delta, standard error, and z-value, these are part of the t-test calculation, and you typically wouldn't calculate these separately when using a t-test.
# If you need to understand these, they're more relevant in the context of manual calculations or when not using a t-test directly.


"""

### Creating a Pandas dataframe to bypass SQL calculations entirely

In [ ]:

%sql


WITH
 aa_purchase AS (
  SELECT dol.upm_id
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1

  -- AA period purchase. dynamically based on when the member receives their first communication.
)

, demand_bucket AS (
  SELECT tct.upm_id
  , tct.test_control
  , aa.upm_id as upm_id_aa_buyer
  , aa.demand
  , aa.orders
  , aa.units
  , CASE
    WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
    WHEN aa.demand < 50 THEN '000-050'
    WHEN aa.demand < 100 THEN '050-100'
    WHEN aa.demand < 200 THEN '100-200'
    WHEN aa.demand < 500 THEN '200-500'
    WHEN aa.demand < 1000 THEN '500-1000'
    WHEN aa.demand >=1000 THEN '1000+'
    END AS demand_per_buyer_bucket
  , PERCENT_RANK() OVER (
    PARTITION BY test_control
    , CASE
      WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
      WHEN aa.demand < 50 THEN '000-050'
      WHEN aa.demand < 100 THEN '050-100'
      WHEN aa.demand < 200 THEN '100-200'
      WHEN aa.demand < 500 THEN '200-500'
      WHEN aa.demand < 1000 THEN '500-1000'
      WHEN aa.demand >=1000 THEN '1000+'
      END ORDER BY RANDOM(0)
    ) AS percent_rank
  FROM test_control_table tct
  LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
)

, balanced_aud_aa as (
  SELECT DISTINCT upm_id
  FROM demand_bucket db
  -- remove outliers
  WHERE (demand < 2400 OR demand IS NULL)
-- To rebalance, paste text block from template in place of defaults
AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
	)
)

, mhub as (
 SELECT source_id as upm_id
 , member_id
 , preferred_gender
 FROM MEMBER.MEMBER_HUB
 WHERE ctry_2_cd = 'US'
 AND lower(dpa_status) IN ('activate','reactivate')
)

-- calculate a dynamic member activity window for each communication received by a member
-- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
-- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
, member_activities as (
  SELECT mgd.member_id
  ,SUM(logged_in_visits_count) AS visits
  ,SUM(pdp_favorite_count) AS pdp_favorite_count
  ,SUM(add_to_cart_count) AS add_to_cart_count
  ,SUM(physical_activity_count) AS physical_activity
  ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
  FROM test_control_table tct
  INNER JOIN mhub mh ON tct.upm_id = mh.upm_id
  LEFT JOIN aud_select_workspace.member_agg_member_growth_daily mgd ON mh.member_id = mgd.member_id
  WHERE mgd.activity_dt >= tct.campaign_send_dt
    AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
    AND mgd.preferred_retail_geo = 'NA'
    AND mgd.experience_name != 'agnostic'
  GROUP BY 1
)

-- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
, ab_purchase as (
  SELECT dol.upm_id as upm_id_ab_buyer
  , order_dt as order_ts
  , order_nbr
  ,SUM(grd_amt_excl_tax_usd) AS demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
  ,COUNT(DISTINCT order_nbr) AS orders
  ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(first_send_dt,7)
  GROUP BY 1,2,3
)

-- For each member, rank each order by how near it was to the last campaign send
-- purchases must be after the campaign send and within 7 days of the send to be attributed
, aud_order_join as (
SELECT a.*
, b.*
,(bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 as order_send_diff_minutes
,RANK() OVER(PARTITION BY a.upm_id, b.order_ts ORDER BY (bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 ASC) as nearest_order
FROM test_control_table a
LEFT JOIN ab_purchase b ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(a.campaign_send_ts,7))
ORDER BY a.upm_id,a.campaign_send_dt, nearest_order asc
)

-- Aggregate each distinct send, attributing only the nearest order.
-- This prevent double counting of orders when a members receives multiple sends within the measurement window.
, aud_order_agg as (
SELECT upm_id AS upm_id_ab_buyer
  ,SUM(demand) AS demand
  ,SUM(base_FW_demand) AS base_FW_demand
  ,SUM(base_AP_demand) AS base_AP_demand
  ,SUM(base_EQ_demand) AS base_EQ_demand
  ,SUM(clr_FW_demand) AS clr_FW_demand
  ,SUM(clr_AP_demand) AS clr_AP_demand
  ,SUM(clr_EQ_demand) AS clr_EQ_demand
  ,SUM(launch_FW_demand) AS launch_FW_demand
  ,SUM(launch_AP_demand) AS launch_AP_demand
  ,SUM(launch_EQ_demand) AS launch_EQ_demand
  ,SUM(rest_demand) AS rest_demand
  ,SUM(orders) AS orders
  ,SUM(units) AS units
FROM aud_order_join a
WHERE nearest_order = 1
AND demand IS NOT NULL -- additional filter for extraneous records
GROUP BY 1
ORDER BY 1
)

, historical_purchase AS (
  SELECT aoa.upm_id_ab_buyer as upm_id
  , COUNT(DISTINCT order_nbr) as histOrders
  FROM aud_order_agg aoa  -- eligible audience
  INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

-- due to how messages are set up on Bluecore, the campaign_cd is not unique to a particular day. We concat with the message_sent_ts
-- only using the date ignores cases where multiple sends occur on the same day, potentially undercounting
,email_sent AS (
  SELECT fed.upm_id
  , COUNT(DISTINCT concat(campaign_cd, LEFT(message_sent_ts,21))) as email_circ
  FROM COMMS.FACT_EMAIL_DELIVERY fed
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(message_sent_ts AS DATE) between ${hivevar:msmt_start_dt} and ${hivevar:msmt_end_dt}
  AND retail_geo_name = 'NA'
  AND fed.upm_id IS NOT NULL
  GROUP BY upm_id
)

-- returns all the instances of the campaign codes for each member along with the response types and revenue if applicable
, email_response as (
  SELECT DISTINCT fer.upm_id
  , LEFT(response_ts,10) AS response_ts
  , response_type
  , revenue_dollar
  FROM comms.fact_email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(LEFT(response_ts,10) AS DATE) between first_send_dt and DATE_ADD(last_send_dt,7)
  AND retail_geo_name = 'NA'
  AND fer.upm_id IS NOT NULL
 )

-- calculates the the total number of opens, clicks, purchases and last touch attributed demand for each member for the campaign
-- responses are only considered if they occur between when the message was sent and 7 days later.
, email_calc AS (
  SELECT fer.upm_id
  , COUNT(CASE WHEN response_type= 'email open' THEN fer.upm_id ELSE NULL END) AS email_opens
  , COUNT(CASE WHEN response_type='link click' THEN fer.upm_id ELSE NULL END) AS email_clicks
  , COUNT(CASE WHEN response_type='ecommerce-purchase' THEN fer.upm_id ELSE NULL END) AS email_purch
  , SUM(CASE WHEN response_type='ecommerce-purchase' THEN revenue_dollar ELSE 0 END) AS email_demand
  FROM email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE CAST(fer.response_ts as DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7)
  GROUP BY 1
)
-- total number of total messages send for each member during the campaign.
-- For a singular campaign, this should be one (1), for triggers it could be more depending on how the trigger is set up
, email_delivery_agg as (
  SELECT upm_id
  ,SUM(email_circ) AS email_circ
  FROM email_sent
  GROUP BY 1
)

, email_response_agg as (
  SELECT upm_id
  ,SUM(email_opens) AS email_opens
  ,SUM(email_clicks) AS email_clicks
  ,SUM(email_purch) AS email_purch
  ,SUM(email_demand) AS email_demand
  FROM email_calc
  GROUP BY 1
)

, platform_usage as (
  SELECT tct.upm_id
  , CASE WHEN CAST(first_${hivevar:usage_platform}_activity_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7) THEN 1
         ELSE 0
    END AS platform_user
  FROM aud_select_workspace.gck_first_and_last_touchpoint flt
  INNER JOIN test_control_table tct ON flt.upm_id = tct.upm_id  -- Need first/last_send values as activity window
)

---------------------
-- MAIN QUERY
----------------------
SELECT z.upm_id
  , z.test_control

  -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) AS buying_members
  ,demand

  -- secondary metrics ON order double click
  ,orders
  ,units

  -- secondary metrics ON emails
  , email_circ AS email_sent
  , email_opens
  , email_clicks
  , email_purch
  , email_demand

  -- secondary metrics ON site/app engagement
  , visits AS site_app_visits
  , PDP_FAVORITE_COUNT
  , ADD_TO_CART_COUNT
  , physical_activity

  -- secondary metrics ON margin

  -- calculate margin
  /*
  ,(AVG(base_FW_demand) * 0.38 + AVG(base_AP_demand) * 0.28 + AVG(base_EQ_demand) * 0.32
    + AVG(clr_FW_demand) * 0.23 + AVG(base_AP_demand) * 0.17 + AVG(clr_EQ_demand) * 0.12
    + AVG(launch_FW_demand) * 0.25 + AVG(launch_AP_demand) * 0.08 + AVG(launch_EQ_demand) * 0.19
    + AVG(rest_demand) * 0.3
  ) AS avg_margin
  */

  , base_FW_demand
  , base_AP_demand
  , base_EQ_demand
  , clr_FW_demand
  , clr_AP_demand
  , clr_EQ_demand
  , launch_FW_demand
  , launch_AP_demand
  , launch_EQ_demand
  , rest_demand

    -- secondary metrics ON order history
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder

  /*
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS DPB_1_prior
  */

  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder

  /*
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS DPB_2_prior
  */

    -- optional first usage kpi
  , platform_user

FROM test_control_table z
INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

INNER JOIN mhub mh ON z.upm_id = mh.upm_id
LEFT JOIN member_activities a ON mh.member_id = a.member_id

LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

LEFT JOIN email_delivery_agg cda ON z.upm_id = cda.upm_id
LEFT JOIN email_response_agg cra ON z.upm_id = cra.upm_id

LEFT JOIN platform_usage plt ON z.upm_id = plt.upm_id

GROUP BY 1
ORDER BY 1 DESC;



# Load the data into a PySpark DataFrame
df_spark = spark.sql(ab_data)

# Convert to Pandas DataFrame for further analysis
df = df_spark.toPandas()

# Separate control and test group data
control = df[df['test_control'] == 'control']
test = df[df['test_control'] == 'test']

# Extract input parameters for Demand

# Assuming 'avg_demand' and 'std_demand' were calculated in SQL and are columns in the DataFrame
control_avg_sales = control['avg_demand'].values[0]
test_avg_sales = test['avg_demand'].values[0]
control_std_dev = control['std_demand'].values[0]
test_std_dev = test['std_demand'].values[0]
control_sample_size = control['buying_members'].values[0]
test_sample_size = test['buying_members'].values[0]

# Perform Statistical Analysis for Demand


# Calculate delta and z-value
    """
    The delta, standard error, and z-value, are part of the t-test calculation.

    You typically wouldn't calculate these separately when using a t-test.

    If you need to see these, they're more relevant in the context of manual calculations or when not using a t-test directly.
    """

# Calculate standard error
    """
    A note about standard error:

    * When using stats.ttest_ind with equal_var=False, the test internally computes a version of standard error adjusted for the Welch's t-test.
    * Welch's t-test doesn't assume equal population variances.
    * For educational purposes, you can still calculate a simple standard error of the mean (SEM) for each group.
    * Or calculate the standard error of the difference between two means if you're comparing them directly.
    * Code to do so follows below...

    # Calculate the standard error of the mean for each group
    sem_control = control['avg_demand'].sem()
    sem_test = test['avg_demand'].sem()

    # Calculate the standard error for the difference between two means
    # Again, this formula is for illustrative purposes; the t-test already accounts for this in its calculation
    n_control = len(control['avg_demand'])
    n_test = len(test['avg_demand'])
    std_error_diff = np.sqrt(sem_control**2 / n_control + sem_test**2 / n_test)

    print(f"Standard Error (Control): {sem_control}")
    print(f"Standard Error (Test): {sem_test}")
    print(f"Standard Error of Difference: {std_error_diff}")
    """

# Perform t-test for Avg Demand
t_stat, p_value_demand = stats.ttest_ind(test['avg_demand'], control['avg_demand'], equal_var=False)
confidence_level_demand = 1 - p_value_demand
type_i_error_demand = p_value_demand

# Perform t-test for Conversion Rate
t_stat_conv, p_value_conv = stats.ttest_ind(test['conversion_rate'], control['conversion_rate'], equal_var=False)
confidence_level_conv = 1 - p_value_conv
type_i_error_conv = p_value_conv

# Perform Statistical Analysis for Conversion

# Extracting input parameters for Conversion
control_conv_rate = control['conversion_rate'].values[0]
test_conv_rate = test['conversion_rate'].values[0]
conv_control_sample_size = control['total_members'].values[0]
conv_test_sample_size = test['total_members'].values[0]
lift_conversion_rate = ((test['conversion_rate'].mean() - control['conversion_rate'].mean()) / control['conversion_rate'].mean()) * 100
difference_of_proportions = test_conv_rate - control_conv_rate

# Populate the scorecard DataFrame
scorecard_df = pd.DataFrame({
    'Metric': ['Avg Demand', 'Conversion Rate', 'Lift - Conversion Rate'],
    'P-Value': [p_value_demand, p_value_conv, '-'],
    'Confidence Level': [f"{confidence_level_demand:.2f}", f"{confidence_level_conv:.2f}", '-'],
    'Lift': ['-', '-', f"{lift_conversion_rate:.2f}%"]
})

scorecard_df

"""

# Demand Metrics DataFrame
data_demand = {
    'Metric': ['Avg Demand', 'Std Deviation of Demand', 'Sample Size',
               'Absolute Difference Between Means (Delta)', 'Standard Error', 'Z-Value'],
    'Control Group': [control_avg_sales, control_std_dev, control_sample_size, '', '', ''],
    'Test Group': [test_avg_sales, test_std_dev, test_sample_size, '', '', ''],
    'Value': ['', '', '', delta, standard_error, z_value]
}

demand_metrics_df = pd.DataFrame(data_demand)

# Update rows for Delta, Standard Error, and Z-Value, as they are not group-specific
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Absolute Difference Between Means (Delta)', ['Control Group', 'Test Group', 'Value']] = ['', '', delta]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Standard Error', ['Control Group', 'Test Group', 'Value']] = ['', '', standard_error]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Z-Value', ['Control Group', 'Test Group', 'Value']] = ['', '', z_value]

# Demand Significance DataFrame
data_demand_significance = {
    'Significance Metric': ['Achieved Confidence Level', 'Type I Error (α)'],
    'Value': [confidence_level, type_i_error]
}

demand_significance_df = pd.DataFrame(data_demand_significance)





# Demand Metrics DataFrame
data_demand = {
    'Metric': ['Avg Demand', 'Std Deviation of Demand', 'Sample Size',
               'Absolute Difference Between Means (Delta)', 'Standard Error', 'Z-Value','Achieved Confidence Level', 'Type I Error (α)'],
    'Control Group': [control_avg_sales, control_std_dev, control_sample_size, '', '', '', '', ''],
    'Test Group': [test_avg_sales, test_std_dev, test_sample_size, '', '', '','', ''],
    'Value': ['', '', '', delta, standard_error, z_value, confidence_level, type_i_error]
}

demand_metrics_df = pd.DataFrame(data_demand)

# Update rows for Delta, Standard Error, and Z-Value, as they are in dataframe calcuations
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Absolute Difference Between Means (Delta)', ['Control Group', 'Test Group', 'Value']] = ['', '', delta,]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Standard Error', ['Control Group', 'Test Group', 'Value']] = ['', '', standard_error]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Z-Value', ['Control Group', 'Test Group', 'Value']] = ['', '', z_value]

print(demand_metrics_df)


# Conversion Metrics DataFrame
data_conversion = {
    'Metric': ['Conversion Rate', 'Lift (Absolute % Point Difference)', 'Difference of Proportions',
               'Standard Error', 'Z-Value', 'Achieved Confidence Level', 'Type I Error (α)'],
    'Control Group': [control_conv_rate, '', '', '', '', '', ''],
    'Test Group': [test_conv_rate, '', '', '', '','', ''],
    'Value': ['', lift, difference_of_proportions, conv_standard_error, conv_z_value, conv_confidence_level, conv_type_i_error]
}

conversion_metrics_df = pd.DataFrame(data_conversion)

# Update rows for Delta, Standard Error, and Z-Value, as they are in dataframe calcuations
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Lift (Absolute % Point Difference)', ['Control Group', 'Test Group', 'Value']] = ['', '', lift]
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Difference of Proportions', ['Control Group', 'Test Group', 'Value']] = ['', '', difference_of_proportions]
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Standard Error', ['Control Group', 'Test Group', 'Value']] = ['', '', conv_standard_error]
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Z-Value', ['Control Group', 'Test Group', 'Value']] = ['', '', conv_z_value]

print(conversion_metrics_df)



"""
Update with Stats calcs to convert to Scipy

# Calculate delta
# Calculate standard error
# Calculate z value
# Calculate p-value
# Calculate type I error
# Calculate confidence level
# Calculate statistical significance


import numpy as np
from scipy import stats
import pandas as pd

# Assuming control and test groups are already defined as DataFrames

# Assume 'avg_demand' and 'std_demand' are columns in the DataFrame

# Perform t-test for Avg Demand
t_stat, p_value_demand = stats.ttest_ind(test['avg_demand'], control['avg_demand'], equal_var=False)
confidence_level_demand = 1 - p_value_demand
type_i_error_demand = p_value_demand

# Perform t-test for Conversion Rate (Replace 'conversion_rate' with the actual column names in your DataFrame)
t_stat_conv, p_value_conv = stats.ttest_ind(test['conversion_rate'], control['conversion_rate'], equal_var=False)
confidence_level_conv = 1 - p_value_conv
type_i_error_conv = p_value_conv

# Calculate lift for conversion
lift_conversion_rate = ((test['conversion_rate'].mean() - control['conversion_rate'].mean()) / control['conversion_rate'].mean()) * 100

# The above examples show how to use SciPy for p-value and confidence level calculations.
# For delta, standard error, and z-value, these are part of the t-test calculation, and you typically wouldn't calculate these separately when using a t-test.
# If you need to understand these, they're more relevant in the context of manual calculations or when not using a t-test directly.


"""

### SQL Only

*   checked, this code runs.



In [ ]:



 WITH
 aa_purchase AS (SELECT dol.upm_id
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1

  -- AA period purchase. dynamically based on when the member receives their first communication.
)

, demand_bucket AS (
  SELECT tct.upm_id
  , tct.test_control
  , aa.upm_id as upm_id_aa_buyer
  , aa.demand
  , aa.orders
  , aa.units
  , CASE
    WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
    WHEN aa.demand < 50 THEN '000-050'
    WHEN aa.demand < 100 THEN '050-100'
    WHEN aa.demand < 200 THEN '100-200'
    WHEN aa.demand < 500 THEN '200-500'
    WHEN aa.demand < 1000 THEN '500-1000'
    WHEN aa.demand >=1000 THEN '1000+'
    END AS demand_per_buyer_bucket
  , PERCENT_RANK() OVER (
    PARTITION BY test_control
    , CASE
      WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
      WHEN aa.demand < 50 THEN '000-050'
      WHEN aa.demand < 100 THEN '050-100'
      WHEN aa.demand < 200 THEN '100-200'
      WHEN aa.demand < 500 THEN '200-500'
      WHEN aa.demand < 1000 THEN '500-1000'
      WHEN aa.demand >=1000 THEN '1000+'
      END ORDER BY RANDOM(0)
    ) AS percent_rank
  FROM test_control_table tct
  LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
)

, balanced_aud_aa as (
  SELECT DISTINCT upm_id
  FROM demand_bucket db
  -- remove outliers
  WHERE (demand < 2400 OR demand IS NULL)
-- To rebalance, paste text block from template in place of defaults
AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
	)
)

, mhub as (
 SELECT source_id as upm_id
 , member_id
 , preferred_gender
 FROM MEMBER.MEMBER_HUB
 WHERE ctry_2_cd = 'US'
 AND lower(dpa_status) IN ('activate','reactivate')
)

-- calculate a dynamic member activity window for each communication received by a member
-- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
-- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
, member_activities as (
  SELECT mgd.member_id
  ,SUM(logged_in_visits_count) AS visits
  ,SUM(pdp_favorite_count) AS pdp_favorite_count
  ,SUM(add_to_cart_count) AS add_to_cart_count
  ,SUM(physical_activity_count) AS physical_activity
  ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
  FROM test_control_table tct
  INNER JOIN mhub mh ON tct.upm_id = mh.upm_id
  LEFT JOIN aud_select_workspace.member_agg_member_growth_daily mgd ON mh.member_id = mgd.member_id
  WHERE mgd.activity_dt >= tct.campaign_send_dt
    AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
    AND mgd.preferred_retail_geo = 'NA'
    AND mgd.experience_name != 'agnostic'
  GROUP BY 1
)

-- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
, ab_purchase as (
  SELECT dol.upm_id as upm_id_ab_buyer
  , order_dt as order_ts
  , order_nbr
  ,SUM(grd_amt_excl_tax_usd) AS demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
  ,COUNT(DISTINCT order_nbr) AS orders
  ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(first_send_dt,7)
  GROUP BY 1,2,3
)

-- For each member, rank each order by how near it was to the last campaign send
-- purchases must be after the campaign send and within 7 days of the send to be attributed
, aud_order_join as (
SELECT a.*
, b.*
,(bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 as order_send_diff_minutes
,RANK() OVER(PARTITION BY a.upm_id, b.order_ts ORDER BY (bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 ASC) as nearest_order
FROM test_control_table a
LEFT JOIN ab_purchase b ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(a.campaign_send_ts,7))
ORDER BY a.upm_id,a.campaign_send_dt, nearest_order asc
)

-- Aggregate each distinct send, attributing only the nearest order.
-- This prevent double counting of orders when a members receives multiple sends within the measurement window.
, aud_order_agg as (
SELECT upm_id AS upm_id_ab_buyer
  ,SUM(demand) AS demand
  ,SUM(base_FW_demand) AS base_FW_demand
  ,SUM(base_AP_demand) AS base_AP_demand
  ,SUM(base_EQ_demand) AS base_EQ_demand
  ,SUM(clr_FW_demand) AS clr_FW_demand
  ,SUM(clr_AP_demand) AS clr_AP_demand
  ,SUM(clr_EQ_demand) AS clr_EQ_demand
  ,SUM(launch_FW_demand) AS launch_FW_demand
  ,SUM(launch_AP_demand) AS launch_AP_demand
  ,SUM(launch_EQ_demand) AS launch_EQ_demand
  ,SUM(rest_demand) AS rest_demand
  ,SUM(orders) AS orders
  ,SUM(units) AS units
FROM aud_order_join a
WHERE nearest_order = 1
AND demand IS NOT NULL -- additional filter for extraneous records
GROUP BY 1
ORDER BY 1
)

, historical_purchase AS (
  SELECT aoa.upm_id_ab_buyer as upm_id
  , COUNT(DISTINCT order_nbr) as histOrders
  FROM aud_order_agg aoa  -- eligible audience
  INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

-- due to how messages are set up on Bluecore, the campaign_cd is not unique to a particular day. We concat with the message_sent_ts
-- only using the date ignores cases where multiple sends occur on the same day, potentially undercounting
,email_sent AS (
  SELECT fed.upm_id
  , COUNT(DISTINCT concat(campaign_cd, LEFT(message_sent_ts,21))) as email_circ
  FROM COMMS.FACT_EMAIL_DELIVERY fed
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(message_sent_ts AS DATE) between ${hivevar:msmt_start_dt} and ${hivevar:msmt_end_dt}
  AND retail_geo_name = 'NA'
  AND fed.upm_id IS NOT NULL
  GROUP BY upm_id
)

-- returns all the instances of the campaign codes for each member along with the response types and revenue if applicable
, email_response as (
  SELECT DISTINCT fer.upm_id
  , LEFT(response_ts,10) AS response_ts
  , response_type
  , revenue_dollar
  FROM comms.fact_email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(LEFT(response_ts,10) AS DATE) between first_send_dt and DATE_ADD(last_send_dt,7)
  AND retail_geo_name = 'NA'
  AND fer.upm_id IS NOT NULL
 )

-- calculates the the total number of opens, clicks, purchases and last touch attributed demand for each member for the campaign
-- responses are only considered if they occur between when the message was sent and 7 days later.
, email_calc AS (
  SELECT fer.upm_id
  , COUNT(CASE WHEN response_type= 'email open' THEN fer.upm_id ELSE NULL END) AS email_opens
  , COUNT(CASE WHEN response_type='link click' THEN fer.upm_id ELSE NULL END) AS email_clicks
  , COUNT(CASE WHEN response_type='ecommerce-purchase' THEN fer.upm_id ELSE NULL END) AS email_purch
  , SUM(CASE WHEN response_type='ecommerce-purchase' THEN revenue_dollar ELSE 0 END) AS email_demand
  FROM email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE CAST(fer.response_ts as DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7)
  GROUP BY 1
)
-- total number of total messages send for each member during the campaign.
-- For a singular campaign, this should be one (1), for triggers it could be more depending on how the trigger is set up
, email_delivery_agg as (
  SELECT upm_id
  ,SUM(email_circ) AS email_circ
  FROM email_sent
  GROUP BY 1
)

, email_response_agg as (
  SELECT upm_id
  ,SUM(email_opens) AS email_opens
  ,SUM(email_clicks) AS email_clicks
  ,SUM(email_purch) AS email_purch
  ,SUM(email_demand) AS email_demand
  FROM email_calc
  GROUP BY 1
)

, platform_usage as (
  SELECT tct.upm_id
  , CASE WHEN CAST(first_${hivevar:usage_platform}_activity_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7) THEN 1
         ELSE 0
    END AS platform_user
  FROM aud_select_workspace.gck_first_and_last_touchpoint flt
  INNER JOIN test_control_table tct ON flt.upm_id = tct.upm_id  -- Need first/last_send values as activity window
)

---------------------
-- MAIN QUERY
----------------------
SELECT z.upm_id
  , z.test_control

  -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser

  ,SUM(demand)

  -- secondary metrics ON order double click

  ,SUM(orders)
  ,SUM(units)

  -- secondary metrics ON emails

  , SUM(email_circ) AS email_sent
  , SUM(email_opens)
  , SUM(email_clicks)
  , SUM(email_purch)
  , SUM(email_demand)

  -- secondary metrics ON site/app engagement

  , SUM(visits) AS site_app_visits
  , SUM(PDP_FAVORITE_COUNT)
  , SUM(ADD_TO_CART_COUNT)
  , SUM(physical_activity)

  -- secondary metrics ON margin

  , SUM(base_FW_demand)
  , SUM(base_AP_demand)
  , SUM(base_EQ_demand)
  , SUM(clr_FW_demand)
  , SUM(clr_AP_demand)
  , SUM(clr_EQ_demand)
  , SUM(launch_FW_demand)
  , SUM(launch_AP_demand)
  , SUM(launch_EQ_demand)
  , SUM(rest_demand)

  -- optional first usage kpi

  , SUM(platform_user) AS platform_user

  -- secondary metrics ON order history

  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder


FROM test_control_table z
INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

INNER JOIN mhub mh ON z.upm_id = mh.upm_id
LEFT JOIN member_activities a ON mh.member_id = a.member_id

LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

LEFT JOIN email_delivery_agg cda ON z.upm_id = cda.upm_id
LEFT JOIN email_response_agg cra ON z.upm_id = cra.upm_id

LEFT JOIN platform_usage plt ON z.upm_id = plt.upm_id

GROUP BY 1,2
ORDER BY 1 DESC;

In [ ]:
-- metrics to be aggregated in numpy later:

  ,COUNT(DISTINCT abp.upm_id_ab_buyer) AS buying_members

  -- calculate margin

  ,(AVG(base_FW_demand) * 0.38 + AVG(base_AP_demand) * 0.28 + AVG(base_EQ_demand) * 0.32
    + AVG(clr_FW_demand) * 0.23 + AVG(base_AP_demand) * 0.17 + AVG(clr_EQ_demand) * 0.12
    + AVG(launch_FW_demand) * 0.25 + AVG(launch_AP_demand) * 0.08 + AVG(launch_EQ_demand) * 0.19
    + AVG(rest_demand) * 0.3
  ) AS avg_margin

  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS DPB_1_prior

  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS DPB_2_prior

#### Integrating Updated SQL into new dataframe

In [ ]:
%python

# Execute SQL Query
ab_data = """

 WITH
 aa_purchase AS (SELECT dol.upm_id
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1

  -- AA period purchase. dynamically based on when the member receives their first communication.
)

, demand_bucket AS (
  SELECT tct.upm_id
  , tct.test_control
  , aa.upm_id as upm_id_aa_buyer
  , aa.demand
  , aa.orders
  , aa.units
  , CASE
    WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
    WHEN aa.demand < 50 THEN '000-050'
    WHEN aa.demand < 100 THEN '050-100'
    WHEN aa.demand < 200 THEN '100-200'
    WHEN aa.demand < 500 THEN '200-500'
    WHEN aa.demand < 1000 THEN '500-1000'
    WHEN aa.demand >=1000 THEN '1000+'
    END AS demand_per_buyer_bucket
  , PERCENT_RANK() OVER (
    PARTITION BY test_control
    , CASE
      WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
      WHEN aa.demand < 50 THEN '000-050'
      WHEN aa.demand < 100 THEN '050-100'
      WHEN aa.demand < 200 THEN '100-200'
      WHEN aa.demand < 500 THEN '200-500'
      WHEN aa.demand < 1000 THEN '500-1000'
      WHEN aa.demand >=1000 THEN '1000+'
      END ORDER BY RANDOM(0)
    ) AS percent_rank
  FROM test_control_table tct
  LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
)

, balanced_aud_aa as (
  SELECT DISTINCT upm_id
  FROM demand_bucket db
  -- remove outliers
  WHERE (demand < 2400 OR demand IS NULL)
-- To rebalance, paste text block from template in place of defaults
AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
	)
)

, mhub as (
 SELECT source_id as upm_id
 , member_id
 , preferred_gender
 FROM MEMBER.MEMBER_HUB
 WHERE ctry_2_cd = 'US'
 AND lower(dpa_status) IN ('activate','reactivate')
)

-- calculate a dynamic member activity window for each communication received by a member
-- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
-- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
, member_activities as (
  SELECT mgd.member_id
  ,SUM(logged_in_visits_count) AS visits
  ,SUM(pdp_favorite_count) AS pdp_favorite_count
  ,SUM(add_to_cart_count) AS add_to_cart_count
  ,SUM(physical_activity_count) AS physical_activity
  ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
  FROM test_control_table tct
  INNER JOIN mhub mh ON tct.upm_id = mh.upm_id
  LEFT JOIN aud_select_workspace.member_agg_member_growth_daily mgd ON mh.member_id = mgd.member_id
  WHERE mgd.activity_dt >= tct.campaign_send_dt
    AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
    AND mgd.preferred_retail_geo = 'NA'
    AND mgd.experience_name != 'agnostic'
  GROUP BY 1
)

-- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
, ab_purchase as (
  SELECT dol.upm_id as upm_id_ab_buyer
  , order_dt as order_ts
  , order_nbr
  ,SUM(grd_amt_excl_tax_usd) AS demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
  ,COUNT(DISTINCT order_nbr) AS orders
  ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(first_send_dt,7)
  GROUP BY 1,2,3
)

-- For each member, rank each order by how near it was to the last campaign send
-- purchases must be after the campaign send and within 7 days of the send to be attributed
, aud_order_join as (
SELECT a.*
, b.*
,(bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 as order_send_diff_minutes
,RANK() OVER(PARTITION BY a.upm_id, b.order_ts ORDER BY (bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 ASC) as nearest_order
FROM test_control_table a
LEFT JOIN ab_purchase b ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(a.campaign_send_ts,7))
ORDER BY a.upm_id,a.campaign_send_dt, nearest_order asc
)

-- Aggregate each distinct send, attributing only the nearest order.
-- This prevent double counting of orders when a members receives multiple sends within the measurement window.
, aud_order_agg as (
SELECT upm_id AS upm_id_ab_buyer
  ,SUM(demand) AS demand
  ,SUM(base_FW_demand) AS base_FW_demand
  ,SUM(base_AP_demand) AS base_AP_demand
  ,SUM(base_EQ_demand) AS base_EQ_demand
  ,SUM(clr_FW_demand) AS clr_FW_demand
  ,SUM(clr_AP_demand) AS clr_AP_demand
  ,SUM(clr_EQ_demand) AS clr_EQ_demand
  ,SUM(launch_FW_demand) AS launch_FW_demand
  ,SUM(launch_AP_demand) AS launch_AP_demand
  ,SUM(launch_EQ_demand) AS launch_EQ_demand
  ,SUM(rest_demand) AS rest_demand
  ,SUM(orders) AS orders
  ,SUM(units) AS units
FROM aud_order_join a
WHERE nearest_order = 1
AND demand IS NOT NULL -- additional filter for extraneous records
GROUP BY 1
ORDER BY 1
)

, historical_purchase AS (
  SELECT aoa.upm_id_ab_buyer as upm_id
  , COUNT(DISTINCT order_nbr) as histOrders
  FROM aud_order_agg aoa  -- eligible audience
  INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

-- due to how messages are set up on Bluecore, the campaign_cd is not unique to a particular day. We concat with the message_sent_ts
-- only using the date ignores cases where multiple sends occur on the same day, potentially undercounting
,email_sent AS (
  SELECT fed.upm_id
  , COUNT(DISTINCT concat(campaign_cd, LEFT(message_sent_ts,21))) as email_circ
  FROM COMMS.FACT_EMAIL_DELIVERY fed
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(message_sent_ts AS DATE) between ${hivevar:msmt_start_dt} and ${hivevar:msmt_end_dt}
  AND retail_geo_name = 'NA'
  AND fed.upm_id IS NOT NULL
  GROUP BY upm_id
)

-- returns all the instances of the campaign codes for each member along with the response types and revenue if applicable
, email_response as (
  SELECT DISTINCT fer.upm_id
  , LEFT(response_ts,10) AS response_ts
  , response_type
  , revenue_dollar
  FROM comms.fact_email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(LEFT(response_ts,10) AS DATE) between first_send_dt and DATE_ADD(last_send_dt,7)
  AND retail_geo_name = 'NA'
  AND fer.upm_id IS NOT NULL
 )

-- calculates the the total number of opens, clicks, purchases and last touch attributed demand for each member for the campaign
-- responses are only considered if they occur between when the message was sent and 7 days later.
, email_calc AS (
  SELECT fer.upm_id
  , COUNT(CASE WHEN response_type= 'email open' THEN fer.upm_id ELSE NULL END) AS email_opens
  , COUNT(CASE WHEN response_type='link click' THEN fer.upm_id ELSE NULL END) AS email_clicks
  , COUNT(CASE WHEN response_type='ecommerce-purchase' THEN fer.upm_id ELSE NULL END) AS email_purch
  , SUM(CASE WHEN response_type='ecommerce-purchase' THEN revenue_dollar ELSE 0 END) AS email_demand
  FROM email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE CAST(fer.response_ts as DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7)
  GROUP BY 1
)
-- total number of total messages send for each member during the campaign.
-- For a singular campaign, this should be one (1), for triggers it could be more depending on how the trigger is set up
, email_delivery_agg as (
  SELECT upm_id
  ,SUM(email_circ) AS email_circ
  FROM email_sent
  GROUP BY 1
)

, email_response_agg as (
  SELECT upm_id
  ,SUM(email_opens) AS email_opens
  ,SUM(email_clicks) AS email_clicks
  ,SUM(email_purch) AS email_purch
  ,SUM(email_demand) AS email_demand
  FROM email_calc
  GROUP BY 1
)

, platform_usage as (
  SELECT tct.upm_id
  , CASE WHEN CAST(first_${hivevar:usage_platform}_activity_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7) THEN 1
         ELSE 0
    END AS platform_user
  FROM aud_select_workspace.gck_first_and_last_touchpoint flt
  INNER JOIN test_control_table tct ON flt.upm_id = tct.upm_id  -- Need first/last_send values as activity window
)

---------------------
-- MAIN QUERY
----------------------
SELECT z.upm_id
  , z.test_control

  -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
  ,SUM(demand) AS demand

  -- secondary metrics ON order double click
  ,SUM(orders) AS orders
  ,SUM(units) AS units

  -- secondary metrics ON emails
  , SUM(email_circ) AS email_sent
  , SUM(email_opens) AS email_opens
  , SUM(email_clicks) AS email_clicks
  , SUM(email_purch) AS email_purch
  , SUM(email_demand) AS email_demand

  -- secondary metrics ON site/app engagement
  , SUM(visits) AS site_app_visits
  , SUM(PDP_FAVORITE_COUNT) AS PDP_FAVORITE_COUNT
  , SUM(ADD_TO_CART_COUNT) AS ADD_TO_CART_COUNT
  , SUM(physical_activity) AS physical_activity

  -- secondary metrics ON margin
  , SUM(base_FW_demand) AS base_FW_demand
  , SUM(base_AP_demand) AS base_AP_demand
  , SUM(base_EQ_demand) AS base_EQ_demand
  , SUM(clr_FW_demand) AS clr_FW_demand
  , SUM(clr_AP_demand) AS clr_AP_demand
  , SUM(clr_EQ_demand) AS clr_EQ_demand
  , SUM(launch_FW_demand) AS launch_FW_demand
  , SUM(launch_AP_demand) AS launch_AP_demand
  , SUM(launch_EQ_demand) AS launch_EQ_demand
  , SUM(rest_demand) AS rest_demand

   -- optional first usage kpi
  , SUM(platform_user) AS platform_user

    -- secondary metrics ON order history
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder


FROM test_control_table z
INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

INNER JOIN mhub mh ON z.upm_id = mh.upm_id
LEFT JOIN member_activities a ON mh.member_id = a.member_id

LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

LEFT JOIN email_delivery_agg cda ON z.upm_id = cda.upm_id
LEFT JOIN email_response_agg cra ON z.upm_id = cra.upm_id

LEFT JOIN platform_usage plt ON z.upm_id = plt.upm_id

GROUP BY 1,2
ORDER BY 1 DESC;
"""


# Load the data into a PySpark DataFrame
df_spark = spark.sql(ab_data)

# Convert to Pandas DataFrame for further analysis
df = df_spark.toPandas()

# Separate control and test group data
control = df[df['test_control'] == 'control']
test = df[df['test_control'] == 'test']


## Extracting input parameters for Demand


# Calculate average sales (demand) for control and test groups
control_avg_demand = control['demand'].mean()
test_avg_demand = test['demand'].mean()

# Calculate standard deviation of demand for control and test groups
control_std_dev = control['demand'].std()
test_std_dev = test['demand'].std()

# Calculate the # of buying members for the control and test groups
control_sample_size = control[control['orders'] > 0]['upm_id'].nunique()
test_sample_size = test[test['orders'] > 0]['upm_id'].nunique()



## Perform Statistical Analysis for Demand


# Calculate delta and z-value
    """
    The delta, standard error, and z-value, are part of the t-test calculation.

    You typically wouldn't calculate these separately when using a t-test.

    If you need to see these, they're more relevant in the context of manual calculations or when not using a t-test directly.
    """

# Calculate standard error
    """
    A note about standard error:

    * When using stats.ttest_ind with equal_var=False, the test internally computes a version of standard error adjusted for the Welch's t-test.
    * Welch's t-test doesn't assume equal population variances.
    * For educational purposes, you can still calculate a simple standard error of the mean (SEM) for each group.
    * Or calculate the standard error of the difference between two means if you're comparing them directly.
    * Code to do so follows below...

    # Calculate the standard error of the mean for each group
    sem_control = control['avg_demand'].sem()
    sem_test = test['avg_demand'].sem()

    # Calculate the standard error for the difference between two means
    # Again, this formula is for illustrative purposes; the t-test already accounts for this in its calculation
    n_control = len(control['avg_demand'])
    n_test = len(test['avg_demand'])
    std_error_diff = np.sqrt(sem_control**2 / n_control + sem_test**2 / n_test)

    print(f"Standard Error (Control): {sem_control}")
    print(f"Standard Error (Test): {sem_test}")
    print(f"Standard Error of Difference: {std_error_diff}")
    """

# Perform t-test for Avg Demand
t_stat, p_value_demand = stats.ttest_ind(test['demand'], control['demand'], equal_var=False)
confidence_level_demand = 1 - p_value_demand
type_i_error_demand = p_value_demand

print(f"T-statistic for Demand: {t_stat}")
print(f"P-value for Demand: {p_value_demand}")
print(f"Confidence Level for Demand: {confidence_level_demand*100:.2f}%")
print(f"Type I Error for Demand: {type_i_error_demand:.4f}")


## Extracting input parameters for Conversion


# Total count of all unique members in the test and control groups
conv_control_sample_size = control['upm_id'].nunique()
conv_test_sample_size = test['upm_id'].nunique()

# Calculate conversion rate for the test and control groups (total buying members / total members)
control_conv_rate = control_sample_size / conv_control_sample_size if conv_control_sample_size > 0 else 0
test_conv_rate = test_sample_size / conv_test_sample_size if conv_test_sample_size > 0 else 0

# Calculate lift for the conversion rate
lift = abs(test_conv_rate - control_conv_rate)

# Calculate difference of proportions
difference_of_proportions = test_conv_rate - control_conv_rate

print(f"Control Conversion Rate: {control_conv_rate*100:.2f}%")
print(f"Test Conversion Rate: {test_conv_rate*100:.2f}%")
print(f"Absolute Lift in Conversion Rate: {lift*100:.2f}%")
print(f"Difference_of_proportions: {difference_of_proportions:.2f}")



## Perform Statistical Analysis for Conversion

# Add a 'buyer' column to both test and control DataFrames
# 'buyer' = 1 if 'orders' > 0, else 0
control['buyer'] = (control['orders'] > 0).astype(int)
test['buyer'] = (test['orders'] > 0).astype(int)

# Perform t-test for Conversion Rate
t_stat_conversion, p_value_conversion = stats.ttest_ind(test['buyer'], control['buyer'], equal_var=False)

# Calculate confidence level and Type I error for Conversion Rate
confidence_level_conversion = 1 - p_value_conversion
type_i_error_conversion = p_value_conversion

print(f"T-statistic for Conversion: {t_stat_conversion}")
print(f"P-value for Conversion: {p_value_conversion}")
print(f"Confidence Level for Conversion: {confidence_level_conversion*100:.2f}%")
print(f"Type I Error for Conversion: {type_i_error_conversion:.4f}")


## Populate the scorecard DataFrame

scorecard_df = pd.DataFrame({
    'Metric': ['Avg Demand', 'Conversion Rate', 'Lift - Conversion Rate'],
    'P-Value': [p_value_demand, p_value_conv, '-'],
    'Confidence Level': [f"{confidence_level_demand:.2f}", f"{confidence_level_conv:.2f}", '-'],
    'Lift': ['-', '-', f"{lift_conversion_rate:.2f}%"]
})

scorecard_df

"""

# Demand Metrics DataFrame
data_demand = {
    'Metric': ['Avg Demand', 'Std Deviation of Demand', 'Sample Size',
               'Absolute Difference Between Means (Delta)', 'Standard Error', 'Z-Value'],
    'Control Group': [control_avg_sales, control_std_dev, control_sample_size, '', '', ''],
    'Test Group': [test_avg_sales, test_std_dev, test_sample_size, '', '', ''],
    'Value': ['', '', '', delta, standard_error, z_value]
}

demand_metrics_df = pd.DataFrame(data_demand)

# Update rows for Delta, Standard Error, and Z-Value, as they are not group-specific
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Absolute Difference Between Means (Delta)', ['Control Group', 'Test Group', 'Value']] = ['', '', delta]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Standard Error', ['Control Group', 'Test Group', 'Value']] = ['', '', standard_error]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Z-Value', ['Control Group', 'Test Group', 'Value']] = ['', '', z_value]

# Demand Significance DataFrame
data_demand_significance = {
    'Significance Metric': ['Achieved Confidence Level', 'Type I Error (α)'],
    'Value': [confidence_level, type_i_error]
}

demand_significance_df = pd.DataFrame(data_demand_significance)





# Demand Metrics DataFrame
data_demand = {
    'Metric': ['Avg Demand', 'Std Deviation of Demand', 'Sample Size',
               'Absolute Difference Between Means (Delta)', 'Standard Error', 'Z-Value','Achieved Confidence Level', 'Type I Error (α)'],
    'Control Group': [control_avg_sales, control_std_dev, control_sample_size, '', '', '', '', ''],
    'Test Group': [test_avg_sales, test_std_dev, test_sample_size, '', '', '','', ''],
    'Value': ['', '', '', delta, standard_error, z_value, confidence_level, type_i_error]
}

demand_metrics_df = pd.DataFrame(data_demand)

# Update rows for Delta, Standard Error, and Z-Value, as they are in dataframe calcuations
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Absolute Difference Between Means (Delta)', ['Control Group', 'Test Group', 'Value']] = ['', '', delta,]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Standard Error', ['Control Group', 'Test Group', 'Value']] = ['', '', standard_error]
demand_metrics_df.loc[demand_metrics_df['Metric'] == 'Z-Value', ['Control Group', 'Test Group', 'Value']] = ['', '', z_value]

print(demand_metrics_df)


# Conversion Metrics DataFrame
data_conversion = {
    'Metric': ['Conversion Rate', 'Lift (Absolute % Point Difference)', 'Difference of Proportions',
               'Standard Error', 'Z-Value', 'Achieved Confidence Level', 'Type I Error (α)'],
    'Control Group': [control_conv_rate, '', '', '', '', '', ''],
    'Test Group': [test_conv_rate, '', '', '', '','', ''],
    'Value': ['', lift, difference_of_proportions, conv_standard_error, conv_z_value, conv_confidence_level, conv_type_i_error]
}

conversion_metrics_df = pd.DataFrame(data_conversion)

# Update rows for Delta, Standard Error, and Z-Value, as they are in dataframe calcuations
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Lift (Absolute % Point Difference)', ['Control Group', 'Test Group', 'Value']] = ['', '', lift]
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Difference of Proportions', ['Control Group', 'Test Group', 'Value']] = ['', '', difference_of_proportions]
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Standard Error', ['Control Group', 'Test Group', 'Value']] = ['', '', conv_standard_error]
conversion_metrics_df.loc[conversion_metrics_df['Metric'] == 'Z-Value', ['Control Group', 'Test Group', 'Value']] = ['', '', conv_z_value]

print(conversion_metrics_df)



"""


#### Missing Metrics

Add these to dataframe later

In [ ]:
 # Calculate secondary metrics ON order double click

  ,SUM(demand) / SUM(orders) AS AOV
  ,SUM(demand) / SUM(units) AS AUR
  ,SUM(units) / SUM(orders) AS UPT

# secondary metrics ON emails

 ,SUM(email_opens) / SUM(email_circ) AS email_open_rate
  ,SUM(email_clicks) / SUM(email_circ) AS email_click_rate --against sends
  --,SUM(email_clicks) / SUM(email_opens) AS email_click_rate --against clicks
  ,SUM(email_purch) / SUM(email_circ) AS email_conversion_rate

# secondary metrics ON margin

  ,(AVG(base_FW_demand) * 0.38 + AVG(base_AP_demand) * 0.28 + AVG(base_EQ_demand) * 0.32
    + AVG(clr_FW_demand) * 0.23 + AVG(base_AP_demand) * 0.17 + AVG(clr_EQ_demand) * 0.12
    + AVG(launch_FW_demand) * 0.25 + AVG(launch_AP_demand) * 0.08 + AVG(launch_EQ_demand) * 0.19
    + AVG(rest_demand) * 0.3
  ) AS avg_margin

# secondary metrics ON order history

  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS DPB_1_prior
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS DPB_2_prior

In [ ]:
%python

# Execute SQL Query
ab_data = """

 WITH
 aa_purchase AS (SELECT dol.upm_id
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1

  -- AA period purchase. dynamically based on when the member receives their first communication.
)

, demand_bucket AS (
  SELECT tct.upm_id
  , tct.test_control
  , aa.upm_id as upm_id_aa_buyer
  , aa.demand
  , aa.orders
  , aa.units
  , CASE
    WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
    WHEN aa.demand < 50 THEN '000-050'
    WHEN aa.demand < 100 THEN '050-100'
    WHEN aa.demand < 200 THEN '100-200'
    WHEN aa.demand < 500 THEN '200-500'
    WHEN aa.demand < 1000 THEN '500-1000'
    WHEN aa.demand >=1000 THEN '1000+'
    END AS demand_per_buyer_bucket
  , PERCENT_RANK() OVER (
    PARTITION BY test_control
    , CASE
      WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
      WHEN aa.demand < 50 THEN '000-050'
      WHEN aa.demand < 100 THEN '050-100'
      WHEN aa.demand < 200 THEN '100-200'
      WHEN aa.demand < 500 THEN '200-500'
      WHEN aa.demand < 1000 THEN '500-1000'
      WHEN aa.demand >=1000 THEN '1000+'
      END ORDER BY RANDOM(0)
    ) AS percent_rank
  FROM test_control_table tct
  LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
)

, balanced_aud_aa as (
  SELECT DISTINCT upm_id
  FROM demand_bucket db
  -- remove outliers
  WHERE (demand < 2400 OR demand IS NULL)
-- To rebalance, paste text block from template in place of defaults
AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
	)
)

, mhub as (
 SELECT source_id as upm_id
 , member_id
 , preferred_gender
 FROM MEMBER.MEMBER_HUB
 WHERE ctry_2_cd = 'US'
 AND lower(dpa_status) IN ('activate','reactivate')
)

/*
-- calculate a dynamic member activity window for each communication received by a member
-- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
-- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
, member_activities as (
  SELECT mgd.member_id
  ,SUM(logged_in_visits_count) AS visits
  ,SUM(pdp_favorite_count) AS pdp_favorite_count
  ,SUM(add_to_cart_count) AS add_to_cart_count
  ,SUM(physical_activity_count) AS physical_activity
  ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
  FROM test_control_table tct
  INNER JOIN mhub mh ON tct.upm_id = mh.upm_id
  LEFT JOIN aud_select_workspace.member_agg_member_growth_daily mgd ON mh.member_id = mgd.member_id
  WHERE mgd.activity_dt >= tct.campaign_send_dt
    AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
    AND mgd.preferred_retail_geo = 'NA'
    AND mgd.experience_name != 'agnostic'
  GROUP BY 1
)
*/

-- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
, ab_purchase as (
  SELECT dol.upm_id as upm_id_ab_buyer
  , order_dt as order_ts
  , order_nbr
  ,SUM(grd_amt_excl_tax_usd) AS demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
  ,COUNT(DISTINCT order_nbr) AS orders
  ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(first_send_dt,7)
  GROUP BY 1,2,3
)

-- For each member, rank each order by how near it was to the last campaign send
-- purchases must be after the campaign send and within 7 days of the send to be attributed
, aud_order_join as (
SELECT a.*
, b.*
,(bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 as order_send_diff_minutes
,RANK() OVER(PARTITION BY a.upm_id, b.order_ts ORDER BY (bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 ASC) as nearest_order
FROM test_control_table a
LEFT JOIN ab_purchase b ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(a.campaign_send_ts,7))
ORDER BY a.upm_id,a.campaign_send_dt, nearest_order asc
)

-- Aggregate each distinct send, attributing only the nearest order.
-- This prevent double counting of orders when a members receives multiple sends within the measurement window.
, aud_order_agg as (
SELECT upm_id AS upm_id_ab_buyer
  ,SUM(demand) AS demand
  ,SUM(base_FW_demand) AS base_FW_demand
  ,SUM(base_AP_demand) AS base_AP_demand
  ,SUM(base_EQ_demand) AS base_EQ_demand
  ,SUM(clr_FW_demand) AS clr_FW_demand
  ,SUM(clr_AP_demand) AS clr_AP_demand
  ,SUM(clr_EQ_demand) AS clr_EQ_demand
  ,SUM(launch_FW_demand) AS launch_FW_demand
  ,SUM(launch_AP_demand) AS launch_AP_demand
  ,SUM(launch_EQ_demand) AS launch_EQ_demand
  ,SUM(rest_demand) AS rest_demand
  ,SUM(orders) AS orders
  ,SUM(units) AS units
FROM aud_order_join a
WHERE nearest_order = 1
AND demand IS NOT NULL -- additional filter for extraneous records
GROUP BY 1
ORDER BY 1
)

, historical_purchase AS (
  SELECT aoa.upm_id_ab_buyer as upm_id
  , COUNT(DISTINCT order_nbr) as histOrders
  FROM aud_order_agg aoa  -- eligible audience
  INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

-- due to how messages are set up on Bluecore, the campaign_cd is not unique to a particular day. We concat with the message_sent_ts
-- only using the date ignores cases where multiple sends occur on the same day, potentially undercounting
,email_sent AS (
  SELECT fed.upm_id
  , COUNT(DISTINCT concat(campaign_cd, LEFT(message_sent_ts,21))) as email_circ
  FROM COMMS.FACT_EMAIL_DELIVERY fed
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(message_sent_ts AS DATE) between ${hivevar:msmt_start_dt} and ${hivevar:msmt_end_dt}
  AND retail_geo_name = 'NA'
  AND fed.upm_id IS NOT NULL
  GROUP BY upm_id
)

-- returns all the instances of the campaign codes for each member along with the response types and revenue if applicable
, email_response as (
  SELECT DISTINCT fer.upm_id
  , LEFT(response_ts,10) AS response_ts
  , response_type
  , revenue_dollar
  FROM comms.fact_email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(LEFT(response_ts,10) AS DATE) between first_send_dt and DATE_ADD(last_send_dt,7)
  AND retail_geo_name = 'NA'
  AND fer.upm_id IS NOT NULL
 )

-- calculates the the total number of opens, clicks, purchases and last touch attributed demand for each member for the campaign
-- responses are only considered if they occur between when the message was sent and 7 days later.
, email_calc AS (
  SELECT fer.upm_id
  , COUNT(CASE WHEN response_type= 'email open' THEN fer.upm_id ELSE NULL END) AS email_opens
  , COUNT(CASE WHEN response_type='link click' THEN fer.upm_id ELSE NULL END) AS email_clicks
  , COUNT(CASE WHEN response_type='ecommerce-purchase' THEN fer.upm_id ELSE NULL END) AS email_purch
  , SUM(CASE WHEN response_type='ecommerce-purchase' THEN revenue_dollar ELSE 0 END) AS email_demand
  FROM email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE CAST(fer.response_ts as DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7)
  GROUP BY 1
)
-- total number of total messages send for each member during the campaign.
-- For a singular campaign, this should be one (1), for triggers it could be more depending on how the trigger is set up
, email_delivery_agg as (
  SELECT upm_id
  ,SUM(email_circ) AS email_circ
  FROM email_sent
  GROUP BY 1
)

, email_response_agg as (
  SELECT upm_id
  ,SUM(email_opens) AS email_opens
  ,SUM(email_clicks) AS email_clicks
  ,SUM(email_purch) AS email_purch
  ,SUM(email_demand) AS email_demand
  FROM email_calc
  GROUP BY 1
)

, platform_usage as (
  SELECT tct.upm_id
  , CASE WHEN CAST(first_${hivevar:usage_platform}_activity_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7) THEN 1
         ELSE 0
    END AS platform_user
  FROM aud_select_workspace.gck_first_and_last_touchpoint flt
  INNER JOIN test_control_table tct ON flt.upm_id = tct.upm_id  -- Need first/last_send values as activity window
)

---------------------
-- MAIN QUERY
----------------------
SELECT z.upm_id
  , z.test_control

  -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
  ,SUM(demand) AS demand

  -- secondary metrics ON order double click
  ,SUM(orders) AS orders
  ,SUM(units) AS units

  -- secondary metrics ON emails
  , SUM(email_circ) AS email_sent
  , SUM(email_opens) AS email_opens
  , SUM(email_clicks) AS email_clicks
  , SUM(email_purch) AS email_purch
  , SUM(email_demand) AS email_demand

  -- secondary metrics ON site/app engagement
 -- , SUM(visits) AS site_app_visits
  , SUM(PDP_FAVORITE_COUNT) AS PDP_FAVORITE_COUNT
  , SUM(ADD_TO_CART_COUNT) AS ADD_TO_CART_COUNT
  , SUM(physical_activity) AS physical_activity

  -- secondary metrics ON margin
  , SUM(base_FW_demand) AS base_FW_demand
  , SUM(base_AP_demand) AS base_AP_demand
  , SUM(base_EQ_demand) AS base_EQ_demand
  , SUM(clr_FW_demand) AS clr_FW_demand
  , SUM(clr_AP_demand) AS clr_AP_demand
  , SUM(clr_EQ_demand) AS clr_EQ_demand
  , SUM(launch_FW_demand) AS launch_FW_demand
  , SUM(launch_AP_demand) AS launch_AP_demand
  , SUM(launch_EQ_demand) AS launch_EQ_demand
  , SUM(rest_demand) AS rest_demand

   -- optional first usage kpi
  , SUM(platform_user) AS platform_user

    -- secondary metrics ON order history
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder


FROM test_control_table z
INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

INNER JOIN mhub mh ON z.upm_id = mh.upm_id
LEFT JOIN member_activities a ON mh.member_id = a.member_id

LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

LEFT JOIN email_delivery_agg cda ON z.upm_id = cda.upm_id
LEFT JOIN email_response_agg cra ON z.upm_id = cra.upm_id

LEFT JOIN platform_usage plt ON z.upm_id = plt.upm_id

GROUP BY 1,2
ORDER BY 1 DESC;
"""


# Load the data into a PySpark DataFrame
df_spark = spark.sql(ab_data)

# Convert to Pandas DataFrame for further analysis
df = df_spark.toPandas()

# Separate control and test group data
control = df[df['test_control'] == 'control']
test = df[df['test_control'] == 'test']


## Extracting input parameters for Demand


# Calculate average sales (demand) for control and test groups
# And, replace NaN values in 'demand' with 0

control_avg_demand = control['demand'].fillna(0).mean()
test_avg_demand = test['demand'].fillna(0).mean()
# control_avg_demand = control['demand'].mean()
# test_avg_demand = test['demand'].mean()

# Calculate standard deviation of demand for control and test groups
control_std_dev = control['demand'].fillna(0).std()
test_std_dev = test['demand'].fillna(0).std()

# Calculate the # of buying members for the control and test groups
control_sample_size = control[control['orders'] > 0]['upm_id'].nunique()
test_sample_size = test[test['orders'] > 0]['upm_id'].nunique()



## Perform Statistical Analysis for Demand


# Calculate delta and z-value
"""
The delta, standard error, and z-value, are part of the t-test calculation.

You typically wouldn't calculate these separately when using a t-test.

If you need to see these, they're more relevant in the context of manual calculations or when not using a t-test directly.
"""

# Calculate standard error
"""
A note about standard error:

* When using stats.ttest_ind with equal_var=False, the test internally computes a version of standard error adjusted for the Welch's t-test.
* Welch's t-test doesn't assume equal population variances.
* For educational purposes, you can still calculate a simple standard error of the mean (SEM) for each group.
* Or calculate the standard error of the difference between two means if you're comparing them directly.
* Code to do so follows below...

# Calculate the standard error of the mean for each group
sem_control = control['avg_demand'].sem()
sem_test = test['avg_demand'].sem()

# Calculate the standard error for the difference between two means
# Again, this formula is for illustrative purposes; the t-test already accounts for this in its calculation
n_control = len(control['avg_demand'])
n_test = len(test['avg_demand'])
std_error_diff = np.sqrt(sem_control**2 / n_control + sem_test**2 / n_test)

print(f"Standard Error (Control): {sem_control}")
print(f"Standard Error (Test): {sem_test}")
print(f"Standard Error of Difference: {std_error_diff}")
"""

# Perform t-test for Avg Demand
t_stat, p_value_demand = stats.ttest_ind(test['demand'], control['demand'], equal_var=False)
confidence_level_demand = 1 - p_value_demand
type_i_error_demand = p_value_demand

print(f"T-statistic for Demand: {t_stat}")
print(f"P-value for Demand: {p_value_demand}")
print(f"Confidence Level for Demand: {confidence_level_demand*100:.2f}%")
print(f"Type I Error for Demand: {type_i_error_demand:.4f}")


## Extracting input parameters for Conversion


# Total count of all unique members in the test and control groups
conv_control_sample_size = control['upm_id'].nunique()
conv_test_sample_size = test['upm_id'].nunique()

# Calculate conversion rate for the test and control groups (total buying members / total members)
control_conv_rate = control_sample_size / conv_control_sample_size if conv_control_sample_size > 0 else 0
test_conv_rate = test_sample_size / conv_test_sample_size if conv_test_sample_size > 0 else 0

# Calculate lift for the conversion rate
lift = abs(test_conv_rate - control_conv_rate)

# Calculate difference of proportions
difference_of_proportions = test_conv_rate - control_conv_rate

print(f"Control Conversion Rate: {control_conv_rate*100:.2f}%")
print(f"Test Conversion Rate: {test_conv_rate*100:.2f}%")
print(f"Absolute Lift in Conversion Rate: {lift*100:.2f}%")
print(f"Difference_of_proportions: {difference_of_proportions:.2f}")



## Perform Statistical Analysis for Conversion

# Add a 'buyer' column to both test and control DataFrames
# 'buyer' = 1 if 'orders' > 0, else 0
control['buyer'] = (control['orders'] > 0).astype(int)
test['buyer'] = (test['orders'] > 0).astype(int)

# Perform t-test for Conversion Rate
t_stat_conversion, p_value_conversion = stats.ttest_ind(test['buyer'], control['buyer'], equal_var=False)

# Calculate confidence level and Type I error for Conversion Rate
confidence_level_conversion = 1 - p_value_conversion
type_i_error_conversion = p_value_conversion

print(f"T-statistic for Conversion: {t_stat_conversion}")
print(f"P-value for Conversion: {p_value_conversion}")
print(f"Confidence Level for Conversion: {confidence_level_conversion*100:.2f}%")
print(f"Type I Error for Conversion: {type_i_error_conversion:.4f}")


#### Updated calculations for ttest

In [ ]:
# handling members with a demand of 0

# Replace NaN values in 'demand' with 0
control['demand'] = control['demand'].fillna(0)
test['demand'] = test['demand'].fillna(0)

# Perform the t-test
t_stat, p_value_demand = stats.ttest_ind(test['demand'], control['demand'], equal_var=False)


# handling members with a demand of 0, ignoring those values

# Extracting input parameters for Demand with non-NaN values
control_demand_non_nan = control[control['demand'].notna()]['demand']
test_demand_non_nan = test[test['demand'].notna()]['demand']

# Calculate average sales (demand) for control and test groups without NaN values
control_avg_demand = control_demand_non_nan.mean()
test_avg_demand = test_demand_non_nan.mean()

control_avg_demand = control['demand'].notna().mean()
test_avg_demand = test['demand'].notna().mean()


# Calculate standard deviation of demand for control and test groups without NaN values
control_std_dev = control['demand'].notna().std()
test_std_dev = test['demand'].notna().std()

# Perform t-test for Avg Demand excluding NaN values
t_stat_demand, p_value_demand = stats.ttest_ind(test_demand_non_nan, control_demand_non_nan, equal_var=False, nan_policy='omit')



#### Build the Scorecard dataframe

In [ ]:
%python
## Extracting input parameters for Demand


# Calculate average sales (demand) for control and test groups
# And, replace NaN values in 'demand' with 0

# control_avg_demand = control['demand'].notna().mean()
# test_avg_demand = test['demand'].notna().mean()
control_avg_demand = control['demand'].mean()
test_avg_demand = test['demand'].mean()

# Calculate standard deviation of demand for control and test groups
# control_std_dev = control['demand'].notna().std()
# test_std_dev = test['demand'].notna().std()
control_std_dev = control['demand'].std()
test_std_dev = test['demand'].std()

# Calculate the # of buying members for the control and test groups
control_sample_size = control[control['orders'] > 0]['upm_id'].nunique()
test_sample_size = test[test['orders'] > 0]['upm_id'].nunique()



## Perform Statistical Analysis for Demand


# Calculate delta and z-value
"""
The delta, standard error, and z-value, are part of the t-test calculation.

You typically wouldn't calculate these separately when using a t-test.

If you need to see these, they're more relevant in the context of manual calculations or when not using a t-test directly.
"""

# Calculate standard error
"""
A note about standard error:

* When using stats.ttest_ind with equal_var=False, the test internally computes a version of standard error adjusted for the Welch's t-test.
* Welch's t-test doesn't assume equal population variances.
* For educational purposes, you can still calculate a simple standard error of the mean (SEM) for each group.
* Or, calculate the standard error of the difference between two means if you're comparing them directly.
* Code to do so follows below...

# Calculate the standard error of the mean for each group
sem_control = control['avg_demand'].sem()
sem_test = test['avg_demand'].sem()

# Calculate the standard error for the difference between two means
# Again, this formula is for illustrative purposes; the t-test already accounts for this in its calculation
n_control = len(control['avg_demand'])
n_test = len(test['avg_demand'])
std_error_diff = np.sqrt(sem_control**2 / n_control + sem_test**2 / n_test)

print(f"Standard Error (Control): {sem_control}")
print(f"Standard Error (Test): {sem_test}")
print(f"Standard Error of Difference: {std_error_diff}")
"""

# Perform t-test for Avg Demand, omitting 'nan' values
# t_stat, p_value_demand = stats.ttest_ind(test['demand'].fillna(0), control['demand'].fillna(0), equal_var=False)
t_stat_demand, p_value_demand = stats.ttest_ind(test['demand'], control['demand'], equal_var=False, nan_policy='omit')
confidence_level_demand = 1 - p_value_demand
type_i_error_demand = p_value_demand


print(f"Demand Sample Size - Control: {control_sample_size}")
print(f"Demand Sample Size - Test: {test_sample_size}")
print(f"Average Demand - Control: {control_avg_demand:.2f}")
print(f"Average Demand - Test: {test_avg_demand:.2f}")
print(f"Standard Deviation - Test: {test_std_dev:.2f}")
print(f"Standard Deviation - Control: {control_std_dev:.2f}")

print(f"T-statistic for Demand: {t_stat_demand}")
print(f"P-value for Demand: {p_value_demand}")
print(f"Confidence Level for Demand: {confidence_level_demand*100:.2f}%")
print(f"Type I Error for Demand: {type_i_error_demand:.4f}")


## Extracting input parameters for Conversion


# Total count of all unique members in the test and control groups
conv_control_sample_size = control['upm_id'].nunique()
conv_test_sample_size = test['upm_id'].nunique()

# Calculate conversion rate for the test and control groups (total buying members / total members)
control_conv_rate = control_sample_size / conv_control_sample_size if conv_control_sample_size > 0 else 0
test_conv_rate = test_sample_size / conv_test_sample_size if conv_test_sample_size > 0 else 0

# Calculate lift for the conversion rate
lift = abs(test_conv_rate - control_conv_rate)

# Calculate difference of proportions
difference_of_proportions = test_conv_rate - control_conv_rate

print(f"Control Conversion Rate: {control_conv_rate*100:.2f}%")
print(f"Test Conversion Rate: {test_conv_rate*100:.2f}%")
print(f"Absolute Lift in Conversion Rate: {lift*100:.4f}%")
print(f"Difference_of_proportions: {difference_of_proportions:.4f}")



## Perform Statistical Analysis for Conversion

# Add a 'buyer' column to both test and control DataFrames
# Make sure these are not views but standalone DataFrames to avoid SettingWithCopyWarning
control = control.copy()
test = test.copy()
# Now safely add the 'buyer' column
control.loc[:, 'buyer'] = (control['orders'] > 0).astype(int) # 'buyer' = 1 if 'orders' > 0, else 0
test.loc[:, 'buyer'] = (test['orders'] > 0).astype(int)

# Perform t-test for Conversion Rate
t_stat_conversion, p_value_conversion = stats.ttest_ind(test['buyer'], control['buyer'], equal_var=False)

# Calculate confidence level and Type I error for Conversion Rate
confidence_level_conversion = 1 - p_value_conversion
type_i_error_conversion = p_value_conversion

print(f"T-statistic for Conversion: {t_stat_conversion:.4f}")
print(f"P-value for Conversion: {p_value_conversion:.4f}")
print(f"Confidence Level for Conversion: {confidence_level_conversion*100:.2f}%")
print(f"Type I Error for Conversion: {type_i_error_conversion:.4f}")

In [ ]:
# Assuming control and test are defined DataFrames
# Make sure these are not views but standalone DataFrames to avoid SettingWithCopyWarning
control = control.copy()
test = test.copy()

# Now safely add the 'buyer' column
control.loc[:, 'buyer'] = (control['orders'] > 0).astype(int)
test.loc[:, 'buyer'] = (test['orders'] > 0).astype(int)


### AA OG Code

In [ ]:
--------------------------------------------------------------------------------------------------------------------------------------------
-- AA Period
--------------------------------------------------------------------------------------------------------------------------------------------
WITH

test_control_table AS (
    SELECT aud.upm_id
    , test_control
    , MIN(message_sent_ts) as first_send
    FROM (
        (SELECT distinct fed.upm_id, 'test' as test_control, message_sent_ts from ${hivevar:aud_table_tst} tst INNER JOIN comms.fact_email_delivery fed ON tst.upm_id = fed.upm_id AND campaign_geo_name = 'NA' AND campaign_cd LIKE ${hivevar:camp_cd_tst_str} AND CAST(message_sent_ts AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND ${hivevar:msmt_end_dt})
      UNION
        (SELECT distinct upm_id, 'control' as test_control, ${hivevar:msmt_start_dt} as first_send from ${hivevar:aud_table_ctl})
    ) aud
    ANTI JOIN gwan13.bot_master_nike_com bot ON aud.upm_id = bot.upm_id
    ANTI JOIN aud_select_workspace.reseller_new slr ON aud.upm_id = slr.upm_user_id
    GROUP BY 1,2
)

-- buying member level: purchase behavior
, dol AS (
  SELECT
    dol.upm_id
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM dtc_integrated.dtc_digital_order_line dol
  INNER JOIN test_control_table tct on dol.upm_id = tct.upm_id
  WHERE dol.region_key = 1
    AND dol.rec_excl_ind = 0
    AND dol.ttl_demand_ind = 1
    AND UPPER(univ_cat_desc) <> 'CONVERSE'
    AND dol.upm_id IS NOT NULL
    AND CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send,31) AND DATE_SUB(first_send,1)
    AND (dol.rtn_qty = 0 or dol.rtn_qty is NULL)
  GROUP BY 1
)

-- Bucket each member into a demand group, then randomly rank them within that group
, demand_bucket AS (
  SELECT z.*
  , d.upm_id as upm_id_buyer
  , d.demand
  , CASE WHEN d.demand = 0 OR d.demand IS NULL THEN 'non-buyer'
         WHEN d.demand < 50 THEN '000-050'
         WHEN d.demand < 100 THEN '050-100'
         WHEN d.demand < 200 THEN '100-200'
         WHEN d.demand < 500 THEN '200-500'
         WHEN d.demand < 1000 THEN '500-1000'
         WHEN d.demand >=1000 THEN '1000+'
    END AS demand_per_buyer_bucket
  , d.orders
  , d.units
  , PERCENT_RANK() OVER (PARTITION BY
                      test_control
                    , CASE
                      WHEN d.demand = 0 OR d.demand is NULL THEN 'non-buyer'
                      WHEN d.demand < 50 THEN '000-050'
                      WHEN d.demand < 100 THEN '050-100'
                      WHEN d.demand < 200 THEN '100-200'
                      WHEN d.demand < 500 THEN '200-500'
                      WHEN d.demand < 1000 THEN '500-1000'
                      WHEN d.demand >=1000 THEN '1000+'
                      END ORDER BY RAND(0)) AS percent_rank
  FROM test_control_table z
  LEFT JOIN dol d ON z.upm_id = d.upm_id
)

-----------------
-- MAIN QUERY
-----------------

SELECT
  test_control
  -- primary metrics on incremental revenue: conversion rate, average demand per purchaser
  ,COUNT(DISTINCT upm_id) AS total_members
  ,COUNT(DISTINCT upm_id_buyer) AS buying_members
  ,COUNT(DISTINCT upm_id_buyer) / COUNT(DISTINCT upm_id) AS conversion_rate
  ,AVG(demand) AS avg_demand
  ,STDDEV(demand) AS std_demand

  -- secondary metrics ON order double click
  ,SUM(demand) / SUM(orders) AS AOV
  ,SUM(demand) / SUM(units) AS AUR
  ,SUM(units) / SUM(orders) AS UPT


FROM demand_bucket db
WHERE (demand < 2400 OR demand IS NULL)  -- remove outliers
-- To rebalance, paste text block from template in place of defaults
AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
	)

GROUP BY 1
ORDER BY 1 DESC;

### AB OG Code

In [ ]:
-----------------------------------------------------------------------------------------------------------------------------------
-- AB Period
-----------------------------------------------------------------------------------------------------------------------------------
WITH

tc_camp_sends AS (
    SELECT upm_id
    , CASE WHEN final_exposure_or_holdout like ${hivevar:node_tst_str} THEN 'test'
           WHEN final_exposure_or_holdout like ${hivevar:node_ctl_str} THEN 'control'
      ELSE 'na' END AS test_control
    , MIN(LEFT(timestamp,10)) AS campaign_send_dt
    , MIN(CAST(LEFT(timestamp,19) AS timestamp)) AS campaign_send_ts
    FROM ${hivevar:aud_table} aud
    ANTI JOIN gwan13.bot_master_nike_com bot ON aud.upm_id = bot.upm_id
    ANTI JOIN aud_select_workspace.resellers slr ON aud.upm_id = slr.upm_id
    WHERE timestamp BETWEEN ${hivevar:msmt_start_dt} AND ${hivevar:msmt_end_dt}
    GROUP BY 1,2
)

, tc_first_last_send AS (
  SELECT upm_id
  , MIN(campaign_send_dt) as first_send_dt
  , MAX(campaign_send_dt) as last_send_dt
  FROM tc_camp_sends
  GROUP BY 1
)

, test_control_table AS (
    SELECT aud.*
    , fls.first_send_dt
    , fls.last_send_dt
    , LEAD(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt) AS next_send_dt
    , LAG(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt) AS previous_send_dt
    , (int(to_timestamp(campaign_send_dt)) - int(to_timestamp(LEAD(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt))))/86400 AS difference_next_send_days
    , (int(to_timestamp(campaign_send_dt)) - int(to_timestamp(LAG(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt))))/86400 AS difference_previous_send_days
    FROM tc_camp_sends aud
    INNER JOIN tc_first_last_send fls ON aud.upm_id = fls.upm_id
    ANTI JOIN gwan13.bot_master_nike_com bot ON aud.upm_id = bot.upm_id
    ANTI JOIN aud_select_workspace.resellers slr ON aud.upm_id = slr.upm_id
    AND test_control != 'na'
)

, view_dol AS (
  SELECT dol.upm_id
  , order_nbr
  , order_dt
  , origl_ordered_qty
  , grd_amt_excl_tax_usd
  , line_of_business_desc
  , univ_div_desc
  , first_send_dt
  , last_send_dt
  FROM DTC_INTEGRATED.DTC_DIGITAL_ORDER_LINE dol
  INNER JOIN test_control_table tct ON dol.upm_id = tct.upm_id
  WHERE region_key = 1
    AND rec_excl_ind = 0
    AND ttl_demand_ind = 1
    AND univ_cat_desc <> 'Converse'
    AND dol.upm_id IS NOT NULL
    AND CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_ADD(last_send_dt,7)
    AND (rtn_qty = 0 or rtn_qty IS NULL)
)

-- AA period purchase. dynamically based on when the member receives their first communication. Only returns demand for members in the
, aa_purchase AS (
  SELECT dol.upm_id
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

, demand_bucket AS (
  SELECT tct.upm_id
  , tct.test_control
  , aa.upm_id as upm_id_aa_buyer
  , aa.demand
  , aa.orders
  , aa.units
  , CASE
    WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
    WHEN aa.demand < 50 THEN '000-050'
    WHEN aa.demand < 100 THEN '050-100'
    WHEN aa.demand < 200 THEN '100-200'
    WHEN aa.demand < 500 THEN '200-500'
    WHEN aa.demand < 1000 THEN '500-1000'
    WHEN aa.demand >=1000 THEN '1000+'
    END AS demand_per_buyer_bucket
  , PERCENT_RANK() OVER (
    PARTITION BY test_control
    , CASE
      WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
      WHEN aa.demand < 50 THEN '000-050'
      WHEN aa.demand < 100 THEN '050-100'
      WHEN aa.demand < 200 THEN '100-200'
      WHEN aa.demand < 500 THEN '200-500'
      WHEN aa.demand < 1000 THEN '500-1000'
      WHEN aa.demand >=1000 THEN '1000+'
      END ORDER BY RANDOM(0)
    ) AS percent_rank
  FROM test_control_table tct
  LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
)

, balanced_aud_aa as (
  SELECT DISTINCT upm_id
  FROM demand_bucket db
  -- remove outliers
  WHERE (demand < 2400 OR demand IS NULL)
-- To rebalance, paste text block from template in place of defaults
AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 0.98994)
OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 0.99803)
OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 0.99741)
OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 0.99965)
OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 0.99982)
OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 0.99515)
OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 0.98176))

, mhub as (
 SELECT source_id as upm_id
 , member_id
 , preferred_gender
 FROM MEMBER.MEMBER_HUB
 WHERE ctry_2_cd = 'US'
 AND lower(dpa_status) IN ('activate','reactivate')
)

-- calculate a dynamic member activity window for each communication received by a member
-- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
-- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
, member_activities as (
  SELECT mgd.member_id
  ,SUM(logged_in_visits_count) AS visits
  ,SUM(pdp_favorite_count) AS pdp_favorite_count
  ,SUM(add_to_cart_count) AS add_to_cart_count
  ,SUM(physical_activity_count) AS physical_activity
  ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
  ,SUM(eod_chat_count) AS NEOD_Session
  FROM test_control_table tct
  INNER JOIN mhub mh ON tct.upm_id = mh.upm_id
  LEFT JOIN aud_select_workspace.member_agg_member_growth_daily mgd ON mh.member_id = mgd.member_id
  WHERE mgd.activity_dt >= tct.campaign_send_dt
    AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
    AND mgd.preferred_retail_geo = 'NA'
    AND mgd.experience_name != 'agnostic'
  GROUP BY 1
)

-- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
, ab_purchase as (
  SELECT dol.upm_id as upm_id_ab_buyer
  , order_dt as order_ts
  , order_nbr
  ,SUM(grd_amt_excl_tax_usd) AS demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
  ,COUNT(DISTINCT order_nbr) AS orders
  ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(first_send_dt,7)
  GROUP BY 1,2,3
)

-- For each member, rank each order by how near it was to the last campaign send
-- purchases must be after the campaign send and within 7 days of the send to be attributed
, aud_order_join as (
SELECT a.*
, b.*
,(bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 as order_send_diff_minutes
,RANK() OVER(PARTITION BY a.upm_id, b.order_ts ORDER BY (bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 ASC) as nearest_order
FROM test_control_table a
LEFT JOIN ab_purchase b ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(a.campaign_send_ts,7))
ORDER BY a.upm_id,a.campaign_send_dt, nearest_order asc
)

-- Aggregate each distinct send, attributing only the nearest order.
-- This prevent double counting of orders when a members receives multiple sends within the measurement window.
, aud_order_agg as (
SELECT upm_id AS upm_id_ab_buyer
  ,SUM(demand) AS demand
  ,SUM(base_FW_demand) AS base_FW_demand
  ,SUM(base_AP_demand) AS base_AP_demand
  ,SUM(base_EQ_demand) AS base_EQ_demand
  ,SUM(clr_FW_demand) AS clr_FW_demand
  ,SUM(clr_AP_demand) AS clr_AP_demand
  ,SUM(clr_EQ_demand) AS clr_EQ_demand
  ,SUM(launch_FW_demand) AS launch_FW_demand
  ,SUM(launch_AP_demand) AS launch_AP_demand
  ,SUM(launch_EQ_demand) AS launch_EQ_demand
  ,SUM(rest_demand) AS rest_demand
  ,SUM(orders) AS orders
  ,SUM(units) AS units
FROM aud_order_join a
WHERE nearest_order = 1
AND demand IS NOT NULL -- additional filter for extraneous records
GROUP BY 1
ORDER BY 1
)

, historical_purchase AS (
  SELECT aoa.upm_id_ab_buyer as upm_id
  , COUNT(DISTINCT order_nbr) as histOrders
  FROM aud_order_agg aoa  -- eligible audience
  INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

-- due to how messages are set up on Bluecore, the campaign_cd is not unique to a particular day. We concat with the message_sent_ts
-- only using the date ignores cases where multiple sends occur on the same day, potentially undercounting
,email_sent AS (
    SELECT fed.upm_id
    , COUNT(DISTINCT concat(campaign_cd, LEFT(message_sent_ts,21))) as email_circ
    FROM COMMS.FACT_EMAIL_DELIVERY fed
    WHERE campaign_cd LIKE ${hivevar:camp_cd_str}  -- exclude control codes if using holdout
    AND CAST(LEFT(message_sent_ts,10) AS DATE) between ${hivevar:msmt_start_dt} and ${hivevar:msmt_end_dt}
    AND retail_geo_name = 'NA'
    AND fed.upm_id IS NOT NULL
    GROUP BY upm_id
)

-- returns all the instances of the campaign codes for each member along with the response types and revenue if applicable
, email_response as (
SELECT DISTINCT fer.upm_id
, LEFT(response_ts,10) AS response_ts
, response_type
, revenue_dollar
FROM comms.fact_email_response fer
INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
WHERE campaign_cd LIKE ${hivevar:camp_cd_str}   -- exclude control codes if using holdout
AND LEFT(response_ts,10) between first_send_dt and DATE_ADD(last_send_dt,7)
AND retail_geo_name = 'NA'
AND fer.upm_id IS NOT NULL
 )

-- calculates the the total number of opens, clicks, purchases and last touch attributed demand for each member for the campaign
-- responses are only considered if they occur between when the message was sent and 7 days later.
, email_calc AS (
    SELECT fer.upm_id
    , COUNT(CASE WHEN response_type= 'email open' THEN fer.upm_id ELSE NULL END) AS email_opens
    , COUNT(CASE WHEN response_type='link click' THEN fer.upm_id ELSE NULL END) AS email_clicks
    , COUNT(CASE WHEN response_type='ecommerce-purchase' THEN fer.upm_id ELSE NULL END) AS email_purch
    , SUM(CASE WHEN response_type='ecommerce-purchase' THEN revenue_dollar ELSE 0 END) AS email_demand
    FROM email_response fer
    INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
    WHERE CAST(fer.response_ts as DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7)
    GROUP BY 1
)
-- total number of total messages send for each member during the campaign.
-- For a singular campaign, this should be one (1), for triggers it could be more depending on how the trigger is set up
, email_delivery_agg as (
  SELECT upm_id
  ,SUM(email_circ) AS email_circ
  FROM email_sent
  GROUP BY 1
)

, email_response_agg as (
  SELECT upm_id
  ,SUM(email_opens) AS email_opens
  ,SUM(email_clicks) AS email_clicks
  ,SUM(email_purch) AS email_purch
  ,SUM(email_demand) AS email_demand
  FROM email_calc
  GROUP BY 1
)

, platform_usage as (
  SELECT tct.upm_id
  , CASE WHEN CAST(first_${hivevar:usage_platform}_activity_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7) THEN 1
         ELSE 0
    END AS platform_user
  FROM aud_select_workspace.gck_first_and_last_touchpoint flt
  INNER JOIN test_control_table tct ON flt.upm_id = tct.upm_id  -- Need first/last_send values as activity window
)

---------------------
-- MAIN QUERY
----------------------
SELECT z.test_control
  -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
  ,COUNT(DISTINCT z.upm_id) AS total_members
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) AS buying_members
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) / COUNT(DISTINCT z.upm_id) AS conversion_rate
  ,AVG(demand) AS avg_demand
  ,stddev(demand) AS std_demand
  -- secondary metrics ON order double click
  ,SUM(demand) / SUM(orders) AS AOV
  ,SUM(demand) / SUM(units) AS AUR
  ,SUM(units) / SUM(orders) AS UPT

  -- secondary metrics ON emails
  ,SUM(email_circ) AS email_sent
  ,SUM(email_opens) AS email_opens
  ,SUM(email_clicks) AS email_clicks
  ,SUM(email_purch) AS email_purch
  ,SUM(email_demand) AS email_demand
  ,(CASE WHEN sum(email_circ) = 0 THEN 0 ELSE SUM(email_opens) / SUM(email_circ) END) AS email_open_rate
  ,(CASE WHEN sum(email_opens) = 0 THEN 0 ELSE SUM(email_clicks) / SUM(email_circ) END) AS email_click_rate --against sends
  --,(CASE WHEN sum(email_opens) = 0 THEN 0 ELSE SUM(email_clicks) / SUM(email_opens) END) AS email_click_rate --against clicks
  ,(CASE WHEN sum(email_circ) = 0 THEN 0 ELSE SUM(email_purch) / SUM(email_circ) END) AS email_conversion_rate

  -- secondary metrics ON site/app engagement
  ,AVG(visits) AS avg_site_app_visits
  ,AVG(PDP_FAVORITE_COUNT) AS avg_PDP_FAVORITE_COUNT
  ,AVG(ADD_TO_CART_COUNT) AS avg_ADD_TO_CART_COUNT
  ,AVG(physical_activity) AS avg_physical_activity

  -- secondary metrics ON margin
  ,(AVG(base_FW_demand) * 0.38 + AVG(base_AP_demand) * 0.28 + AVG(base_EQ_demand) * 0.32
    + AVG(clr_FW_demand) * 0.23 + AVG(base_AP_demand) * 0.17 + AVG(clr_EQ_demand) * 0.12
    + AVG(launch_FW_demand) * 0.25 + AVG(launch_AP_demand) * 0.08 + AVG(launch_EQ_demand) * 0.19
    + AVG(rest_demand) * 0.3
  ) AS avg_margin
  ,AVG(base_FW_demand) AS base_FW_demand
  ,AVG(base_AP_demand) AS base_AP_demand
  ,AVG(base_EQ_demand) AS base_EQ_demand
  ,AVG(clr_FW_demand) AS clr_FW_demand
  ,AVG(clr_AP_demand) AS clr_AP_demand
  ,AVG(clr_EQ_demand) AS clr_EQ_demand
  ,AVG(launch_FW_demand) AS launch_FW_demand
  ,AVG(launch_AP_demand) AS launch_AP_demand
  ,AVG(launch_EQ_demand) AS launch_EQ_demand
  ,AVG(rest_demand) AS rest_demand
    -- secondary metrics ON order history
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS DPB_1_prior
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS DPB_2_prior
    -- optional first usage kpi
  ,SUM(platform_user) AS first_usage

/*  -- Optional KPI's  --
  ,SUM(platform_user)/COUNT(DISTINCT z.upm_id) AS first_usage_pct
  ,SUM(NEOD_Session) AS NEOD_Session
*/

FROM test_control_table z
INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

INNER JOIN mhub mh ON z.upm_id = mh.upm_id
LEFT JOIN member_activities a ON mh.member_id = a.member_id

LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

LEFT JOIN email_delivery_agg cda ON z.upm_id = cda.upm_id
LEFT JOIN email_response_agg cra ON z.upm_id = cra.upm_id

LEFT JOIN platform_usage plt ON z.upm_id = plt.upm_id

WHERE (demand < 600 OR demand IS NULL)

GROUP BY 1
ORDER BY 1 DESC;

### Full AB Code - V5

In [ ]:
%python

# Execute SQL Query
ab_data = """
-----------------------------------------------------------------------------------------------------------------------------------
-- AB Period
-----------------------------------------------------------------------------------------------------------------------------------
WITH

tc_first_last_send AS (
    SELECT *
    FROM (
         (SELECT distinct tst.upm_id
          , 'test' as test_control
          , MIN(LEFT(message_sent_ts,10)) AS campaign_send_dt
          , MIN(LEFT(message_sent_ts,21)) AS campaign_send_ts
          , MIN(LEFT(message_sent_ts,10)) as first_send_dt
          , MAX(LEFT(message_sent_ts,10)) as last_send_dt
          FROM ${hivevar:aud_table_tst} tst
          INNER JOIN comms.fact_email_delivery fed ON tst.upm_id = fed.upm_id AND campaign_geo_name = 'NA' AND campaign_cd LIKE ${hivevar:camp_cd_tst_str} AND CAST(message_sent_ts AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND ${hivevar:msmt_end_dt}
          GROUP BY 1,2
          )
      UNION
        (SELECT distinct upm_id
         , 'control' as test_control
         , ${hivevar:msmt_start_dt} AS campaign_send_dt
         , to_timestamp(${hivevar:msmt_start_dt}, 'yyyy-MM-dd') AS campaign_send_ts
         , ${hivevar:msmt_start_dt} as first_send_dt
         , ${hivevar:msmt_start_dt} as last_send_dt
         from ${hivevar:aud_table_ctl}
         )
    )
)

, test_control_table AS (
    SELECT *
    , LEAD(a.campaign_send_dt) OVER (PARTITION BY a.upm_id ORDER BY a.campaign_send_dt) AS next_send_dt
    , LAG(a.campaign_send_dt) OVER (PARTITION BY a.upm_id ORDER BY a.campaign_send_dt) AS previous_send_dt
    , (int(to_timestamp(a.campaign_send_dt)) - int(to_timestamp(LEAD(a.campaign_send_dt) OVER (PARTITION BY a.upm_id ORDER BY a.campaign_send_dt))))/86400 AS difference_next_send_days
    , (int(to_timestamp(a.campaign_send_dt)) - int(to_timestamp(LAG(a.campaign_send_dt) OVER (PARTITION BY a.upm_id ORDER BY a.campaign_send_dt))))/86400 AS difference_previous_send_days
    FROM tc_first_last_send a
    ANTI JOIN gwan13.bot_master_nike_com bot ON a.upm_id = bot.upm_id
    ANTI JOIN aud_select_workspace.reseller_new slr ON a.upm_id = slr.upm_user_id
)

, view_dol AS (
  SELECT dol.upm_id
  , order_nbr
  , order_dt
  , origl_ordered_qty
  , grd_amt_excl_tax_usd
  , line_of_business_desc
  , univ_div_desc
  , first_send_dt
  , last_send_dt
  FROM DTC_INTEGRATED.DTC_DIGITAL_ORDER_LINE dol
  INNER JOIN test_control_table tct ON dol.upm_id = tct.upm_id
  WHERE region_key = 1
    AND rec_excl_ind = 0
    AND ttl_demand_ind = 1
    AND univ_cat_desc <> 'Converse'
    AND dol.upm_id IS NOT NULL
    AND CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_ADD(last_send_dt,7)
    AND (rtn_qty = 0 or rtn_qty IS NULL)
)

-- AA period purchase. dynamically based on when the member receives their first communication. Only returns demand for members in the
, aa_purchase AS (
  SELECT dol.upm_id
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

, demand_bucket AS (
  SELECT tct.upm_id
  , tct.test_control
  , aa.upm_id as upm_id_aa_buyer
  , aa.demand
  , aa.orders
  , aa.units
  , CASE
    WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
    WHEN aa.demand < 50 THEN '000-050'
    WHEN aa.demand < 100 THEN '050-100'
    WHEN aa.demand < 200 THEN '100-200'
    WHEN aa.demand < 500 THEN '200-500'
    WHEN aa.demand < 1000 THEN '500-1000'
    WHEN aa.demand >=1000 THEN '1000+'
    END AS demand_per_buyer_bucket
  , PERCENT_RANK() OVER (
    PARTITION BY test_control
    , CASE
      WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
      WHEN aa.demand < 50 THEN '000-050'
      WHEN aa.demand < 100 THEN '050-100'
      WHEN aa.demand < 200 THEN '100-200'
      WHEN aa.demand < 500 THEN '200-500'
      WHEN aa.demand < 1000 THEN '500-1000'
      WHEN aa.demand >=1000 THEN '1000+'
      END ORDER BY RANDOM(0)
    ) AS percent_rank
  FROM test_control_table tct
  LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
)

, balanced_aud_aa as (
  SELECT DISTINCT upm_id
  FROM demand_bucket db
  -- remove outliers
  WHERE (demand < 2400 OR demand IS NULL)
-- To rebalance, paste text block from template in place of defaults
AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
	)
)

, mhub as (
 SELECT source_id as upm_id
 , member_id
 , preferred_gender
 FROM MEMBER.MEMBER_HUB
 WHERE ctry_2_cd = 'US'
 AND lower(dpa_status) IN ('activate','reactivate')
)

-- calculate a dynamic member activity window for each communication received by a member
-- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
-- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
, member_activities as (
  SELECT mgd.member_id
  ,SUM(logged_in_visits_count) AS visits
  ,SUM(pdp_favorite_count) AS pdp_favorite_count
  ,SUM(add_to_cart_count) AS add_to_cart_count
  ,SUM(physical_activity_count) AS physical_activity
  ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
  FROM test_control_table tct
  INNER JOIN mhub mh ON tct.upm_id = mh.upm_id
  LEFT JOIN aud_select_workspace.member_agg_member_growth_daily mgd ON mh.member_id = mgd.member_id
  WHERE mgd.activity_dt >= tct.campaign_send_dt
    AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
    AND mgd.preferred_retail_geo = 'NA'
    AND mgd.experience_name != 'agnostic'
  GROUP BY 1
)

-- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
, ab_purchase as (
  SELECT dol.upm_id as upm_id_ab_buyer
  , order_dt as order_ts
  , order_nbr
  ,SUM(grd_amt_excl_tax_usd) AS demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
  ,COUNT(DISTINCT order_nbr) AS orders
  ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(first_send_dt,7)
  GROUP BY 1,2,3
)

-- For each member, rank each order by how near it was to the last campaign send
-- purchases must be after the campaign send and within 7 days of the send to be attributed
, aud_order_join as (
SELECT a.*
, b.*
,(bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 as order_send_diff_minutes
,RANK() OVER(PARTITION BY a.upm_id, b.order_ts ORDER BY (bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 ASC) as nearest_order
FROM test_control_table a
LEFT JOIN ab_purchase b ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(a.campaign_send_ts,7))
ORDER BY a.upm_id,a.campaign_send_dt, nearest_order asc
)

-- Aggregate each distinct send, attributing only the nearest order.
-- This prevent double counting of orders when a members receives multiple sends within the measurement window.
, aud_order_agg as (
SELECT upm_id AS upm_id_ab_buyer
  ,SUM(demand) AS demand
  ,SUM(base_FW_demand) AS base_FW_demand
  ,SUM(base_AP_demand) AS base_AP_demand
  ,SUM(base_EQ_demand) AS base_EQ_demand
  ,SUM(clr_FW_demand) AS clr_FW_demand
  ,SUM(clr_AP_demand) AS clr_AP_demand
  ,SUM(clr_EQ_demand) AS clr_EQ_demand
  ,SUM(launch_FW_demand) AS launch_FW_demand
  ,SUM(launch_AP_demand) AS launch_AP_demand
  ,SUM(launch_EQ_demand) AS launch_EQ_demand
  ,SUM(rest_demand) AS rest_demand
  ,SUM(orders) AS orders
  ,SUM(units) AS units
FROM aud_order_join a
WHERE nearest_order = 1
AND demand IS NOT NULL -- additional filter for extraneous records
GROUP BY 1
ORDER BY 1
)

, historical_purchase AS (
  SELECT aoa.upm_id_ab_buyer as upm_id
  , COUNT(DISTINCT order_nbr) as histOrders
  FROM aud_order_agg aoa  -- eligible audience
  INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

-- due to how messages are set up on Bluecore, the campaign_cd is not unique to a particular day. We concat with the message_sent_ts
-- only using the date ignores cases where multiple sends occur on the same day, potentially undercounting
,email_sent AS (
  SELECT fed.upm_id
  , COUNT(DISTINCT concat(campaign_cd, LEFT(message_sent_ts,21))) as email_circ
  FROM COMMS.FACT_EMAIL_DELIVERY fed
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(message_sent_ts AS DATE) between ${hivevar:msmt_start_dt} and ${hivevar:msmt_end_dt}
  AND retail_geo_name = 'NA'
  AND fed.upm_id IS NOT NULL
  GROUP BY upm_id
)

-- returns all the instances of the campaign codes for each member along with the response types and revenue if applicable
, email_response as (
  SELECT DISTINCT fer.upm_id
  , LEFT(response_ts,10) AS response_ts
  , response_type
  , revenue_dollar
  FROM comms.fact_email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(LEFT(response_ts,10) AS DATE) between first_send_dt and DATE_ADD(last_send_dt,7)
  AND retail_geo_name = 'NA'
  AND fer.upm_id IS NOT NULL
 )

-- calculates the the total number of opens, clicks, purchases and last touch attributed demand for each member for the campaign
-- responses are only considered if they occur between when the message was sent and 7 days later.
, email_calc AS (
  SELECT fer.upm_id
  , COUNT(CASE WHEN response_type= 'email open' THEN fer.upm_id ELSE NULL END) AS email_opens
  , COUNT(CASE WHEN response_type='link click' THEN fer.upm_id ELSE NULL END) AS email_clicks
  , COUNT(CASE WHEN response_type='ecommerce-purchase' THEN fer.upm_id ELSE NULL END) AS email_purch
  , SUM(CASE WHEN response_type='ecommerce-purchase' THEN revenue_dollar ELSE 0 END) AS email_demand
  FROM email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE CAST(fer.response_ts as DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7)
  GROUP BY 1
)
-- total number of total messages send for each member during the campaign.
-- For a singular campaign, this should be one (1), for triggers it could be more depending on how the trigger is set up
, email_delivery_agg as (
  SELECT upm_id
  ,SUM(email_circ) AS email_circ
  FROM email_sent
  GROUP BY 1
)

, email_response_agg as (
  SELECT upm_id
  ,SUM(email_opens) AS email_opens
  ,SUM(email_clicks) AS email_clicks
  ,SUM(email_purch) AS email_purch
  ,SUM(email_demand) AS email_demand
  FROM email_calc
  GROUP BY 1
)

, platform_usage as (
  SELECT tct.upm_id
  , CASE WHEN CAST(first_${hivevar:usage_platform}_activity_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7) THEN 1
         ELSE 0
    END AS platform_user
  FROM aud_select_workspace.gck_first_and_last_touchpoint flt
  INNER JOIN test_control_table tct ON flt.upm_id = tct.upm_id  -- Need first/last_send values as activity window
)

---------------------
-- MAIN QUERY
----------------------
SELECT z.upm_id
  , z.test_control

  -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
  ,SUM(demand) AS demand

  -- secondary metrics ON order double click
  ,SUM(orders) AS orders
  ,SUM(units) AS units

  -- secondary metrics ON emails
  , SUM(email_circ) AS email_sent
  , SUM(email_opens) AS email_opens
  , SUM(email_clicks) AS email_clicks
  , SUM(email_purch) AS email_purch
  , SUM(email_demand) AS email_demand

 -- secondary metrics ON site/app engagement
 , SUM(visits) AS site_app_visits
 , SUM(PDP_FAVORITE_COUNT) AS PDP_FAVORITE_COUNT
 , SUM(ADD_TO_CART_COUNT) AS ADD_TO_CART_COUNT
 , SUM(physical_activity) AS physical_activity

  -- secondary metrics ON margin
  , SUM(base_FW_demand) AS base_FW_demand
  , SUM(base_AP_demand) AS base_AP_demand
  , SUM(base_EQ_demand) AS base_EQ_demand
  , SUM(clr_FW_demand) AS clr_FW_demand
  , SUM(clr_AP_demand) AS clr_AP_demand
  , SUM(clr_EQ_demand) AS clr_EQ_demand
  , SUM(launch_FW_demand) AS launch_FW_demand
  , SUM(launch_AP_demand) AS launch_AP_demand
  , SUM(launch_EQ_demand) AS launch_EQ_demand
  , SUM(rest_demand) AS rest_demand

   -- optional first usage kpi
  , SUM(platform_user) AS platform_user

    -- secondary metrics ON order history
  ,SUM(histOrders) AS histOrders
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder


FROM test_control_table z
INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

INNER JOIN mhub mh ON z.upm_id = mh.upm_id
LEFT JOIN member_activities a ON mh.member_id = a.member_id

LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

LEFT JOIN email_delivery_agg cda ON z.upm_id = cda.upm_id
LEFT JOIN email_response_agg cra ON z.upm_id = cra.upm_id

LEFT JOIN platform_usage plt ON z.upm_id = plt.upm_id

GROUP BY 1,2
ORDER BY 1 DESC;
"""

# Load the data into a PySpark DataFrame
df_spark = spark.sql(ab_data)

# Convert to Pandas DataFrame for further analysis
df = df_spark.toPandas()

# Specify your personal schema and table name
full_table_name = "your_personal_schema.your_table_name"

# Save the DataFrame as a table in your personal schema
df_result.write.mode("overwrite").saveAsTable(full_table_name)

""" new stuff """

# Import necessary libraries
from pyspark.sql import SparkSession

# Create and fetch widget values
dbutils.widgets.text("s3_bucket_path", "s3://default-bucket/personal-schemas", "S3 Bucket Path")
dbutils.widgets.text("personal_schema", "my_personal_schema", "Personal Schema")
dbutils.widgets.text("table_name", "my_table", "Table Name")

s3_bucket_path = dbutils.widgets.get("s3_bucket_path")
personal_schema = dbutils.widgets.get("personal_schema")
table_name = dbutils.widgets.get("table_name")

# Assuming 'balanced_df' is your pandas DataFrame, convert it to a Spark DataFrame
balanced_spark_df = spark.createDataFrame(balanced_df)

# Construct the path for the external table
full_path = f"{s3_bucket_path}/{personal_schema}/{table_name}"

# Save the DataFrame as an external table in your personal schema in S3
# Ensure the database (schema) exists
spark.sql(f"CREATE DATABASE IF NOT EXISTS {personal_schema} LOCATION '{s3_bucket_path}/{personal_schema}'")
spark.sql(f"USE {personal_schema}")

# Save the DataFrame as an external table at the specified path
balanced_spark_df.write.mode("overwrite").option("path", full_path).saveAsTable(table_name)



#### AB Scorecard Metrics

In [ ]:
%python
from scipy import stats
import numpy as np
import pandas as pd


## Extracting input parameters for Demand


# Calculate average sales (demand) for control and test groups
control_avg_demand = control['demand'].mean()
test_avg_demand = test['demand'].mean()

# Calculate standard deviation of demand for control and test groups
control_std_dev = control['demand'].std()
test_std_dev = test['demand'].std()

# Calculate the # of buying members for the control and test groups
control_sample_size = control[control['orders'] > 0]['upm_id'].nunique()
test_sample_size = test[test['orders'] > 0]['upm_id'].nunique()


## Perform Statistical Analysis for Demand


# Calculate delta and z-value
"""
Delta, standard error, and z-value, are part of the t-test calculation.

We typically wouldn't calculate these separately when using a t-test.

If we need to see these, they're more relevant in the context of manual calculations or when not using a t-test directly.
"""

# Calculate standard error
"""
A note about standard error:

* When using stats.ttest_ind with equal_var=False, the test internally computes a version of standard error adjusted for the Welch's t-test.
* Welch's t-test doesn't assume equal population variances.
* For educational purposes, you can still calculate a simple standard error of the mean (SEM) for each group.
* Or, calculate the standard error of the difference between two means if you're comparing them directly.
* Code to do so follows below...

# Calculate the standard error of the mean for each group
sem_control = control['avg_demand'].sem()
sem_test = test['avg_demand'].sem()

# Calculate the standard error for the difference between two means
# Again, this formula is for illustrative purposes; the t-test already accounts for this in its calculation
n_control = len(control['avg_demand'])
n_test = len(test['avg_demand'])
std_error_diff = np.sqrt(sem_control**2 / n_control + sem_test**2 / n_test)

print(f"Standard Error (Control): {sem_control}")
print(f"Standard Error (Test): {sem_test}")
print(f"Standard Error of Difference: {std_error_diff}")
"""


## Performing the t-test and ignoring NaN values


# Two-tailed t-test for Avg Demand
t_stat_two_tailed, p_value_two_tailed = stats.ttest_ind(test['demand'].dropna(), control['demand'].dropna(), equal_var=False, nan_policy='omit')
# Calculating confidence levels for two-tailed test
confidence_level_two_tailed = 1 - p_value_two_tailed
type_i_error_demand_two_tailed = p_value_two_tailed

# One-tailed t-test for Avg Demand (assuming we are checking if test > control)
"""
Note:
We halve the p-value and adjust its interpretation based on the direction of the t-statistic.

The one-tailed test results are specifically meant for the hypothesis that the test group mean is greater than the control group mean.
"""
if t_stat_two_tailed > 0:
    # Halve the p-value for positive t-statistic where test mean is hypothesized to be greater than control mean
    p_value_one_tailed = p_value_two_tailed / 2
    confidence_level_one_tailed = 1 - p_value_one_tailed
    print("\nOne-Tailed Test Results (Test > Control):")
    print(f"T-statistic: {t_stat_two_tailed}")  # Same T-statistic as the two-tailed test
    print(f"One-Tailed P-value: {p_value_one_tailed}")
    print(f"Confidence Level: {confidence_level_one_tailed*100:.2f}%")
else:
    # If the t-statistic is negative, indicating the test group mean is not greater than the control mean
    print("\nOne-Tailed Test Results (Test > Control):")
    print(f"T-statistic: {t_stat_two_tailed}")
    print("The outcome does not support the hypothesized direction (test > control).")
    print("Interpretation of confidence levels or significance is not applicable for this outcome.")

# Output the results for two-tailed test
print("\nTwo-Tailed Test Results:")
print(f"T-statistic: {t_stat_two_tailed}")
print(f"P-value: {p_value_two_tailed}")
print(f"Confidence Level: {confidence_level_two_tailed*100:.2f}%")

"""
# Perform t-test for Avg Demand, omitting 'nan' values
t_stat_demand, p_value_demand = stats.ttest_ind(test['demand'], control['demand'], equal_var=False, nan_policy='omit')
confidence_level_demand = 1 - p_value_demand
type_i_error_demand = p_value_demand

print(f"T-statistic for Demand: {t_stat_demand}")
print(f"P-value for Demand: {p_value_demand}")
print(f"Confidence Level for Demand: {confidence_level_demand*100:.2f}%")
print(f"Type I Error for Demand: {type_i_error_demand:.4f}")

"""

print(f"Demand Sample Size - Control: {control_sample_size}")
print(f"Demand Sample Size - Test: {test_sample_size}")
print(f"Average Demand - Control: {control_avg_demand:.2f}")
print(f"Average Demand - Test: {test_avg_demand:.2f}")
print(f"Standard Deviation - Test: {test_std_dev:.2f}")
print(f"Standard Deviation - Control: {control_std_dev:.2f}")

print(f"T-statistic for Demand: {t_stat_two_tailed}")
print(f"P-value for Demand: {p_value_two_tailed}")
print(f"Confidence Level for Demand: {confidence_level_two_tailed*100:.2f}%")


## Extracting input parameters for Conversion


# Total count of all unique members in the test and control groups
control_conv_sample_size = control['upm_id'].nunique()
test_conv_sample_size = test['upm_id'].nunique()

# Calculate conversion rate for the test and control groups (total buying members / total members)
control_conv_rate = control_sample_size / control_conv_sample_size if control_conv_sample_size > 0 else 0
test_conv_rate = test_sample_size / test_conv_sample_size if test_conv_sample_size > 0 else 0

# Calculate lift for the conversion rate
lift = abs(test_conv_rate - control_conv_rate)

# Calculate difference of proportions
difference_of_proportions = test_conv_rate - control_conv_rate

print(f"Control Conversion Rate: {control_conv_rate*100:.2f}%")
print(f"Test Conversion Rate: {test_conv_rate*100:.2f}%")
print(f"Absolute Lift in Conversion Rate: {lift*100:.4f}%")
print(f"Difference_of_proportions: {difference_of_proportions:.4f}")



## Perform Statistical Analysis for Conversion

# Make sure these are not views but standalone DataFrames to avoid SettingWithCopyWarning
control = control.copy()
test = test.copy()
# Add a 'buyer' column to both test and control DataFrames
control.loc[:, 'buyer'] = (control['orders'] > 0).astype(int) # 'buyer' = 1 if 'orders' > 0, else 0
test.loc[:, 'buyer'] = (test['orders'] > 0).astype(int)

## Perform t-test for Conversion Rate
t_stat_conversion, p_value_conversion = stats.ttest_ind(test['buyer'], control['buyer'], equal_var=False)

# One-tailed test interpretation for Conversion Rate (assuming we are checking if test > control)
if t_stat_conversion > 0:
    """ Halve the p-value for positive t-statistic where test conversion rate is hypothesized to be greater than control"""
    p_value_one_tailed_conversion = p_value_conversion / 2
    confidence_level_one_tailed_conversion = 1 - p_value_one_tailed_conversion
    print("\nOne-Tailed Test Results (Test > Control) for Conversion Rate:")
    print(f"T-statistic: {t_stat_conversion:.4f}")
    print(f"One-Tailed P-value: {p_value_one_tailed_conversion:.4f}")
    print(f"Confidence Level: {confidence_level_one_tailed_conversion*100:.2f}%")
else:
    # If the t-statistic is negative, indicating the test group conversion rate is not greater than the control
    print("\nOne-Tailed Test Results (Test > Control) for Conversion Rate:")
    print(f"T-statistic: {t_stat_conversion:.4f}")
    print("The outcome does not support the hypothesized direction (test > control) for conversion rate.")
    print("Interpretation of confidence levels or significance is not applicable for this outcome.")

"""
# Perform t-test for Conversion Rate
t_stat_conversion, p_value_conversion = stats.ttest_ind(test['buyer'], control['buyer'], equal_var=False)

# Calculate confidence level and Type I error for Conversion Rate
confidence_level_conversion = 1 - p_value_conversion
type_i_error_conversion = p_value_conversion

print(f"T-statistic for Conversion: {t_stat_conversion:.4f}")
print(f"P-value for Conversion: {p_value_conversion:.4f}")
print(f"Confidence Level for Conversion: {confidence_level_conversion*100:.2f}%")
print(f"Type I Error for Conversion: {type_i_error_conversion:.4f}")
"""

## Secondary metrics ON order double click


# AOV for test & control groups
control_AOV = control['demand'].sum() / control['orders'].sum()
test_AOV = test['demand'].sum() / test['orders'].sum()

# AUR for test & control groups
control_AUR = control['demand'].sum() / control['units'].sum()
test_AUR = test['demand'].sum() / test['units'].sum()

# UPT for test & control groups
control_UPT = control['units'].sum() / control['orders'].sum()
test_UPT = test['units'].sum() / test['orders'].sum()


## Secondary metrics ON emails


# Email Open Rate for test & control groups
control_email_open_rate = control['email_opens'] / control['email_sent']
test_email_open_rate = test['email_opens'] / test['email_sent']

# Email Click Rate against sends for test & control groups
control_email_click_rate = control['email_clicks'] / control['email_sent']
test_email_click_rate = test['email_clicks'] / test['email_sent']

# Email Click Rate against opens for test & control groups
control_email_click_rate = control['email_clicks'] / control['email_opens']
test_email_click_rate = test['email_clicks'] / test['email_opens']

# Email Conversion Rate for test & control groups

control_email_conversion_rate = control['email_purch'] / control['email_sent']
test_email_conversion_rate = control['email_purch'] / control['email_sent']


## Secondary metrics ON site/app engagement


# Average Site/App Visits for test and control groups
control_avg_site_app_visits = control['site_app_visits'].mean()
test_avg_site_app_visits = test['site_app_visits'].mean()

# Average PDP Favorite Count for test and control groups
control_avg_site_app_visits = control['PDP_FAVORITE_COUNT'].mean()
test_avg_site_app_visits = test['PDP_FAVORITE_COUNT'].mean()

# Average Add to Cart Count for test and control groups
control_avg_site_app_visits = control['ADD_TO_CART_COUNT'].mean()
test_avg_site_app_visits = test['ADD_TO_CART_COUNT'].mean()

# Average Add to Cart Count for test and control groups
control_avg_site_app_visits = control['physical_activity'].mean()
test_avg_site_app_visits = test['physical_activity'].mean()


## Secondary metrics ON margin


# Average Margin for test & control groups

control_avg_margin = (
    control['base_FW_demand'].mean() * 0.38 +
    control['base_AP_demand'].mean() * 0.28 +
    control['base_EQ_demand'].mean() * 0.32 +
    control['clr_FW_demand'].mean() * 0.23 +
    control['clr_AP_demand'].mean() * 0.17 +
    control['clr_EQ_demand'].mean() * 0.12 +
    control['launch_FW_demand'].mean() * 0.25 +
    control['launch_AP_demand'].mean() * 0.08 +
    control['launch_EQ_demand'].mean() * 0.19 +
    control['rest_demand'].mean() * 0.3
)

test_avg_margin = (
    test['base_FW_demand'].mean() * 0.38 +
    test['base_AP_demand'].mean() * 0.28 +
    test['base_EQ_demand'].mean() * 0.32 +
    test['clr_FW_demand'].mean() * 0.23 +
    test['clr_AP_demand'].mean() * 0.17 +
    test['clr_EQ_demand'].mean() * 0.12 +
    test['launch_FW_demand'].mean() * 0.25 +
    test['launch_AP_demand'].mean() * 0.08 +
    test['launch_EQ_demand'].mean() * 0.19 +
    test['rest_demand'].mean() * 0.3
)

test_base_FW_demand = test['base_FW_demand'].mean()
test_base_AP_demand = test['base_AP_demand'].mean()
test_base_EQ_demand = test['base_EQ_demand'].mean()
test_clr_FW_demand = test['clr_FW_demand'].mean()
test_clr_AP_demand = test['clr_AP_demand'].mean()
test_clr_EQ_demand = test['clr_EQ_demand'].mean()
test_launch_FW_demand = test['launch_FW_demand'].mean()
test_launch_AP_demand = test['launch_AP_demand'].mean()
test_launch_EQ_demand = test['launch_EQ_demand'].mean()
test_rest_demand = test['rest_demand'].mean()

control_base_FW_demand = control['base_FW_demand'].mean()
control_base_AP_demand = control['base_AP_demand'].mean()
control_base_EQ_demand = control['base_EQ_demand'].mean()
control_clr_FW_demand = control['clr_FW_demand'].mean()
control_clr_AP_demand = control['clr_AP_demand'].mean()
control_clr_EQ_demand = control['clr_EQ_demand'].mean()
control_launch_FW_demand = control['launch_FW_demand'].mean()
control_launch_AP_demand = control['launch_AP_demand'].mean()
control_launch_EQ_demand = control['launch_EQ_demand'].mean()
control_rest_demand = control['rest_demand'].mean()


## Secondary metrics ON order history

# For the control group

# control_total_members = control['upm_id'].nunique()
control_pct_buyers_1_priorOrder = (control['histOrders'] == 1).sum() / control_conv_sample_size
control_DPB_1_prior = control[control['histOrders'] == 1]['demand'].sum() / (control['histOrders'] == 1).sum()
control_pct_buyers_2_priorOrder = (control['histOrders'] == 2).sum() / control_conv_sample_size
control_DPB_2_prior = control[control['histOrders'] == 2]['demand'].sum() / (control['histOrders'] == 2).sum()

# For the test group

# test_total_members = test['upm_id'].nunique()
test_pct_buyers_1_priorOrder = (test['histOrders'] == 1).sum() / test_conv_sample_size
test_DPB_1_prior = test[test['histOrders'] == 1]['demand'].sum() / (test['histOrders'] == 1).sum()
test_pct_buyers_2_priorOrder = (test['histOrders'] == 2).sum() / test_conv_sample_size
test_DPB_2_prior = test[test['histOrders'] == 2]['demand'].sum() / (test['histOrders'] == 2).sum()


# Secondary KPI's for the control group

control_demand_per_push_sent = control_conv_rate * control_avg_demand
control_transactions_per_buyer = control_avg_demand / control_AOV
control_margin_per_email_sent = control_avg_margin * control_conv_rate
control_footwear_demand_per_member = (control['base_FW_demand'] + control['clr_FW_demand'] + control['launch_FW_demand']) * control_conv_rate
control_apparel_demand_per_member = (control['base_AP_demand'] + control['clr_AP_demand'] + control['launch_AP_demand']) * control_conv_rate
control_equipment_demand_per_member = (control['base_EQ_demand'] + control['clr_EQ_demand'] + control['launch_EQ_demand']) * control_conv_rate
control_regular_product_demand_per_member = (control['base_FW_demand'] + control['base_AP_demand'] + control['base_EQ_demand']) * control_conv_rate
control_clearance_demand_per_member = (control['clr_FW_demand'] + control['clr_AP_demand'] + control['clr_EQ_demand']) * control_conv_rate
control_launch_demand_per_member = (control['launch_FW_demand'] + control['launch_AP_demand'] + control['launch_EQ_demand']) * control_conv_rate

# Secondary KPI's for the test group

test_demand_per_push_sent = test_conv_rate * test_avg_demand
test_transactions_per_buyer = test_avg_demand / test_AOV
test_margin_per_email_sent = test_avg_margin * test_conv_rate
test_footwear_demand_per_member = (test['base_FW_demand'] + test['clr_FW_demand'] + test['launch_FW_demand']) * test_conv_rate
test_apparel_demand_per_member = (test['base_AP_demand'] + test['clr_AP_demand'] + test['launch_AP_demand']) * test_conv_rate
test_equipment_demand_per_member = (test['base_EQ_demand'] + test['clr_EQ_demand'] + test['launch_EQ_demand']) * test_conv_rate
test_regular_product_demand_per_member = (test['base_FW_demand'] + test['base_AP_demand'] + test['base_EQ_demand']) * test_conv_rate
test_clearance_demand_per_member = (test['clr_FW_demand'] + test['clr_AP_demand'] + test['clr_EQ_demand']) * test_conv_rate
test_launch_demand_per_member = (test['launch_FW_demand'] + test['launch_AP_demand'] + test['launch_EQ_demand']) * test_conv_rate

# Metrics for the scorecard
scorecard_metrics = [
    ('Audience Size', 'conv_sample_size'),
    ('Demand per push sent', 'demand_per_push_sent'),
    ('Conversion rate', 'conversion_rate'),
    ('Demand per buyer', 'avg_demand'),
    ('AOV', 'AOV'),
    ('AUR', 'AUR'),
    ('UPT', 'UPT'),
    ('# Transactions per buyer', 'transactions_per_buyer'),
    ('Members with 1 previous order', '1_priorOrder'),
    ('Conversion rate (1x buyers/email receivers)', 'pctBuyers_1_priorOrder'),
    ('Demand per Buyer (1 previous order)', 'DPB_1_prior'),
    ('Members with 2 previous orders', '2_priorOrder'),
    ('Conversion rate (2x buyers/email receivers)', 'pctBuyers_2_priorOrder'),
    ('Demand per buyer (2 previous orders)', 'DPB_2_prior'),
    ('Margin per buyer', 'avg_margin'),
    ('Margin per email sent', 'margin_per_email_sent'),
    ('Email open rate (against sends)', 'email_open_rate'),
    ('Email click rate (against sends)', 'email_click_rate'),
    ('Visits per known member', 'avg_site_app_visits'),
    ('PDP favorite per known member', 'avg_PDP_FAVORITE_COUNT'),
    ('Add to cart per known member', 'avg_ADD_TO_CART_COUNT'),
    ('Workouts per known member', 'avg_physical_activity'),
    ('Footwear demand per member', 'footwear_demand_per_member'),
    ('Apparel demand per member', 'apparel_demand_per_member'),
    ('Equipment demand per member', 'equipment_demand_per_member'),
    ('Regular product demand per member', 'regular_product_demand_per_member'),
    ('Clearance demand per member', 'clearance_demand_per_member'),
    ('Launch demand per member', 'launch_demand_per_member')
]


# Create an empty DataFrame for the scorecard
scorecard_df = pd.DataFrame(columns=['Metric', 'Control', 'Test', 'Lift', 'Confidence Level'])

# Iterate through metrics to populate the scorecard
for metric_name, variable_name in scorecard_metrics:
    # Retrieve scalar values for control and test groups
    control_value = locals().get(f'control_{variable_name}', control[variable_name].mean() if variable_name in control.columns else np.nan)
    test_value = locals().get(f'test_{variable_name}', test[variable_name].mean() if variable_name in test.columns else np.nan)

    # Ensure control_value and test_value are scalars to avoid ambiguous truth value error
    control_value = control_value if np.isscalar(control_value) else control_value.mean()
    test_value = test_value if np.isscalar(test_value) else test_value.mean()

    # Calculate lift with a check to ensure control_value is not zero to avoid division by zero
    lift = (test_value - control_value) / control_value * 100 if control_value != 0 else np.nan

    # Determine the confidence level to display based on the metric
    if metric_name in ['Conversion rate', 'Demand per buyer']:
        formatted_confidence_level = f"{confidence_level_two_tailed * 100:.2f}%"
    else:
        formatted_confidence_level = ""  # Leave blank for metrics without a specific confidence level

    # Append data to the scorecard DataFrame
    scorecard_df = scorecard_df.append({
        'Metric': metric_name,
        'Control': f"{control_value:.3f}" if not pd.isna(control_value) else "N/A",
        'Test': f"{test_value:.3f}" if not pd.isna(test_value) else "N/A",
        'Lift': f"{lift:.2f}%" if not np.isnan(lift) else "N/A",
        'Confidence Level': formatted_confidence_level
    }, ignore_index=True)

# Display the scorecard
scorecard_df

SyntaxError: unterminated string literal (detected at line 392) (<ipython-input-2-bb93f39f4cb1>, line 392)

#### AGG Test

In [ ]:
%python
from scipy import stats
import numpy as np
import pandas as pd


## Extracting input parameters for Demand


# Calculate average sales (demand) for control and test groups
control_avg_demand = control['demand'].mean()
test_avg_demand = test['demand'].mean()

# Calculate standard deviation of demand for control and test groups
control_std_dev = control['demand'].std()
test_std_dev = test['demand'].std()

# Calculate the # of buying members for the control and test groups
control_sample_size = control[control['orders'] > 0]['upm_id'].nunique()
test_sample_size = test[test['orders'] > 0]['upm_id'].nunique()


## Perform Statistical Analysis for Demand


## Performing the t-test and ignoring NaN values

# Two-tailed t-test for Avg Demand
t_stat_two_tailed, p_value_two_tailed = stats.ttest_ind(test['demand'].dropna(), control['demand'].dropna(), equal_var=False, nan_policy='omit')
# Calculating confidence levels for two-tailed test
confidence_level_two_tailed = 1 - p_value_two_tailed
type_i_error_demand_two_tailed = p_value_two_tailed

# One-tailed t-test for Avg Demand (assuming we are checking if test > control)
"""
Note:
We halve the p-value and adjust its interpretation based on the direction of the t-statistic.

The one-tailed test results are specifically meant for the hypothesis that the test group mean is greater than the control group mean.
"""
if t_stat_two_tailed > 0:
    # Halve the p-value for positive t-statistic where test mean is hypothesized to be greater than control mean
    p_value_one_tailed = p_value_two_tailed / 2
    confidence_level_one_tailed = 1 - p_value_one_tailed
    print("\nOne-Tailed Test Results (Test > Control):")
    print(f"T-statistic: {t_stat_two_tailed}")  # Same T-statistic as the two-tailed test
    print(f"One-Tailed P-value: {p_value_one_tailed}")
    print(f"Confidence Level: {confidence_level_one_tailed*100:.2f}%")
else:
    # If the t-statistic is negative, indicating the test group mean is not greater than the control mean
    print("\nOne-Tailed Test Results (Test > Control):")
    print(f"T-statistic: {t_stat_two_tailed}")
    print("The outcome does not support the hypothesized direction (test > control).")
    print("Interpretation of confidence levels or significance is not applicable for this outcome.")

# Output the results for two-tailed test
print("\nTwo-Tailed Test Results:")
print(f"T-statistic: {t_stat_two_tailed}")
print(f"P-value: {p_value_two_tailed}")
print(f"Confidence Level: {confidence_level_two_tailed*100:.2f}%")

print(f"Demand Sample Size - Control: {control_sample_size}")
print(f"Demand Sample Size - Test: {test_sample_size}")
print(f"Average Demand - Control: {control_avg_demand:.2f}")
print(f"Average Demand - Test: {test_avg_demand:.2f}")
print(f"Standard Deviation - Test: {test_std_dev:.2f}")
print(f"Standard Deviation - Control: {control_std_dev:.2f}")

print(f"T-statistic for Demand: {t_stat_two_tailed}")
print(f"P-value for Demand: {p_value_two_tailed}")
print(f"Confidence Level for Demand: {confidence_level_two_tailed*100:.2f}%")


## Extracting input parameters for Conversion


# Total count of all unique members in the test and control groups
control_conv_sample_size = control['upm_id'].nunique()
test_conv_sample_size = test['upm_id'].nunique()

# Calculate conversion rate for the test and control groups (total buying members / total members)
control_conv_rate = control_sample_size / control_conv_sample_size if control_conv_sample_size > 0 else 0
test_conv_rate = test_sample_size / test_conv_sample_size if test_conv_sample_size > 0 else 0

# Calculate lift for the conversion rate
lift = abs(test_conv_rate - control_conv_rate)

# Calculate difference of proportions
difference_of_proportions = test_conv_rate - control_conv_rate

print(f"Control Conversion Rate: {control_conv_rate*100:.2f}%")
print(f"Test Conversion Rate: {test_conv_rate*100:.2f}%")
print(f"Absolute Lift in Conversion Rate: {lift*100:.4f}%")
print(f"Difference_of_proportions: {difference_of_proportions:.4f}")



## Perform Statistical Analysis for Conversion

# Make sure these are not views but standalone DataFrames to avoid SettingWithCopyWarning
control = control.copy()
test = test.copy()
# Add a 'buyer' column to both test and control DataFrames
control.loc[:, 'buyer'] = (control['orders'] > 0).astype(int) # 'buyer' = 1 if 'orders' > 0, else 0
test.loc[:, 'buyer'] = (test['orders'] > 0).astype(int)

## Perform t-test for Conversion Rate
t_stat_conversion, p_value_conversion = stats.ttest_ind(test['buyer'], control['buyer'], equal_var=False)

# One-tailed test interpretation for Conversion Rate (assuming we are checking if test > control)
if t_stat_conversion > 0:
    """ Halve the p-value for positive t-statistic where test conversion rate is hypothesized to be greater than control"""
    p_value_one_tailed_conversion = p_value_conversion / 2
    confidence_level_one_tailed_conversion = 1 - p_value_one_tailed_conversion
    print("\nOne-Tailed Test Results (Test > Control) for Conversion Rate:")
    print(f"T-statistic: {t_stat_conversion:.4f}")
    print(f"One-Tailed P-value: {p_value_one_tailed_conversion:.4f}")
    print(f"Confidence Level: {confidence_level_one_tailed_conversion*100:.2f}%")
else:
    # If the t-statistic is negative, indicating the test group conversion rate is not greater than the control
    print("\nOne-Tailed Test Results (Test > Control) for Conversion Rate:")
    print(f"T-statistic: {t_stat_conversion:.4f}")
    print("The outcome does not support the hypothesized direction (test > control) for conversion rate.")
    print("Interpretation of confidence levels or significance is not applicable for this outcome.")


## Secondary metrics ON order double click


# AOV for test & control groups
control_AOV = control['demand'].sum() / control['orders'].sum()
test_AOV = test['demand'].sum() / test['orders'].sum()

# AUR for test & control groups
control_AUR = control['demand'].sum() / control['units'].sum()
test_AUR = test['demand'].sum() / test['units'].sum()

# UPT for test & control groups
control_UPT = control['units'].sum() / control['orders'].sum()
test_UPT = test['units'].sum() / test['orders'].sum()


## Secondary metrics ON emails


# Email Open Rate for test & control groups
control_email_open_rate = control['email_opens'] / control['email_sent']
test_email_open_rate = test['email_opens'] / test['email_sent']

# Email Click Rate against sends for test & control groups
control_email_click_rate = control['email_clicks'] / control['email_sent']
test_email_click_rate = test['email_clicks'] / test['email_sent']

# Email Click Rate against opens for test & control groups
control_email_click_rate = control['email_clicks'] / control['email_opens']
test_email_click_rate = test['email_clicks'] / test['email_opens']

# Email Conversion Rate for test & control groups

control_email_conversion_rate = control['email_purch'] / control['email_sent']
test_email_conversion_rate = control['email_purch'] / control['email_sent']


## Secondary metrics ON site/app engagement


# Average Site/App Visits for test and control groups
control_avg_site_app_visits = control['site_app_visits'].mean()
test_avg_site_app_visits = test['site_app_visits'].mean()

# Average PDP Favorite Count for test and control groups
control_avg_site_app_visits = control['PDP_FAVORITE_COUNT'].mean()
test_avg_site_app_visits = test['PDP_FAVORITE_COUNT'].mean()

# Average Add to Cart Count for test and control groups
control_avg_site_app_visits = control['ADD_TO_CART_COUNT'].mean()
test_avg_site_app_visits = test['ADD_TO_CART_COUNT'].mean()

# Average Add to Cart Count for test and control groups
control_avg_site_app_visits = control['physical_activity'].mean()
test_avg_site_app_visits = test['physical_activity'].mean()


## Secondary metrics ON margin


# Average Margin for test & control groups

control_avg_margin = (
    control['base_FW_demand'].mean() * 0.38 +
    control['base_AP_demand'].mean() * 0.28 +
    control['base_EQ_demand'].mean() * 0.32 +
    control['clr_FW_demand'].mean() * 0.23 +
    control['clr_AP_demand'].mean() * 0.17 +
    control['clr_EQ_demand'].mean() * 0.12 +
    control['launch_FW_demand'].mean() * 0.25 +
    control['launch_AP_demand'].mean() * 0.08 +
    control['launch_EQ_demand'].mean() * 0.19 +
    control['rest_demand'].mean() * 0.3
)

test_avg_margin = (
    test['base_FW_demand'].mean() * 0.38 +
    test['base_AP_demand'].mean() * 0.28 +
    test['base_EQ_demand'].mean() * 0.32 +
    test['clr_FW_demand'].mean() * 0.23 +
    test['clr_AP_demand'].mean() * 0.17 +
    test['clr_EQ_demand'].mean() * 0.12 +
    test['launch_FW_demand'].mean() * 0.25 +
    test['launch_AP_demand'].mean() * 0.08 +
    test['launch_EQ_demand'].mean() * 0.19 +
    test['rest_demand'].mean() * 0.3
)

test_base_FW_demand = test['base_FW_demand'].mean()
test_base_AP_demand = test['base_AP_demand'].mean()
test_base_EQ_demand = test['base_EQ_demand'].mean()
test_clr_FW_demand = test['clr_FW_demand'].mean()
test_clr_AP_demand = test['clr_AP_demand'].mean()
test_clr_EQ_demand = test['clr_EQ_demand'].mean()
test_launch_FW_demand = test['launch_FW_demand'].mean()
test_launch_AP_demand = test['launch_AP_demand'].mean()
test_launch_EQ_demand = test['launch_EQ_demand'].mean()
test_rest_demand = test['rest_demand'].mean()

control_base_FW_demand = control['base_FW_demand'].mean()
control_base_AP_demand = control['base_AP_demand'].mean()
control_base_EQ_demand = control['base_EQ_demand'].mean()
control_clr_FW_demand = control['clr_FW_demand'].mean()
control_clr_AP_demand = control['clr_AP_demand'].mean()
control_clr_EQ_demand = control['clr_EQ_demand'].mean()
control_launch_FW_demand = control['launch_FW_demand'].mean()
control_launch_AP_demand = control['launch_AP_demand'].mean()
control_launch_EQ_demand = control['launch_EQ_demand'].mean()
control_rest_demand = control['rest_demand'].mean()


## Secondary metrics ON order history

# For the control group

# control_total_members = control['upm_id'].nunique()
control_pct_buyers_1_priorOrder = (control['histOrders'] == 1).sum() / control_conv_sample_size
control_DPB_1_prior = control[control['histOrders'] == 1]['demand'].sum() / (control['histOrders'] == 1).sum()
control_pct_buyers_2_priorOrder = (control['histOrders'] == 2).sum() / control_conv_sample_size
control_DPB_2_prior = control[control['histOrders'] == 2]['demand'].sum() / (control['histOrders'] == 2).sum()

# For the test group

# test_total_members = test['upm_id'].nunique()
test_pct_buyers_1_priorOrder = (test['histOrders'] == 1).sum() / test_conv_sample_size
test_DPB_1_prior = test[test['histOrders'] == 1]['demand'].sum() / (test['histOrders'] == 1).sum()
test_pct_buyers_2_priorOrder = (test['histOrders'] == 2).sum() / test_conv_sample_size
test_DPB_2_prior = test[test['histOrders'] == 2]['demand'].sum() / (test['histOrders'] == 2).sum()


# Secondary KPI's for the control group

control_demand_per_push_sent = control_conv_rate * control_avg_demand
control_transactions_per_buyer = control_avg_demand / control_AOV
control_margin_per_email_sent = control_avg_margin * control_conv_rate
control_footwear_demand_per_member = (control['base_FW_demand'] + control['clr_FW_demand'] + control['launch_FW_demand']) * control_conv_rate
control_apparel_demand_per_member = (control['base_AP_demand'] + control['clr_AP_demand'] + control['launch_AP_demand']) * control_conv_rate
control_equipment_demand_per_member = (control['base_EQ_demand'] + control['clr_EQ_demand'] + control['launch_EQ_demand']) * control_conv_rate
control_regular_product_demand_per_member = (control['base_FW_demand'] + control['base_AP_demand'] + control['base_EQ_demand']) * control_conv_rate
control_clearance_demand_per_member = (control['clr_FW_demand'] + control['clr_AP_demand'] + control['clr_EQ_demand']) * control_conv_rate
control_launch_demand_per_member = (control['launch_FW_demand'] + control['launch_AP_demand'] + control['launch_EQ_demand']) * control_conv_rate

# Secondary KPI's for the test group

test_demand_per_push_sent = test_conv_rate * test_avg_demand
test_transactions_per_buyer = test_avg_demand / test_AOV
test_margin_per_email_sent = test_avg_margin * test_conv_rate
test_footwear_demand_per_member = (test['base_FW_demand'] + test['clr_FW_demand'] + test['launch_FW_demand']) * test_conv_rate
test_apparel_demand_per_member = (test['base_AP_demand'] + test['clr_AP_demand'] + test['launch_AP_demand']) * test_conv_rate
test_equipment_demand_per_member = (test['base_EQ_demand'] + test['clr_EQ_demand'] + test['launch_EQ_demand']) * test_conv_rate
test_regular_product_demand_per_member = (test['base_FW_demand'] + test['base_AP_demand'] + test['base_EQ_demand']) * test_conv_rate
test_clearance_demand_per_member = (test['clr_FW_demand'] + test['clr_AP_demand'] + test['clr_EQ_demand']) * test_conv_rate
test_launch_demand_per_member = (test['launch_FW_demand'] + test['launch_AP_demand'] + test['launch_EQ_demand']) * test_conv_rate

# Metrics for the scorecard
scorecard_metrics = [
    ('Audience Size', 'conv_sample_size'),
    ('Demand per push sent', 'demand_per_push_sent'),
    ('Conversion rate', 'conversion_rate'),
    ('Demand per buyer', 'avg_demand'),
    ('AOV', 'AOV'),
    ('AUR', 'AUR'),
    ('UPT', 'UPT'),
    ('# Transactions per buyer', 'transactions_per_buyer'),
    ('Members with 1 previous order', '1_priorOrder'),
    ('Conversion rate (1x buyers/email receivers)', 'pctBuyers_1_priorOrder'),
    ('Demand per Buyer (1 previous order)', 'DPB_1_prior'),
    ('Members with 2 previous orders', '2_priorOrder'),
    ('Conversion rate (2x buyers/email receivers)', 'pctBuyers_2_priorOrder'),
    ('Demand per buyer (2 previous orders)', 'DPB_2_prior'),
    ('Margin per buyer', 'avg_margin'),
    ('Margin per email sent', 'margin_per_email_sent'),
    ('Email open rate (against sends)', 'email_open_rate'),
    ('Email click rate (against sends)', 'email_click_rate'),
    ('Visits per known member', 'avg_site_app_visits'),
    ('PDP favorite per known member', 'avg_PDP_FAVORITE_COUNT'),
    ('Add to cart per known member', 'avg_ADD_TO_CART_COUNT'),
    ('Workouts per known member', 'avg_physical_activity'),
    ('Footwear demand per member', 'footwear_demand_per_member'),
    ('Apparel demand per member', 'apparel_demand_per_member'),
    ('Equipment demand per member', 'equipment_demand_per_member'),
    ('Regular product demand per member', 'regular_product_demand_per_member'),
    ('Clearance demand per member', 'clearance_demand_per_member'),
    ('Launch demand per member', 'launch_demand_per_member')
]


# Create an empty DataFrame for the scorecard
scorecard_df = pd.DataFrame(columns=['Metric', 'Control', 'Test', 'Lift', 'Confidence Level'])

# Iterate through metrics to populate the scorecard
for metric_name, variable_name in scorecard_metrics:
    # Retrieve scalar values for control and test groups
    control_value = locals().get(f'control_{variable_name}', control[variable_name].mean() if variable_name in control.columns else np.nan)
    test_value = locals().get(f'test_{variable_name}', test[variable_name].mean() if variable_name in test.columns else np.nan)

    # Ensure control_value and test_value are scalars to avoid ambiguous truth value error
    control_value = control_value if np.isscalar(control_value) else control_value.mean()
    test_value = test_value if np.isscalar(test_value) else test_value.mean()

    # Calculate lift with a check to ensure control_value is not zero to avoid division by zero
    lift = (test_value - control_value) / control_value * 100 if control_value != 0 else np.nan

    # Determine the confidence level to display based on the metric
    if metric_name in ['Conversion rate', 'Demand per buyer']:
        formatted_confidence_level = f"{confidence_level_two_tailed * 100:.2f}%"
    else:
        formatted_confidence_level = ""  # Leave blank for metrics without a specific confidence level

    # Append data to the scorecard DataFrame
    scorecard_df = scorecard_df.append({
        'Metric': metric_name,
        'Control': f"{control_value:.3f}" if not pd.isna(control_value) else "N/A",
        'Test': f"{test_value:.3f}" if not pd.isna(test_value) else "N/A",
        'Lift': f"{lift:.2f}%" if not np.isnan(lift) else "N/A",
        'Confidence Level': formatted_confidence_level
    }, ignore_index=True)

# Display the scorecard
scorecard_df

### ERROR

# Create an empty DataFrame for the scorecard
scorecard_df = pd.DataFrame(columns=['Metric', 'Control', 'Test', 'Lift', 'Confidence Level'])

# Iterate through metrics to populate the scorecard
for metric_name, variable_name in scorecard_metrics:
    # Determine the appropriate aggregation function
    if variable_name in ['1_priorOrder', '2_priorOrder']:
        control_agg_func = 'sum'
        test_agg_func = 'sum'
    else:
        control_agg_func = 'mean'
        test_agg_func = 'mean'

    # Retrieve scalar values for control and test groups
    control_value = locals().get(f'control_{variable_name}', control[variable_name].agg(control_agg_func) if variable_name in control.columns else np.nan)
    test_value = locals().get(f'test_{variable_name}', test[variable_name].agg(test_agg_func) if variable_name in test.columns else np.nan)

    # Calculate lift, ensuring control_value is not zero to avoid division by zero
    lift = (test_value - control_value) / control_value * 100 if control_value != 0 else np.nan

    # Determine the confidence level to display based on the metric
    formatted_confidence_level = ""
    if variable_name == 'conversion_rate':
        confidence_level_variable = 'confidence_level_one_tailed_conversion' if 'confidence_level_one_tailed_conversion' in locals() else 'confidence_level_two_tailed_conversion'
    elif variable_name == 'avg_demand':
        confidence_level_variable = 'confidence_level_one_tailed_demand' if 'confidence_level_one_tailed_demand' in locals() else 'confidence_level_two_tailed_demand'
    
    confidence_level = locals().get(confidence_level_variable, 0)
    formatted_confidence_level = f"{confidence_level:.2f}%" if confidence_level else ""

    # Append data to the scorecard DataFrame
    scorecard_df = scorecard_df.append({
        'Metric': metric_name,
        'Control': f"{control_value:.3f}" if not pd.isna(control_value) else "N/A",
        'Test': f"{test_value:.3f}" if not pd.isna(test_value) else "N/A",
        'Lift': f"{lift:.2f}%" if not np.isnan(lift) else "N/A",
        'Confidence Level': formatted_confidence_level
    }, ignore_index=True)

# Display the scorecard
print(scorecard_df)


#### Adding Confidence Level to dataframe

In [ ]:
# Create an empty DataFrame for the scorecard
scorecard_df = pd.DataFrame(columns=['Metric', 'Control', 'Test', 'Lift', 'Confidence Level'])

# Populating the DataFrame with calculated values
metrics = {
    'Demand per buyer': ('demand', confidence_level_demand),
    'Conversion rate': ('conversion', confidence_level_conversion)
}

for metric_name, (variable_suffix, confidence_level) in metrics.items():
    control_var = 'control_' + variable_suffix
    test_var = 'test_' + variable_suffix

    control_value = locals()[control_var] if control_var in locals() else control[variable_suffix].mean()
    test_value = locals()[test_var] if test_var in locals() else test[variable_suffix].mean()

    # Calculate lift directly based on the metric
    if metric_name == 'Conversion rate':
        lift = abs(test_conv_rate - control_conv_rate)
    else:
        # If you have a specific way to calculate lift for other metrics, apply here
        # For demonstration, using a simple difference for demand, or adjust as needed
        lift = test_value - control_value

    # Appending values to DataFrame
    scorecard_df = scorecard_df.append({
        'Metric': metric_name,
        'Control': control_value,
        'Test': test_value,
        'Lift': lift,
        'Confidence Level': confidence_level
    }, ignore_index=True)

# Function to format values
def format_values(value, metric_name):
    if metric_name in ['Conversion rate', 'Lift', 'Confidence Level']:
        return f"{value * 100:.2f}%" if isinstance(value, float) else value  # Format as percentage
    else:
        return f"{value:.2f}" if isinstance(value, float) else value  # Format other values to two decimal places

# Applying formatting to the scorecard
for col in ['Control', 'Test', 'Lift', 'Confidence Level']:
    scorecard_df[col] = scorecard_df.apply(lambda row: format_values(row[col], row['Metric']), axis=1)

# Display the Formatted Scorecard DataFrame
scorecard_df


#### Old Scorecard Code

In [ ]:
# Create an empty DataFrame for the scorecard
scorecard_df = pd.DataFrame(columns=['Metric', 'Control', 'Test', 'Lift', 'Confidence Level'])


# Populating the DataFrame with calculated values
for metric_name, variable_name in scorecard_metrics:
    control_value = locals().get('control_' + variable_name) if 'control_' + variable_name in locals() else control[variable_name].mean()
    test_value = locals().get('test_' + variable_name) if 'test_' + variable_name in locals() else test[variable_name].mean()
    # lift = calculate_lift(control_value, test_value)
    lift = abs(test_value - ontrol_value)
    significance = calculate_significance(control, test, variable_name) # confidence_level_demand

    scorecard_df = scorecard_df.append({'Metric': metric_name, 'Control': control_value, 'Test': test_value, 'Lift': lift, 'Confidence Level': confidence_level_demand}, ignore_index=True)

# Function to format values
def format_values(value, metric_name):
    if metric_name in ['Conversion rate', 'Lift', 'Confidence Level' ]:
        return f"{value * 100:.3f}%" if isinstance(value, float) else value  # Format as percentage
    else:
        return f"{value:.3f}" if isinstance(value, float) else value  # Format other values to three decimal places

# Applying formatting to the scorecard
for col in ['Control', 'Test', 'Lift', 'Confidence Level']:
    scorecard_df[col] = scorecard_df.apply(lambda row: format_values(row[col], row['Metric']), axis=1)

# Display the Formatted Scorecard DataFrame
scorecard_df


#### Old Scorecard Code - V2

In [ ]:
# Create an empty DataFrame for the scorecard
scorecard_df = pd.DataFrame(columns=['Metric', 'Control', 'Test', 'Lift', 'Confidence Level'])

# Iterate through the metrics
for metric_name, variable_name in scorecard_metrics:
    control_value = locals().get('control_' + variable_name, control[variable_name].mean())
    test_value = locals().get('test_' + variable_name, test[variable_name].mean())
    lift = abs(test_value - control_value)

    # Only apply confidence level to conversion and demand
    if variable_name in ['conversion_rate', 'avg_demand']:
        # Use the pre-calculated 'confidence_level_two_tailed' for conversion rate and avg demand
        formatted_confidence_level = f"{confidence_level_two_tailed*100:.2f}%"  # Format the confidence level as percentage
    else:
        formatted_confidence_level =  ""  # Leave blank for other metrics

    # Append row to DataFrame
    scorecard_df = scorecard_df.append({
        'Metric': metric_name,
        'Control': control_value,
        'Test': test_value,
        'Lift': lift,  # Use the global 'lift' value for all metrics
        'Confidence Level': formatted_confidence_level  # Conditionally added
    }, ignore_index=True)

# Apply formatting to values
def format_values(value, metric_name):
    # Assuming this function correctly formats the values based on metric name
    if metric_name in ['Conversion rate', 'Lift', 'Confidence Level'] and value != "N/A":
        return f"{float(value):.3f}%"  # Format as percentage if applicable and not "N/A"
    return value  # Return unmodified value for "N/A" or non-floats

for col in ['Control', 'Test', 'Lift', 'Confidence Level']:
    scorecard_df[col] = scorecard_df.apply(lambda row: format_values(row[col], row['Metric']), axis=1)

# Display the formatted scorecard DataFrame
print(scorecard_df)

#### Old Scorecard Code V3

In [ ]:
from scipy import stats
import numpy as np
import pandas as pd

# Assuming 'control' and 'test' DataFrames are already defined with necessary columns

# Define your metrics configuration
scorecard_metrics = [
    # Define your metrics here
    # Example: ('Metric Name', 'column_name_in_df'),
]

# Initialize scorecard DataFrame
scorecard_df = pd.DataFrame(columns=['Metric', 'Control', 'Test', 'Lift', 'Confidence Level'])

# Iterate through each metric to populate the scorecard
for metric_name, variable_name in scorecard_metrics:
    # Retrieve or calculate metric values for control and test groups
    control_value = control[variable_name].mean() if variable_name in control else 0
    test_value = test[variable_name].mean() if variable_name in test else 0

    # Calculate lift, handling division by zero
    lift = ((test_value - control_value) / control_value * 100) if control_value != 0 else "N/A"

    # Determine if the metric is eligible for confidence level display
    if variable_name in ['conversion_rate', 'avg_demand']:
        formatted_confidence_level = f"{confidence_level_two_tailed * 100:.2f}%"
    else:
        formatted_confidence_level = ""  # Leave blank for metrics without a specific confidence level

    # Append the metric's data to the scorecard
    scorecard_df = scorecard_df.append({
        'Metric': metric_name,
        'Control': f"{control_value:.3f}",
        'Test': f"{test_value:.3f}",
        'Lift': f"{lift}%" if lift != "N/A" else lift,
        'Confidence Level': formatted_confidence_level
    }, ignore_index=True)

# Display the scorecard
print(scorecard_df)


#### Old Scorecard V4

In [ ]:
# Create an empty DataFrame for the scorecard
scorecard_df = pd.DataFrame(columns=['Metric', 'Control', 'Test', 'Lift', 'Confidence Level'])

# Iterate through metrics to populate the scorecard
for metric_name, variable_name in scorecard_metrics:
    # Retrieve scalar values for control and test groups
    control_value = locals().get(f'control_{variable_name}', control[variable_name].mean() if variable_name in control.columns else np.nan)
    test_value = locals().get(f'test_{variable_name}', test[variable_name].mean() if variable_name in test.columns else np.nan)

    # Ensure control_value and test_value are scalars to avoid ambiguous truth value error
    control_value = control_value if np.isscalar(control_value) else control_value.mean()
    test_value = test_value if np.isscalar(test_value) else test_value.mean()

    # Calculate lift with a check to ensure control_value is not zero to avoid division by zero
    lift = (test_value - control_value) / control_value * 100 if control_value != 0 else np.nan

    # Determine the confidence level to display based on the metric
    if metric_name in ['Conversion rate', 'Demand per buyer']:
        formatted_confidence_level = f"{confidence_level_two_tailed * 100:.2f}%"
    else:
        formatted_confidence_level = ""  # Leave blank for metrics without a specific confidence level

    # Append data to the scorecard DataFrame
    scorecard_df = scorecard_df.append({
        'Metric': metric_name,
        'Control': f"{control_value:.3f}" if not pd.isna(control_value) else "N/A",
        'Test': f"{test_value:.3f}" if not pd.isna(test_value) else "N/A",
        'Lift': f"{lift:.2f}%" if not np.isnan(lift) else "N/A",
        'Confidence Level': formatted_confidence_level
    }, ignore_index=True)

# Display the scorecard
scorecard_df


""" ALSO this update """

# Create an empty DataFrame for the scorecard
scorecard_df = pd.DataFrame(columns=['Metric', 'Control', 'Test', 'Lift', 'Confidence Level'])

# Iterate through metrics to populate the scorecard
for metric_name, variable_name in scorecard_metrics:
    # Check if the metric needs sum aggregation instead of mean
    if variable_name in ['1_priorOrder', '2_priorOrder']:
        control_agg_func = 'sum'
        test_agg_func = 'sum'
    else:
        control_agg_func = 'mean'
        test_agg_func = 'mean'

    # Retrieve or calculate metric values for control and test groups with appropriate aggregation
    control_value = locals().get(f'control_{variable_name}', control[variable_name].agg(control_agg_func) if variable_name in control.columns else np.nan)
    test_value = locals().get(f'test_{variable_name}', test[variable_name].agg(test_agg_func) if variable_name in test.columns else np.nan)

    # Calculate lift with a check to ensure control_value is not zero to avoid division by zero
    lift = (test_value - control_value) / control_value * 100 if control_value != 0 else np.nan

    # Determine the confidence level to display based on the metric
    confidence_level_variable = ''
    if variable_name == 'conversion_rate':
        confidence_level_variable = 'confidence_level_one_tailed_conversion' if 'confidence_level_one_tailed_conversion' in locals() else 'confidence_level_two_tailed_conversion'
    elif variable_name == 'avg_demand':
        confidence_level_variable = 'confidence_level_one_tailed_demand' if 'confidence_level_one_tailed_demand' in locals() else 'confidence_level_two_tailed_demand'
    confidence_level = locals().get(confidence_level_variable, 0)  # Ensure default value is 0

    formatted_confidence_level = f"{confidence_level * 100:.2f}%" if confidence_level else ""

    # Append data to the scorecard DataFrame
    scorecard_df = scorecard_df.append({
        'Metric': metric_name,
        'Control': f"{control_value:.3f}" if not pd.isna(control_value) else "N/A",
        'Test': f"{test_value:.3f}" if not pd.isna(test_value) else "N/A",
        'Lift': f"{lift:.2f}%" if not np.isnan(lift) else "N/A",
        'Confidence Level': formatted_confidence_level
    }, ignore_index=True)

# Display the scorecard
print(scorecard_df)


""" THIS Update too """

# Create an empty DataFrame for the scorecard
scorecard_df = pd.DataFrame(columns=['Metric', 'Control', 'Test', 'Lift', 'Confidence Level'])

# Iterate through metrics to populate the scorecard
for metric_name, variable_name in scorecard_metrics:
    # Attempt to retrieve pre-calculated scalar values for control and test groups or calculate mean
    control_value = locals().get(f'control_{variable_name}', control[variable_name].mean() if variable_name in control.columns else np.nan)
    test_value = locals().get(f'test_{variable_name}', test[variable_name].mean() if variable_name in test.columns else np.nan)

    # Calculate lift, ensuring control_value is not zero to avoid division by zero
    lift = "N/A" if control_value == 0 else (test_value - control_value) / control_value * 100

    # Determine the confidence level to display based on the metric
    if metric_name == 'Conversion rate':
        formatted_confidence_level = f"{locals().get('confidence_level_two_tailed_conversion', 0) * 100:.2f}%" if 'confidence_level_two_tailed_conversion' in locals() else ""
    elif metric_name == 'Demand per buyer':
        formatted_confidence_level = f"{locals().get('confidence_level_two_tailed_demand', 0) * 100:.2f}%" if 'confidence_level_two_tailed_demand' in locals() else ""
    else:
        formatted_confidence_level = ""  # Leave blank for metrics without a specific confidence level

    # Append data to the scorecard DataFrame
    scorecard_df = scorecard_df.append({
        'Metric': metric_name,
        'Control': f"{control_value:.3f}" if not pd.isna(control_value) else "N/A",
        'Test': f"{test_value:.3f}" if not pd.isna(test_value) else "N/A",
        'Lift': f"{lift:.2f}%" if lift != "N/A" else lift,
        'Confidence Level': formatted_confidence_level
    }, ignore_index=True)

# Display the scorecard
print(scorecard_df)


### Notes

* The Excel spreadsheet managed by the teamm has a '**Statistical Signifigance**' column, which purports to be reporting **1−α**.
> Except this column is actually using the Excel formula =NORMSDIST(z-value). Is this formula actually reporting the *p-value*?
* There is also a column that purports to be reporting **α**.
> Except this column is actually using the Excel formula = 1 - NORMSDIST(z-value). Is this formula actually reporting **1−α** ?

The Excel sheet seems to have reversed the p-value and confidence level columns, or am I wrong?


#### Confirming the mix up

Directly equating the CDF with "Achieved Confidence Level" and `1−CDF` with "Type 1 Error" is a misinterpretation.

The correct approach would be to use the p-value derived from the test statistic (z-value in this case) to compare against **α** to make a decision about the null hypothesis.

The *Achieved Confidence Level* could be thought of as
`1−p-value`in a loose sense when you're interpreting how confident you are that the observed effect is not due to random chance, but this terminology isn't standard.

p

### AB Scorecard V6

##### Full AB SQL

In [ ]:
%python

# Execute SQL Query

df_spark = spark.sql("""

-----------------------------------------------------------------------------------------------------------------------------------
-- AB Period
-----------------------------------------------------------------------------------------------------------------------------------
WITH

tc_first_last_send AS (
    SELECT *
    FROM (
         (SELECT distinct tst.upm_id
          , 'test' as test_control
          , MIN(LEFT(message_sent_ts,10)) AS campaign_send_dt
          , MIN(LEFT(message_sent_ts,21)) AS campaign_send_ts
          , MIN(LEFT(message_sent_ts,10)) as first_send_dt
          , MAX(LEFT(message_sent_ts,10)) as last_send_dt
          FROM ${hivevar:aud_table_tst} tst
          INNER JOIN comms.fact_email_delivery fed ON tst.upm_id = fed.upm_id AND campaign_geo_name = 'NA' AND campaign_cd LIKE ${hivevar:camp_cd_tst_str} AND CAST(message_sent_ts AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND ${hivevar:msmt_end_dt}
          GROUP BY 1,2
          )
      UNION
        (SELECT distinct upm_id
         , 'control' as test_control
         , ${hivevar:msmt_start_dt} AS campaign_send_dt
         , to_timestamp(${hivevar:msmt_start_dt}, 'yyyy-MM-dd') AS campaign_send_ts
         , ${hivevar:msmt_start_dt} as first_send_dt
         , ${hivevar:msmt_start_dt} as last_send_dt
         from ${hivevar:aud_table_ctl}
         )
    )
)

, test_control_table AS (
    SELECT *
    , LEAD(a.campaign_send_dt) OVER (PARTITION BY a.upm_id ORDER BY a.campaign_send_dt) AS next_send_dt
    , LAG(a.campaign_send_dt) OVER (PARTITION BY a.upm_id ORDER BY a.campaign_send_dt) AS previous_send_dt
    , (int(to_timestamp(a.campaign_send_dt)) - int(to_timestamp(LEAD(a.campaign_send_dt) OVER (PARTITION BY a.upm_id ORDER BY a.campaign_send_dt))))/86400 AS difference_next_send_days
    , (int(to_timestamp(a.campaign_send_dt)) - int(to_timestamp(LAG(a.campaign_send_dt) OVER (PARTITION BY a.upm_id ORDER BY a.campaign_send_dt))))/86400 AS difference_previous_send_days
    FROM tc_first_last_send a
    ANTI JOIN gwan13.bot_master_nike_com bot ON a.upm_id = bot.upm_id
    ANTI JOIN aud_select_workspace.reseller_new slr ON a.upm_id = slr.upm_user_id
)

, view_dol AS (
  SELECT dol.upm_id
  , order_nbr
  , order_dt
  , origl_ordered_qty
  , grd_amt_excl_tax_usd
  , line_of_business_desc
  , univ_div_desc
  , first_send_dt
  , last_send_dt
  FROM DTC_INTEGRATED.DTC_DIGITAL_ORDER_LINE dol
  INNER JOIN test_control_table tct ON dol.upm_id = tct.upm_id
  WHERE region_key = 1
    AND rec_excl_ind = 0
    AND ttl_demand_ind = 1
    AND univ_cat_desc <> 'Converse'
    AND dol.upm_id IS NOT NULL
    AND CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_ADD(last_send_dt,7)
    AND (rtn_qty = 0 or rtn_qty IS NULL)
)

-- AA period purchase. dynamically based on when the member receives their first communication. Only returns demand for members in the
, aa_purchase AS (
  SELECT dol.upm_id
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

, demand_bucket AS (
  SELECT tct.upm_id
  , tct.test_control
  , aa.upm_id as upm_id_aa_buyer
  , aa.demand
  , aa.orders
  , aa.units
  , CASE
    WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
    WHEN aa.demand < 50 THEN '000-050'
    WHEN aa.demand < 100 THEN '050-100'
    WHEN aa.demand < 200 THEN '100-200'
    WHEN aa.demand < 500 THEN '200-500'
    WHEN aa.demand < 1000 THEN '500-1000'
    WHEN aa.demand >=1000 THEN '1000+'
    END AS demand_per_buyer_bucket
  , PERCENT_RANK() OVER (
    PARTITION BY test_control
    , CASE
      WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
      WHEN aa.demand < 50 THEN '000-050'
      WHEN aa.demand < 100 THEN '050-100'
      WHEN aa.demand < 200 THEN '100-200'
      WHEN aa.demand < 500 THEN '200-500'
      WHEN aa.demand < 1000 THEN '500-1000'
      WHEN aa.demand >=1000 THEN '1000+'
      END ORDER BY RANDOM(0)
    ) AS percent_rank
  FROM test_control_table tct
  LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
)

, balanced_aud_aa as (
  SELECT DISTINCT upm_id
  FROM demand_bucket db
  -- remove outliers
  WHERE (demand < 2400 OR demand IS NULL)
-- To rebalance, paste text block from template in place of defaults
AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
	)
)

, mhub as (
 SELECT source_id as upm_id
 , member_id
 , preferred_gender
 FROM MEMBER.MEMBER_HUB
 WHERE ctry_2_cd = 'US'
 AND lower(dpa_status) IN ('activate','reactivate')
)

-- calculate a dynamic member activity window for each communication received by a member
-- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
-- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
, member_activities as (
  SELECT mgd.member_id
  ,SUM(logged_in_visits_count) AS visits
  ,SUM(pdp_favorite_count) AS pdp_favorite_count
  ,SUM(add_to_cart_count) AS add_to_cart_count
  ,SUM(physical_activity_count) AS physical_activity
  ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
  FROM test_control_table tct
  INNER JOIN mhub mh ON tct.upm_id = mh.upm_id
  LEFT JOIN aud_select_workspace.member_agg_member_growth_daily mgd ON mh.member_id = mgd.member_id
  WHERE mgd.activity_dt >= tct.campaign_send_dt
    AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
    AND mgd.preferred_retail_geo = 'NA'
    AND mgd.experience_name != 'agnostic'
  GROUP BY 1
)

-- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
, ab_purchase as (
  SELECT dol.upm_id as upm_id_ab_buyer
  , order_dt as order_ts
  , order_nbr
  ,SUM(grd_amt_excl_tax_usd) AS demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
  ,COUNT(DISTINCT order_nbr) AS orders
  ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(first_send_dt,7)
  GROUP BY 1,2,3
)

-- For each member, rank each order by how near it was to the last campaign send
-- purchases must be after the campaign send and within 7 days of the send to be attributed
, aud_order_join as (
SELECT a.*
, b.*
,(bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 as order_send_diff_minutes
,RANK() OVER(PARTITION BY a.upm_id, b.order_ts ORDER BY (bigint(to_timestamp(b.order_ts))-bigint(to_timestamp(a.campaign_send_ts)))/60 ASC) as nearest_order
FROM test_control_table a
LEFT JOIN ab_purchase b ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(a.campaign_send_ts,7))
ORDER BY a.upm_id,a.campaign_send_dt, nearest_order asc
)

-- Aggregate each distinct send, attributing only the nearest order.
-- This prevent double counting of orders when a members receives multiple sends within the measurement window.
, aud_order_agg as (
SELECT upm_id AS upm_id_ab_buyer
  ,SUM(demand) AS demand
  ,SUM(base_FW_demand) AS base_FW_demand
  ,SUM(base_AP_demand) AS base_AP_demand
  ,SUM(base_EQ_demand) AS base_EQ_demand
  ,SUM(clr_FW_demand) AS clr_FW_demand
  ,SUM(clr_AP_demand) AS clr_AP_demand
  ,SUM(clr_EQ_demand) AS clr_EQ_demand
  ,SUM(launch_FW_demand) AS launch_FW_demand
  ,SUM(launch_AP_demand) AS launch_AP_demand
  ,SUM(launch_EQ_demand) AS launch_EQ_demand
  ,SUM(rest_demand) AS rest_demand
  ,SUM(orders) AS orders
  ,SUM(units) AS units
FROM aud_order_join a
WHERE nearest_order = 1
AND demand IS NOT NULL -- additional filter for extraneous records
GROUP BY 1
ORDER BY 1
)

, historical_purchase AS (
  SELECT aoa.upm_id_ab_buyer as upm_id
  , COUNT(DISTINCT order_nbr) as histOrders
  FROM aud_order_agg aoa  -- eligible audience
  INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

-- due to how messages are set up on Bluecore, the campaign_cd is not unique to a particular day. We concat with the message_sent_ts
-- only using the date ignores cases where multiple sends occur on the same day, potentially undercounting
,email_sent AS (
  SELECT fed.upm_id
  , COUNT(DISTINCT concat(campaign_cd, LEFT(message_sent_ts,21))) as email_circ
  FROM COMMS.FACT_EMAIL_DELIVERY fed
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(message_sent_ts AS DATE) between ${hivevar:msmt_start_dt} and ${hivevar:msmt_end_dt}
  AND retail_geo_name = 'NA'
  AND fed.upm_id IS NOT NULL
  GROUP BY upm_id
)

-- returns all the instances of the campaign codes for each member along with the response types and revenue if applicable
, email_response as (
  SELECT DISTINCT fer.upm_id
  , LEFT(response_ts,10) AS response_ts
  , response_type
  , revenue_dollar
  FROM comms.fact_email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE (campaign_cd LIKE ${hivevar:camp_cd_tst_str} OR campaign_cd LIKE ${hivevar:camp_cd_ctl_str})
  AND CAST(LEFT(response_ts,10) AS DATE) between first_send_dt and DATE_ADD(last_send_dt,7)
  AND retail_geo_name = 'NA'
  AND fer.upm_id IS NOT NULL
 )

-- calculates the the total number of opens, clicks, purchases and last touch attributed demand for each member for the campaign
-- responses are only considered if they occur between when the message was sent and 7 days later.
, email_calc AS (
  SELECT fer.upm_id
  , COUNT(CASE WHEN response_type= 'email open' THEN fer.upm_id ELSE NULL END) AS email_opens
  , COUNT(CASE WHEN response_type='link click' THEN fer.upm_id ELSE NULL END) AS email_clicks
  , COUNT(CASE WHEN response_type='ecommerce-purchase' THEN fer.upm_id ELSE NULL END) AS email_purch
  , SUM(CASE WHEN response_type='ecommerce-purchase' THEN revenue_dollar ELSE 0 END) AS email_demand
  FROM email_response fer
  INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id  -- Need first/last_send values as activity window
  WHERE CAST(fer.response_ts as DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7)
  GROUP BY 1
)
-- total number of total messages send for each member during the campaign.
-- For a singular campaign, this should be one (1), for triggers it could be more depending on how the trigger is set up
, email_delivery_agg as (
  SELECT upm_id
  ,SUM(email_circ) AS email_circ
  FROM email_sent
  GROUP BY 1
)

, email_response_agg as (
  SELECT upm_id
  ,SUM(email_opens) AS email_opens
  ,SUM(email_clicks) AS email_clicks
  ,SUM(email_purch) AS email_purch
  ,SUM(email_demand) AS email_demand
  FROM email_calc
  GROUP BY 1
)

, platform_usage as (
  SELECT tct.upm_id
  , CASE WHEN CAST(first_${hivevar:usage_platform}_activity_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7) THEN 1
         ELSE 0
    END AS platform_user
  FROM aud_select_workspace.gck_first_and_last_touchpoint flt
  INNER JOIN test_control_table tct ON flt.upm_id = tct.upm_id  -- Need first/last_send values as activity window
)

---------------------
-- MAIN QUERY
----------------------
SELECT z.upm_id
  , z.test_control

  -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
  ,SUM(demand) AS demand

  -- secondary metrics ON order double click
  ,SUM(orders) AS orders
  ,SUM(units) AS units

  -- secondary metrics ON emails
  , SUM(email_circ) AS email_sent
  , SUM(email_opens) AS email_opens
  , SUM(email_clicks) AS email_clicks
  , SUM(email_purch) AS email_purch
  , SUM(email_demand) AS email_demand

 -- secondary metrics ON site/app engagement
 , SUM(visits) AS site_app_visits
 , SUM(PDP_FAVORITE_COUNT) AS PDP_FAVORITE_COUNT
 , SUM(ADD_TO_CART_COUNT) AS ADD_TO_CART_COUNT
 , SUM(physical_activity) AS physical_activity

  -- secondary metrics ON margin
  , SUM(base_FW_demand) AS base_FW_demand
  , SUM(base_AP_demand) AS base_AP_demand
  , SUM(base_EQ_demand) AS base_EQ_demand
  , SUM(clr_FW_demand) AS clr_FW_demand
  , SUM(clr_AP_demand) AS clr_AP_demand
  , SUM(clr_EQ_demand) AS clr_EQ_demand
  , SUM(launch_FW_demand) AS launch_FW_demand
  , SUM(launch_AP_demand) AS launch_AP_demand
  , SUM(launch_EQ_demand) AS launch_EQ_demand
  , SUM(rest_demand) AS rest_demand

   -- optional first usage kpi
  , SUM(platform_user) AS platform_user

    -- secondary metrics ON order history
  ,SUM(histOrders) AS histOrders
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder


FROM test_control_table z
INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

INNER JOIN mhub mh ON z.upm_id = mh.upm_id
LEFT JOIN member_activities a ON mh.member_id = a.member_id

LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

LEFT JOIN email_delivery_agg cda ON z.upm_id = cda.upm_id
LEFT JOIN email_response_agg cra ON z.upm_id = cra.upm_id

LEFT JOIN platform_usage plt ON z.upm_id = plt.upm_id

GROUP BY 1,2
ORDER BY 1 DESC;
"""
)

# view data in spark df
display(df_spark)

# Convert to Pandas DataFrame for further analysis
df = df_spark.toPandas()


##### Example from Ben Li

In [ ]:
df_aggr = df_clean.groupby('variation').agg({'upm_id': pd.Series.nunique, # visitors
                                              'visits': [np.sum],
                                              'visits_with_pw_view': [np.count_nonzero], # get ratio pw view rate, unique event - count only once
                                              'visits_with_pdp_view': [np.count_nonzero], # get ratio pvr
                                              'visits_with_atc': [np.count_nonzero], # get ratio atc
                                              'visits_with_cart_view': [np.count_nonzero], # get ratio cart view rate
                                              'visits_with_checkout': [np.count_nonzero], # get ratio checkout rate
                                              'visits_with_order': [np.count_nonzero], # get ratio cvr
                                              'tot_pw_view': [np.sum],
                                              'tot_pdp_view': [np.sum],
                                              'tot_atc': [np.sum],
                                              'tot_cart_view': [np.sum],
                                              'tot_checkout': [np.sum],
                                              'tot_orders': [np.sum],
                                              'tot_units': [np.sum],
                                              'tot_demand': [np.sum] # get demand per visitor
                                              # 'tot_demand': [np.mean] # demand per visitor
                                              }).rename(columns={'upm_id': 'visitors'}).round(2)

# display(df_aggr)
# all `visits_with_xx` metrics are unique now (as `visitors_with_xx`) because of the count_nonzero() function used.
df_aggr

##### V6 Draft

In [ ]:
%python
import numpy as np
import pandas as pd
from scipy import stats

# Step 1: Grouping by 'group' and aggregating metrics
df_aggr = aggregated_metrics = df.groupby('test_control').agg({'upm_id': pd.Series.nunique,  # Unique members
                                                                'orders': 'sum',  # Total orders
                                                                'units': 'mean',  # Average units purchased
                                                                'demand': 'mean',  # Average sales (demand)

                                                            }).rename(columns={'upm_id': 'members', 'demand': 'avg_demand'}).round(2)

# Step 2: Calculating lift

# Transpose for easier manipulation
aggregated_metrics = aggregated_metrics.T

# Calculate lift as ((test - control) / control) * 100
lift = (aggregated_metrics['test'] - aggregated_metrics['control']) / aggregated_metrics['control'] * 100
lift = lift.rename('lift')

# Combine the original metrics with the lift calculation
result = pd.concat([aggregated_metrics, lift], axis=1)

result


##### V7 Draft

In [ ]:
%python
import numpy as np
import pandas as pd
from scipy import stats

# Step 1: Add a 'buyer' column to indicate whether an order was made
df['buyer'] = (df['orders'] > 0).astype(int)

# Step 2: aggregate metrics
aggregated_metrics = df.groupby('test_control').agg({
    'upm_id': pd.Series.nunique,  # Unique members (total_member_cnt)
    'buyer': 'sum',  # buying_member_cnt
    'orders': 'sum',  # Total orders
    'units': 'sum',  # Total units purchased
    'demand': ['mean', 'sum'], # Avg sales (demand) and Total Sales
}).rename(columns={
    'upm_id': 'Total Members',
    'demand_mean': 'Demand per Buyer',
    'demand_sum': 'Total Demand',
    'buyer': 'Buying Members',
}).round(2)

# Step 3: Calculate additional metrics
aggregated_metrics['Conversion Rate'] = (aggregated_metrics['Buying Members'] / aggregated_metrics['Total Members'])
#   ,COUNT(DISTINCT abp.upm_id_ab_buyer) / COUNT(DISTINCT z.upm_id) AS conversion_rate
aggregated_metrics['Demand per Send'] = (aggregated_metrics['Demand per Buyer'] * aggregated_metrics['Conversion Rate']).round(2)
aggregated_metrics['AOV'] = (aggregated_metrics['Total Demand']/aggregated_metrics['orders']).round(2)
aggregated_metrics['AUR'] = (aggregated_metrics['units']/aggregated_metrics['orders']).round(2)
aggregated_metrics['UPT'] = (aggregated_metrics['units']/aggregated_metrics['orders']).round(2)
aggregated_metrics['# Txn per Buyer'] = (aggregated_metrics['units']/aggregated_metrics['orders']).round(2)


# Step 4: Perform t-test for conversion rate
test_buyers = df[df['test_control'] == 'test']['buyer']
control_buyers = df[df['test_control'] == 'control']['buyer']
t_stat_conversion, p_value_conversion = stats.ttest_ind(test_buyers, control_buyers, equal_var=False)

# Step 5: Add confidence level and p-value to aggregated_metrics
aggregated_metrics['t_stat_conversion'] = t_stat_conversion
aggregated_metrics['p_value_conversion'] = p_value_conversion

# Step 6: Transpose for easier manipulation
aggregated_metrics = aggregated_metrics.T

# Step 7: Calculate lift
# ((test - control) / control) * 100
lift = (aggregated_metrics.get('test', 0) - aggregated_metrics.get('control', 0)) / aggregated_metrics.get('control', 1) * 100
lift = lift.rename('lift')
# Omit lift calculation for metrics that don't apply, like t_stat and p_value
lift['t_stat_conversion'] = ''
lift['p_value_conversion'] = ''

# Step 8: Combine the original metrics with the lift calculation
result = pd.concat([aggregated_metrics, lift], axis=1)

# Step 9: Display the result
result


#### V8 Draft

In [ ]:
%python
import numpy as np
import pandas as pd
from scipy import stats

# Step 1: Add 'buyer' and 'prior_order' columns

df['buyer'] = (df['orders'] > 0).astype(int) # 'buyer' = 1 if 'orders' > 0, else 0
df['demand_1_prior'] = np.where(df['1_priorOrder'] == 1, df['demand'], 0) # demand for members with 1 prior order
df['demand_2_prior'] = np.where(df['2_priorOrder'] == 1, df['demand'], 0) # demand for members with 2 prior orders
df['member_1_prior'] = np.where(df['1_priorOrder'] == 1, 1, 0) # Count of members with 1 prior order
df['member_2_prior'] = np.where(df['2_priorOrder'] == 1, 1, 0) # Count of members with 2 prior orders

# Step 2: Add temporary calculations for members with prior purchase (not to be included in the final DataFrame)

temp_calculations = df.groupby('test_control').agg({
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
}).rename(columns={
    'demand_1_prior': 'Total Demand 1 Prior',
    'demand_2_prior': 'Total Demand 2 Prior',
    'member_1_prior': 'Members 1 Prior',
    'member_2_prior': 'Members 2 Prior',
})

# Step 3: Calculate DPB_1_prior and DPB_2_prior using previously defined temporary calculations

dpb_1_prior = temp_calculations['Total Demand 1 Prior'] / temp_calculations['Members 1 Prior']
dpb_2_prior = temp_calculations['Total Demand 2 Prior'] / temp_calculations['Members 2 Prior']

# Step 4: Aggregate metrics

aggregated_metrics = df.groupby('test_control').agg({
    'upm_id': pd.Series.nunique,  # Unique members
    'buyer': 'sum',  # Buying member count
    'orders': 'sum',
    'units': 'sum',
    'demand': ['sum','mean'],  # Average and total demand
    '1_priorOrder': 'sum', # Members with 1 previous order
    '2_priorOrder': 'sum', # Members with w previous orders
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
    'email_opens': 'sum',
    'email_sent': 'sum',
    'email_clicks': 'sum',
    'site_app_visits': 'mean',
    'PDP_FAVORITE_COUNT': 'mean',
    'ADD_TO_CART_COUNT': 'mean',
    'physical_activity': 'mean',
    'base_FW_demand': 'mean',
    'clr_FW_demand': 'mean',
    'launch_FW_demand': 'mean',
    'base_AP_demand': 'mean',
    'clr_AP_demand': 'mean',
    'launch_AP_demand': 'mean',
    'base_EQ_demand': 'mean',
    'clr_EQ_demand': 'mean',
    'launch_EQ_demand': 'mean',
})

# Step 5: Flatten the MultiIndex columns

aggregated_metrics.columns = ['_'.join(col).strip() for col in aggregated_metrics.columns.values]
aggregated_metrics = aggregated_metrics.rename(columns={
    'upm_id_nunique': 'Total Members',
    'buyer_sum': 'Buying Members',
    'demand_mean': 'Demand per Buyer',
    'demand_sum': 'Total Demand',
    'units_sum': 'Total Units',
    'orders_sum': 'Total Orders',
    # Secondary KPIs - Repeat buyers
    '1_priorOrder_sum': 'Members with 1 previous order',
    'dpb_1_prior': 'Demand per Buyer (1 previous order)',
    '2_priorOrder_sum':'Members with 2 previous orders',
    'dpb_2_prior': 'Demand per Buyer (2 previous orders)',
    # Secondary KPIs - Site/app Engagement metrics
    'site_app_visits_mean':'Visits per known member',
    'PDP_FAVORITE_COUNT_mean':'PDP favorite per known member',
    'ADD_TO_CART_COUNT_mean': 'Add to cart per known member',
    'physical_activity_mean': 'Workouts per known member',
})


## Step 6: Calculate additional metrics


aggregated_metrics['Conversion Rate'] = aggregated_metrics['Buying Members'] / aggregated_metrics['Total Members']
aggregated_metrics['Demand per Send'] = aggregated_metrics['Demand per Buyer'] * aggregated_metrics['Conversion Rate']

# Secondary KPIs - order deep dive
aggregated_metrics['AOV'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Orders']
aggregated_metrics['AUR'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Units']
aggregated_metrics['UPT'] = aggregated_metrics['Total Units'] / aggregated_metrics['Total Orders']
aggregated_metrics['# Txn per Buyer'] = aggregated_metrics['Demand per Buyer']/aggregated_metrics['AOV']

# Secondary KPIs - Repeat buyers
aggregated_metrics['Conversion rate (1x buyers/email receivers)'] = aggregated_metrics['Members with 1 previous order'] / aggregated_metrics['Total Members']
aggregated_metrics['Conversion rate (2x buyers/email receivers)'] = aggregated_metrics['Members with 2 previous orders'] / aggregated_metrics['Total Members']
aggregated_metrics['DPB_1_prior'] = aggregated_metrics['demand_1_prior_sum'] / aggregated_metrics.get('member_1_prior_sum', 1)
aggregated_metrics['DPB_2_prior'] = aggregated_metrics['demand_2_prior_sum'] / aggregated_metrics.get('member_2_prior_sum', 1)

# Secondary KPIs - Email/Push Engagement metrics (all email/push received during measurement windows)
aggregated_metrics['Email Open Rate (against sends)'] = aggregated_metrics['email_opens_sum'] / aggregated_metrics['email_sent_sum']
aggregated_metrics['Email Click Rate (against sends)'] = aggregated_metrics['email_clicks_sum'] / aggregated_metrics['email_sent_sum']

# Secondary KPIs - product mix
aggregated_metrics['Footwear Demand per Member'] = ( aggregated_metrics['base_FW_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Apparel Demand per Member'] = ( aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Equipment Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['launch_EQ_demand_mean'] ) * aggregated_metrics['Conversion Rate']

# Secondary KPIs - LOB
aggregated_metrics['Regular Product Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['base_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Clearance Demand per Member'] = ( aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Launch Demand per Member'] = ( aggregated_metrics['launch_EQ_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

# Step 7: Remove temporary calculation columns as they're not needed in final DataFrame
aggregated_metrics.drop(columns=['demand_1_prior_sum', 'demand_2_prior_sum', 'member_1_prior_sum', 'member_2_prior_sum', 'email_opens_sum','email_sent_sum', 'email_clicks_sum', 'base_FW_demand_mean',
 'clr_FW_demand_mean', 'launch_FW_demand_mean', 'base_AP_demand_mean', 'clr_AP_demand_mean', 'launch_AP_demand_mean', 'base_EQ_demand_mean', 'clr_EQ_demand_mean','launch_EQ_demand_mean'], inplace=True)

# Step 8: Perform t-test for conversion rate
test_buyers = df[df['test_control'] == 'test']['buyer']
control_buyers = df[df['test_control'] == 'control']['buyer']
t_stat_conversion, p_value_conversion = stats.ttest_ind(test_buyers, control_buyers, equal_var=False)

# Step 9: Add confidence level and p-value to aggregated_metrics
aggregated_metrics['t_stat_conversion'] = t_stat_conversion
aggregated_metrics['p_value_conversion'] = p_value_conversion

# Step 10: Transpose the table
pd.options.display.float_format = '{:.2f}'.format
aggregated_metrics = aggregated_metrics.T

# Step 11: Calculate lift
# ((test - control) / control) * 100
lift = (aggregated_metrics.get('test', 0) - aggregated_metrics.get('control', 0)) / aggregated_metrics.get('control', 1) * 100
lift = lift.rename('lift')
# Omit lift calculation for metrics that don't apply, like t_stat and p_value
lift['t_stat_conversion'] = ''
lift['p_value_conversion'] = ''

# Step 12: Combine the original metrics with the lift calculation
result = pd.concat([aggregated_metrics, lift], axis=1)

# Step 13: Display the result
result

#### V11

In [ ]:
%python
import numpy as np
import pandas as pd
from scipy import stats

# Assuming 'df' is your DataFrame with necessary columns

# Step 1: Add 'buyer' and 'prior_order' indicator columns
df['buyer'] = (df['orders'] > 0).astype(int)
df['demand_1_prior'] = np.where(df['1_priorOrder'] == 1, df['demand'], np.nan)
df['demand_2_prior'] = np.where(df['2_priorOrder'] == 2, df['demand'], np.nan)
df['member_1_prior'] = np.where(df['1_priorOrder'] == 1, 1, 0)
df['member_2_prior'] = np.where(df['2_priorOrder'] == 2, 1, 0)

# Step 2: Perform temporary calculations for DPB_1_prior and DPB_2_prior
temp_calculations = df.groupby('test_control').agg({
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
}).rename(columns={
    'demand_1_prior': 'Total Demand 1 Prior',
    'demand_2_prior': 'Total Demand 2 Prior',
    'member_1_prior': 'Members 1 Prior',
    'member_2_prior': 'Members 2 Prior',
})

# Step 3: Calculate DPB_1_prior and DPB_2_prior using temporary calculations
dpb_1_prior = temp_calculations['Total Demand 1 Prior'] / temp_calculations['Members 1 Prior']
dpb_2_prior = temp_calculations['Total Demand 2 Prior'] / temp_calculations['Members 2 Prior']

# Step 4: Aggregate metrics
aggregated_metrics = df.groupby('test_control').agg({
    'upm_id': 'nunique',
    'buyer': 'sum',
    'orders': 'sum',
    'units': 'sum',
    'demand': ['sum', 'mean'],
    '1_priorOrder': 'sum',
    '2_priorOrder': 'sum',
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
    'email_opens': 'sum',
    'email_sent': 'sum',
    'email_clicks': 'sum',
    'site_app_visits': 'mean',
    'PDP_FAVORITE_COUNT': 'mean',
    'ADD_TO_CART_COUNT': 'mean',
    'physical_activity': 'mean',
    'base_FW_demand': 'mean',
    'clr_FW_demand': 'mean',
    'launch_FW_demand': 'mean',
    'base_AP_demand': 'mean',
    'clr_AP_demand': 'mean',
    'launch_AP_demand': 'mean',
    'base_EQ_demand': 'mean',
    'clr_EQ_demand': 'mean',
    'launch_EQ_demand': 'mean',
})

# Step 5: Correct the MultiIndex handling by flattening and renaming
aggregated_metrics.columns = ['_'.join(col).strip() for col in aggregated_metrics.columns.values]
aggregated_metrics = aggregated_metrics.rename(columns={
    'upm_id_nunique': 'Total Members',
    'buyer_sum': 'Buying Members',
    'demand_mean': 'Demand per Buyer',
    'demand_sum': 'Total Demand',
    'units_sum': 'Total Units',
    'orders_sum': 'Total Orders',
    # Secondary KPIs - Repeat buyers
    '1_priorOrder_sum': 'Members with 1 previous order',
    'dpb_1_prior': 'Demand per Buyer (1 previous order)',
    '2_priorOrder_sum':'Members with 2 previous orders',
    'dpb_2_prior': 'Demand per Buyer (2 previous orders)',
    # Secondary KPIs - Site/app Engagement metrics
    'site_app_visits_mean':'Visits per known member',
    'PDP_FAVORITE_COUNT_mean':'PDP favorite per known member',
    'ADD_TO_CART_COUNT_mean': 'Add to cart per known member',
    'physical_activity_mean': 'Workouts per known member',
})

# Step 6: Calculate DPB_1_prior and DPB_2_prior based on the temporary calculations and integrate directly into the DataFrame
# aggregated_metrics['DPB_1_prior'] = aggregated_metrics['demand_1_prior'] / aggregated_metrics['member_1_prior']
# aggregated_metrics['DPB_2_prior'] = aggregated_metrics['demand_2_prior'] / aggregated_metrics['member_2_prior']

# Step 7: Calculate additional metrics

aggregated_metrics['Conversion Rate'] = aggregated_metrics['Buying Members'] / aggregated_metrics['Total Members']
aggregated_metrics['Demand per Send'] = aggregated_metrics['Demand per Buyer'] * aggregated_metrics['Conversion Rate']
# Secondary KPIs - order deep dive
aggregated_metrics['AOV'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Orders']
aggregated_metrics['AUR'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Units']
aggregated_metrics['UPT'] = aggregated_metrics['Total Units'] / aggregated_metrics['Total Orders']
aggregated_metrics['# Txn per Buyer'] = aggregated_metrics['Demand per Buyer']/aggregated_metrics['AOV']
# Secondary KPIs - Repeat buyers
aggregated_metrics['Conversion rate (1x buyers/email receivers)'] = aggregated_metrics['Members with 1 previous order'] / aggregated_metrics['Total Members']
aggregated_metrics['Conversion rate (2x buyers/email receivers)'] = aggregated_metrics['Members with 2 previous orders'] / aggregated_metrics['Total Members']
aggregated_metrics['DPB_1_prior'] = aggregated_metrics['demand_1_prior_sum'] / aggregated_metrics.get('member_1_prior_sum', 1)
aggregated_metrics['DPB_2_prior'] = aggregated_metrics['demand_2_prior_sum'] / aggregated_metrics.get('member_2_prior_sum', 1)
# Secondary KPIs - Email/Push Engagement metrics (all email/push received during measurement windows)
aggregated_metrics['Email Open Rate (against sends)'] = aggregated_metrics['email_opens_sum'] / aggregated_metrics['email_sent_sum']
aggregated_metrics['Email Click Rate (against sends)'] = aggregated_metrics['email_clicks_sum'] / aggregated_metrics['email_sent_sum']
# Secondary KPIs - product mix
aggregated_metrics['Footwear Demand per Member'] = ( aggregated_metrics['base_FW_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Apparel Demand per Member'] = ( aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Equipment Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['launch_EQ_demand_mean'] ) * aggregated_metrics['Conversion Rate']
# Secondary KPIs - LOB
aggregated_metrics['Regular Product Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['base_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Clearance Demand per Member'] = ( aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Launch Demand per Member'] = ( aggregated_metrics['launch_EQ_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']


# Step 7: Remove temporary calculation columns as they're not needed in final DataFrame
aggregated_metrics.drop(columns=['demand_1_prior_sum', 'demand_2_prior_sum', 'member_1_prior_sum', 'member_2_prior_sum', 'email_opens_sum','email_sent_sum', 'email_clicks_sum', 'base_FW_demand_mean',
 'clr_FW_demand_mean', 'launch_FW_demand_mean', 'base_AP_demand_mean', 'clr_AP_demand_mean', 'launch_AP_demand_mean', 'base_EQ_demand_mean', 'clr_EQ_demand_mean','launch_EQ_demand_mean'], inplace=True)

# Step 8: Perform t-test for 'Conversion Rate' and 'Demand per Buyer'

# Separate the test and control groups to perform the t-tests correctly.
test_group = df[df['test_control'] == 'test']
control_group = df[df['test_control'] == 'control']

# Conversion Rate t-test
conversion_rate_test = test_group['buyer'].sum() / test_group['upm_id'].nunique()
conversion_rate_control = control_group['buyer'].sum() / control_group['upm_id'].nunique()

# Demand per Buyer t-test
demand_per_buyer_test = test_group[test_group['buyer'] > 0]['demand'].sum() / test_group['buyer'].sum()
demand_per_buyer_control = control_group[control_group['buyer'] > 0]['demand'].sum() / control_group['buyer'].sum()

t_stat_conversion, p_value_conversion = stats.ttest_ind(
    test_group[test_group['buyer'] > 0]['demand'],
    control_group[control_group['buyer'] > 0]['demand'],
    equal_var=False, nan_policy='omit'
)

t_stat_demand_per_buyer, p_value_demand_per_buyer = stats.ttest_ind(
    test_group[test_group['buyer'] > 0]['demand'] / test_group['buyer'],
    control_group[control_group['buyer'] > 0]['demand'] / control_group['buyer'],
    equal_var=False, nan_policy='omit'
)

# Step 9: Transpose the table
aggregated_metrics = aggregated_metrics.T

# Step 10: Calculate lift for all metrics except 'Conversion Rate' and 'Demand per Buyer'
lift_columns = ['Metric', 'Test', 'Control', 'Lift', 't-stat', 'p-value']
scorecard = pd.DataFrame(columns=lift_columns)

for metric in aggregated_metrics.index:  # Iterating over the index after transpose
    if metric not in ['Conversion Rate', 'Demand per Buyer']:
        test_val = aggregated_metrics.loc[metric, 'test']
        control_val = aggregated_metrics.loc[metric, 'control']
        lift = ((test_val - control_val) / control_val) * 100 if control_val != 0 else np.nan
        # Append each metric row to the scorecard DataFrame
        scorecard = scorecard.append({
            'Metric': metric,
            'Test': test_val,
            'Control': control_val,
            'Lift': lift,
            't-stat': '',
            'p-value': ''
        }, ignore_index=True)

# Make sure to set 'Metric' column as index if needed to match your scorecard structure
# scorecard.set_index('Metric', inplace=True)

# Step 11: Add t_stat and p_value for 'Conversion Rate' and 'Demand per Buyer' to the scorecard
scorecard.loc[scorecard['Metric'] == 'Conversion Rate', ['t-stat', 'p-value']] = [t_stat_conversion, p_value_conversion]
scorecard.loc[scorecard['Metric'] == 'Demand per Buyer', ['t-stat', 'p-value']] = [t_stat_demand_per_buyer, p_value_demand_per_buyer]

# Step 12: Exclude 'Conversion Rate' and 'Demand per Buyer' from lift calculation if not already done
scorecard.loc[scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Lift'] = np.nan # Set Lift to NaN for 'Conversion Rate' and 'Demand per Buyer' to denote exclusion

# Step 13: Display the scorecard
scorecard


#### V12

In [ ]:

%python
import numpy as np
import pandas as pd
from scipy import stats

# Assuming 'df' is your DataFrame with necessary columns

# Step 1: Add 'buyer' and 'prior_order' indicator columns
df['buyer'] = (df['orders'] > 0).astype(int)
df['demand_1_prior'] = np.where(df['1_priorOrder'] == 1, df['demand'], np.nan)
df['demand_2_prior'] = np.where(df['2_priorOrder'] == 2, df['demand'], np.nan)
df['member_1_prior'] = np.where(df['1_priorOrder'] == 1, 1, 0)
df['member_2_prior'] = np.where(df['2_priorOrder'] == 2, 1, 0)

# Step 2: Perform temporary calculations for DPB_1_prior and DPB_2_prior
temp_calculations = df.groupby('test_control').agg({
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
}).rename(columns={
    'demand_1_prior': 'Total Demand 1 Prior',
    'demand_2_prior': 'Total Demand 2 Prior',
    'member_1_prior': 'Members 1 Prior',
    'member_2_prior': 'Members 2 Prior',
})

# Step 3: Calculate DPB_1_prior and DPB_2_prior using temporary calculations
dpb_1_prior = temp_calculations['Total Demand 1 Prior'] / temp_calculations['Members 1 Prior']
dpb_2_prior = temp_calculations['Total Demand 2 Prior'] / temp_calculations['Members 2 Prior']

# Step 4: Aggregate metrics
aggregated_metrics = df.groupby('test_control').agg({
    'upm_id': 'nunique',
    'buyer': 'sum',
    'orders': 'sum',
    'units': 'sum',
    'demand': ['sum', 'mean'],
    '1_priorOrder': 'sum',
    '2_priorOrder': 'sum',
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
    'email_opens': 'sum',
    'email_sent': 'sum',
    'email_clicks': 'sum',
    'site_app_visits': 'mean',
    'PDP_FAVORITE_COUNT': 'mean',
    'ADD_TO_CART_COUNT': 'mean',
    'physical_activity': 'mean',
    'base_FW_demand': 'mean',
    'clr_FW_demand': 'mean',
    'launch_FW_demand': 'mean',
    'base_AP_demand': 'mean',
    'clr_AP_demand': 'mean',
    'launch_AP_demand': 'mean',
    'base_EQ_demand': 'mean',
    'clr_EQ_demand': 'mean',
    'launch_EQ_demand': 'mean',
})

# Step 5: Correct the MultiIndex handling by flattening and renaming
aggregated_metrics.columns = ['_'.join(col).strip() for col in aggregated_metrics.columns.values]
aggregated_metrics = aggregated_metrics.rename(columns={
    'upm_id_nunique': 'Total Members',
    'buyer_sum': 'Buying Members',
    'demand_mean': 'Demand per Buyer',
    'demand_sum': 'Total Demand',
    'units_sum': 'Total Units',
    'orders_sum': 'Total Orders',
    # Secondary KPIs - Repeat buyers
    '1_priorOrder_sum': 'Members with 1 previous order',
    '2_priorOrder_sum':'Members with 2 previous orders',
    # Secondary KPIs - Site/app Engagement metrics
    'site_app_visits_mean':'Visits per known member',
    'PDP_FAVORITE_COUNT_mean':'PDP favorite per known member',
    'ADD_TO_CART_COUNT_mean': 'Add to cart per known member',
    'physical_activity_mean': 'Workouts per known member',
})

# Step 6: Calculate DPB_1_prior and DPB_2_prior based on the temporary calculations and integrate directly into the DataFrame
# aggregated_metrics['DPB_1_prior'] = aggregated_metrics['demand_1_prior'] / aggregated_metrics['member_1_prior']
# aggregated_metrics['DPB_2_prior'] = aggregated_metrics['demand_2_prior'] / aggregated_metrics['member_2_prior']

# Step 7: Calculate additional metrics

aggregated_metrics['Conversion Rate'] = aggregated_metrics['Buying Members'] / aggregated_metrics['Total Members']
aggregated_metrics['Demand per Send'] = aggregated_metrics['Demand per Buyer'] * aggregated_metrics['Conversion Rate']
# Secondary KPIs - order deep dive
aggregated_metrics['AOV'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Orders']
aggregated_metrics['AUR'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Units']
aggregated_metrics['UPT'] = aggregated_metrics['Total Units'] / aggregated_metrics['Total Orders']
aggregated_metrics['# Txn per Buyer'] = aggregated_metrics['Demand per Buyer']/aggregated_metrics['AOV']
# Secondary KPIs - Repeat buyers
aggregated_metrics['Conversion rate (1x buyers/email receivers)'] = aggregated_metrics['Members with 1 previous order'] / aggregated_metrics['Total Members']
aggregated_metrics['Conversion rate (2x buyers/email receivers)'] = aggregated_metrics['Members with 2 previous orders'] / aggregated_metrics['Total Members']
aggregated_metrics['Demand per Buyer (1 previous order)'] = aggregated_metrics['demand_1_prior_sum'] / aggregated_metrics.get('member_1_prior_sum', 1)
aggregated_metrics['Demand per Buyer (2 previous order)'] = aggregated_metrics['demand_2_prior_sum'] / aggregated_metrics.get('member_2_prior_sum', 1)
# Secondary KPIs - Email/Push Engagement metrics (all email/push received during measurement windows)
aggregated_metrics['Email Open Rate (against sends)'] = aggregated_metrics['email_opens_sum'] / aggregated_metrics['email_sent_sum']
aggregated_metrics['Email Click Rate (against sends)'] = aggregated_metrics['email_clicks_sum'] / aggregated_metrics['email_sent_sum']
# Secondary KPIs - product mix
aggregated_metrics['Footwear Demand per Member'] = ( aggregated_metrics['base_FW_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Apparel Demand per Member'] = ( aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Equipment Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['launch_EQ_demand_mean'] ) * aggregated_metrics['Conversion Rate']
# Secondary KPIs - LOB
aggregated_metrics['Regular Product Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['base_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Clearance Demand per Member'] = ( aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Launch Demand per Member'] = ( aggregated_metrics['launch_EQ_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']


# Step 7: Remove temporary calculation columns as they're not needed in final DataFrame
aggregated_metrics.drop(columns=['demand_1_prior_sum', 'demand_2_prior_sum', 'member_1_prior_sum', 'member_2_prior_sum', 'email_opens_sum','email_sent_sum', 'email_clicks_sum', 'base_FW_demand_mean',
 'clr_FW_demand_mean', 'launch_FW_demand_mean', 'base_AP_demand_mean', 'clr_AP_demand_mean', 'launch_AP_demand_mean', 'base_EQ_demand_mean', 'clr_EQ_demand_mean','launch_EQ_demand_mean'], inplace=True)

# Step 8: Perform t-test for 'Conversion Rate' and 'Demand per Buyer'

# Separate the test and control groups to perform the t-tests correctly.
test_group = df[df['test_control'] == 'test']
control_group = df[df['test_control'] == 'control']

# Conversion Rate t-test
t_stat_conversion, p_value_conversion = stats.ttest_ind(test_group['buyer'], control_group['buyer'], equal_var=False)
confidence_level_conversion = 1 - p_value_conversion
type_i_error_conversion = p_value_conversion

# Demand per Buyer t-test
# 'nan' values imputed with 0
"""
t_stat_demand, p_value_demand = stats.ttest_ind(test_group['demand'].fillna(0), control_group['demand'].fillna(0), equal_var=False)
confidence_level_demand = 1 - p_value_demand
type_i_error_demand = p_value_demand
"""
# 'nan' values omitted
t_stat_demand, p_value_demand = stats.ttest_ind(test_group['demand'], control_group['demand'], equal_var=False, nan_policy='omit')
confidence_level_demand = 1 - p_value_demand
type_i_error_demand = p_value_demand
""" Note, results for demand ttest are the same with 'nan' values omitted or imputed to 0 """

"""
print(f"T-statistic for Demand: {t_stat_demand}")
print(f"P-value for Demand: {p_value_demand}")
print(f"Confidence Level for Demand: {confidence_level_demand*100:.2f}%")
print(f"Type I Error for Demand: {type_i_error_demand:.4f}")


print(f"T-statistic for Demand: {t_stat_conversion}")
print(f"P-value for Demand: {p_value_conversion}")
print(f"Confidence Level for Demand: {confidence_level_conversion*100:.2f}%")
print(f"Type I Error for Demand: {type_i_error_conversion:.4f}")
"""

# Step 9: Transpose the table
pd.options.display.float_format = '{:.2f}'.format
aggregated_metrics = aggregated_metrics.T

# Step 10: Calculate lift for all metrics except 'Conversion Rate' and 'Demand per Buyer'
scorecard = pd.DataFrame(index=aggregated_metrics.index, columns=['Metric', 'Test', 'Control', 'Lift', 't-stat', 'p-value'])

"""
# Step 10: Calculate lift for all metrics except 'Conversion Rate' and 'Demand per Buyer'
lift_columns = ['Metric', 'Test', 'Control', 'Lift', 't-stat', 'p-value']
scorecard = pd.DataFrame(columns=lift_columns)

for metric in aggregated_metrics.columns:
    for metric in aggregated_metrics.index:  # Assuming metrics are now index after transpose
      if metric not in ['Conversion Rate', 'Demand per Buyer']:
            test_val = aggregated_metrics.loc[metric, 'test']
            control_val = aggregated_metrics.loc[metric, 'control']
            lift = ((test_val - control_val) / control_val) * 100 if control_val != 0 else np.nan
      scorecard = scorecard.append({
            'Metric': metric,
            'Test': aggregated_metrics.loc[metric, 'test'],
            'Control': aggregated_metrics.loc[metric, 'control'],
            'Lift': lift,
            't-stat': '',
            'p-value': ''
      }, ignore_index=True)
"""

# Populate Test and Control values from aggregated_metrics
scorecard['Test'] = aggregated_metrics['test']
scorecard['Control'] = aggregated_metrics['control']

# Calculate Lift
for metric in aggregated_metrics.index:
    if metric not in ['Conversion Rate', 'Demand per Buyer']: # excluding Conversion_Rate and Demand_per_Buyer
        test_val = scorecard.at[metric, 'Test']
        control_val = scorecard.at[metric, 'Control']
        lift = ((test_val - control_val) / control_val) * 100 if control_val != 0 else np.nan
        scorecard.at[metric, 'Lift'] = lift

# Step 11: Add t_stat and p_value for 'Conversion Rate' and 'Demand per Buyer'
scorecard.at['Conversion Rate', 't-stat'] = t_stat_conversion
scorecard.at['Conversion Rate', 'p-value'] = p_value_conversion
scorecard.at['Demand per Buyer', 't-stat'] = t_stat_demand
scorecard.at['Demand per Buyer', 'p-value'] = p_value_demand

# Step 12: Ensure 'Metric' column is correctly populated
# A byproduct of transposing the dataframe with some calculations only being applied selectively
scorecard['Metric'] = scorecard.index
scorecard.reset_index(drop=False, inplace=True)
scorecard.drop(columns=['index'], inplace=True)

# Step 13: Exclude 'Conversion Rate' and 'Demand per Buyer' from lift calculation
scorecard.loc[scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Lift'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 't-stat'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'p-value'] = ''

# Step 14: Display the scorecard
scorecard



#### V13 - Latest Working Version

In [ ]:
%python
import numpy as np
import pandas as pd
from scipy import stats

# Assuming 'df' is your DataFrame with necessary columns

# Step 1: Add 'buyer' and 'prior_order' indicator columns
df_ab['buyer'] = (df_ab['orders'] > 0).astype(int)
df_ab['demand_1_prior'] = np.where(df_ab['1_priorOrder'] == 1, df_ab['demand'], np.nan)
df_ab['demand_2_prior'] = np.where(df_ab['2_priorOrder'] == 1, df_ab['demand'], np.nan)
df_ab['member_1_prior'] = np.where(df_ab['1_priorOrder'] == 1, 1, 0)
df_ab['member_2_prior'] = np.where(df_ab['2_priorOrder'] == 1, 1, 0)

# Step 2: Perform temporary calculations for DPB_1_prior and DPB_2_prior
temp_calculations = df_ab.groupby('test_control').agg({
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
}).rename(columns={
    'demand_1_prior': 'Total Demand 1 Prior',
    'demand_2_prior': 'Total Demand 2 Prior',
    'member_1_prior': 'Members 1 Prior',
    'member_2_prior': 'Members 2 Prior',
})

# Step 3: Calculate DPB_1_prior and DPB_2_prior using temporary calculations
dpb_1_prior = temp_calculations['Total Demand 1 Prior'] / temp_calculations['Members 1 Prior']
dpb_2_prior = temp_calculations['Total Demand 2 Prior'] / temp_calculations['Members 2 Prior']

# Step 4: Aggregate metrics
aggregated_metrics = df_ab.groupby('test_control').agg({
    'upm_id': 'nunique',
    'buyer': 'sum',
    'orders': 'sum',
    'units': 'sum',
    'demand': ['sum', 'mean'],
    '1_priorOrder': 'sum',
    '2_priorOrder': 'sum',
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
    'email_opens': 'sum',
    'email_sent': 'sum',
    'email_clicks': 'sum',
    'site_app_visits': 'mean',
    'PDP_FAVORITE_COUNT': 'mean',
    'ADD_TO_CART_COUNT': 'mean',
    'physical_activity': 'mean',
    'base_FW_demand': 'mean',
    'clr_FW_demand': 'mean',
    'launch_FW_demand': 'mean',
    'base_AP_demand': 'mean',
    'clr_AP_demand': 'mean',
    'launch_AP_demand': 'mean',
    'base_EQ_demand': 'mean',
    'clr_EQ_demand': 'mean',
    'launch_EQ_demand': 'mean',
})

# Step 5: Correct the MultiIndex handling by flattening and renaming
aggregated_metrics.columns = ['_'.join(col).strip() for col in aggregated_metrics.columns.values]
aggregated_metrics = aggregated_metrics.rename(columns={
    'upm_id_nunique': 'Total Members',
    'buyer_sum': 'Buying Members',
    'demand_mean': 'Demand per Buyer',
    'demand_sum': 'Total Demand',
    'units_sum': 'Total Units',
    'orders_sum': 'Total Orders',
    # Secondary KPIs - Repeat buyers
    '1_priorOrder_sum': 'Members with 1 previous order',
    '2_priorOrder_sum':'Members with 2 previous orders',
    # Secondary KPIs - Site/app Engagement metrics
    'site_app_visits_mean':'Visits per known member',
    'PDP_FAVORITE_COUNT_mean':'PDP favorite per known member',
    'ADD_TO_CART_COUNT_mean': 'Add to cart per known member',
    'physical_activity_mean': 'Workouts per known member',
})

# Step 6: Calculate DPB_1_prior and DPB_2_prior based on the temporary calculations and integrate directly into the DataFrame
# aggregated_metrics['DPB_1_prior'] = aggregated_metrics['demand_1_prior'] / aggregated_metrics['member_1_prior']
# aggregated_metrics['DPB_2_prior'] = aggregated_metrics['demand_2_prior'] / aggregated_metrics['member_2_prior']

# Step 7: Calculate additional metrics

aggregated_metrics['Conversion Rate'] = aggregated_metrics['Buying Members'] / aggregated_metrics['Total Members']
aggregated_metrics['Demand per Send'] = aggregated_metrics['Demand per Buyer'] * aggregated_metrics['Conversion Rate']
# Secondary KPIs - order deep dive
aggregated_metrics['AOV'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Orders']
aggregated_metrics['AUR'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Units']
aggregated_metrics['UPT'] = aggregated_metrics['Total Units'] / aggregated_metrics['Total Orders']
aggregated_metrics['# Txn per Buyer'] = aggregated_metrics['Demand per Buyer']/aggregated_metrics['AOV']
# Secondary KPIs - Repeat buyers
aggregated_metrics['Conversion rate (1x buyers/email receivers)'] = aggregated_metrics['Members with 1 previous order'] / aggregated_metrics['Total Members']
aggregated_metrics['Conversion rate (2x buyers/email receivers)'] = aggregated_metrics['Members with 2 previous orders'] / aggregated_metrics['Total Members']
aggregated_metrics['Demand per Buyer (1 previous order)'] = aggregated_metrics['demand_1_prior_sum'] / aggregated_metrics.get('member_1_prior_sum', 1)
aggregated_metrics['Demand per Buyer (2 previous order)'] = aggregated_metrics['demand_2_prior_sum'] / aggregated_metrics.get('member_2_prior_sum', 1)
# Secondary KPIs - Email/Push Engagement metrics (all email/push received during measurement windows)
aggregated_metrics['Email Open Rate (against sends)'] = aggregated_metrics['email_opens_sum'] / aggregated_metrics['email_sent_sum']
aggregated_metrics['Email Click Rate (against sends)'] = aggregated_metrics['email_clicks_sum'] / aggregated_metrics['email_sent_sum']
# Secondary KPIs - product mix
aggregated_metrics['Footwear Demand per Member'] = ( aggregated_metrics['base_FW_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Apparel Demand per Member'] = ( aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Equipment Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['launch_EQ_demand_mean'] ) * aggregated_metrics['Conversion Rate']
# Secondary KPIs - LOB
aggregated_metrics['Regular Product Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['base_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Clearance Demand per Member'] = ( aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Launch Demand per Member'] = ( aggregated_metrics['launch_EQ_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']


# Step 7: Remove temporary calculation columns as they're not needed in final DataFrame
aggregated_metrics.drop(columns=['demand_1_prior_sum', 'demand_2_prior_sum', 'member_1_prior_sum', 'member_2_prior_sum', 'email_opens_sum','email_sent_sum', 'email_clicks_sum', 'base_FW_demand_mean',
 'clr_FW_demand_mean', 'launch_FW_demand_mean', 'base_AP_demand_mean', 'clr_AP_demand_mean', 'launch_AP_demand_mean', 'base_EQ_demand_mean', 'clr_EQ_demand_mean','launch_EQ_demand_mean'], inplace=True)

# Step 8: Perform t-test for 'Conversion Rate' and 'Demand per Buyer'

# Separate the test and control groups to perform the t-tests correctly.
test_group = df_ab[df_ab['test_control'] == 'test']
control_group = df_ab[df_ab['test_control'] == 'control']

# Conversion Rate Chi-square

# Create a contingency table
conv_contingency_table = pd.crosstab(df_ab['test_control'], df_ab['buyer']) # Count num of buyers and non-buyers in each group

# Perform the Chi-square test
chi2_stat_conversion, p_value_conversion, dof_conversion, expected_cnt_conversion = chi2_contingency(conv_contingency_table)

# Demand per Buyer T-test
"""
# 'nan' values imputed with 0
t_stat_demand, p_value_demand = stats.ttest_ind(test_group['demand'].fillna(0), control_group['demand'].fillna(0), equal_var=False)
"""
# 'nan' values omitted
t_stat_demand, p_value_demand = stats.ttest_ind(test_group['demand'], control_group['demand'], equal_var=False, nan_policy='omit')
""" Note, results for demand ttest are the same with 'nan' values omitted or imputed to 0 """

# Step 9: Transpose the table
pd.options.display.float_format = '{:.2f}'.format
aggregated_metrics = aggregated_metrics.T

# Step 10: Calculate lift for all metrics except 'Conversion Rate' and 'Demand per Buyer'
scorecard = pd.DataFrame(index=aggregated_metrics.index, columns=['Metric', 'Test', 'Control', 'Lift','p-value','T or Chi2 stat'])

# Populate Test and Control values from aggregated_metrics
scorecard['Test'] = aggregated_metrics['test']
scorecard['Control'] = aggregated_metrics['control']

# Calculate Lift
for metric in aggregated_metrics.index:
    if metric not in ['Conversion Rate', 'Demand per Buyer']: # excluding Conversion_Rate and Demand_per_Buyer
        test_val = scorecard.at[metric, 'Test']
        control_val = scorecard.at[metric, 'Control']
        lift = ((test_val - control_val) / control_val) * 100 if control_val != 0 else np.nan
        scorecard.at[metric, 'Lift'] = lift

# Step 11: Add t_stat and p_value for 'Conversion Rate' and 'Demand per Buyer' to dataframe
scorecard.at['Conversion Rate', 'T or Chi2 stat'] = chi2_stat_conversion
scorecard.at['Conversion Rate', 'p-value'] = p_value_conversion
scorecard.at['Demand per Buyer', 'T or Chi2 stat'] = t_stat_demand
scorecard.at['Demand per Buyer', 'p-value'] = p_value_demand

# Step 12: Ensure 'Metric' column is correctly populated
# A byproduct of transposing the dataframe with some calculations only being applied selectively
scorecard['Metric'] = scorecard.index
scorecard.reset_index(drop=False, inplace=True)
scorecard.drop(columns=['index'], inplace=True)

# Step 13: Exclude 'Conversion Rate' and 'Demand per Buyer' from lift calculation
scorecard.loc[scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Lift'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'T or Chi2 stat'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'p-value'] = ''

# Step 14: Display the scorecard
scorecard


In [ ]:
%python

alpha = 0.1

print(f"P-value for Demand: {p_value_demand:.4f}")
print(f"P-value for Conversion: {p_value_conversion:.4f}")

# Check for statistical significance for Demand
if p_value_demand < alpha:
    print("Demand is Significant")
else:
    print("Demand Not Significant")

# Check for statistical significance for Conversion
if p_value_conversion < alpha:
    print("Conversion is Significant")
else:
    print("Conversion Not Significant")

##### Mann Whitney U (Wilcox) Test - V1

In [ ]:
%python

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html

"""
The Mann-Whitney U test is a nonparametric test of the null hypothesis that the distribution underlying sample x is the same as the distribution underlying sample y.

It is often used as a test of difference in location between distributions.
"""

alpha = 0.1

# result_rev = stats.mannwhitneyu(test_group['demand'], control_group['demand'])

# print('p_value = ', result_rev[1].round(10))

result_rev = stats.mannwhitneyu(test_group['demand'], control_group['demand'])

print('p_value = ', result_rev[1].round(10))

# Check for statistical significance
if result_rev[1] < alpha:
    print("Significant")
else:
    print("Not Significant")

##### Mann Whitney U (Wilcox) Test - V2

In [ ]:
%python
import numpy as np
import pandas as pd
from scipy import stats

# https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.mannwhitneyu.html

"""
The Mann-Whitney U test is a nonparametric test of the null hypothesis that the distribution underlying sample x is the same as the distribution underlying sample y.

It is often used as a test of difference in location between distributions.
"""

# Separate the test and control groups
test_group_ab = df_ab[df_ab['test_control'] == 'test']
control_group_ab = df_ab[df_ab['test_control'] == 'control']

# Perform the Mann-Whitney U test on 'demand', omitting NaN values
"""Note: It's important to handle NaN values. Depending on the data, we might want to exclude them."""

# u_statistic, p_value = stats.mannwhitneyu(test_group_ab['demand'], control_group_ab['demand'], alternative='two-sided', nan_policy='omit')
# Returns a TypeError: mannwhitneyu() got an unexpected keyword argument 'nan_policy'

mw_u_statistic_ab, mw_p_value_ab = stats.mannwhitneyu(test_group_ab['demand'].dropna(), control_group_ab['demand'].dropna(), alternative='two-sided')

# Interpretation
if mw_p_value_ab > alpha:
    print(f"\nMann Whitney U Statistic: {mw_u_statistic_ab}")
    print(f"P-value: {mw_p_value_ab}")
    print(f"\nP-value at: {mw_p_value_ab}. This is above alpha at {alpha}.")
    print(f"\nTherefore, we do not reject the null hypothesis (that there is no difference between the distributions of the two indepdent samples being compared).")
else:
    print(f"\nMann Whitney U Statistic: {mw_u_statistic_ab}")
    print(f"\nP-value: {mw_p_value_ab}")
    print(f"\nP-value at: {mw_p_value_ab}. This is below alpha at {alpha}.")
    print(f"\nTherefore, we reject the null hypothesis (that there is no difference between the distributions of the two indepdent samples being compared).")

**Results**

* Mann Whitney U Statistic: 2910485.0
* P-value: 0.7567487973086111

### TLDR Summary

When interpreting these values:
- A low **p-value** (<0.10) across either metric suggests the test group's performance is statistically significantly different from the control group's, indicating our test has likely influenced buying behavior.
- The $\chi$<sup>2</sup> and **t-statistics** provide a measure of the *magnitude* of this difference, but the p-value helps us understand if the difference is statistically significant.

### Understanding the Chi-square Statistic and p-value for Conversion

**Conversion Analysis:**

We created a binary column called 'buyer' which contains a 1 for members who made a purchase, and a 0 for members who did not.

We then compared the proportion of buyers to non-buyers between the test and control groups.

We used the **Chi-square (\(\chi^2\)) $\chi$<sup>2</sup> statistic** and **p-value** for this purpose.

- **Chi-square Statistic $\chi$<sup>2</sup>**: This number tells us how much the observed conversion rates deviate from what we would expect if there were no difference between the test and control groups.

  > A higher **$\chi$<sup>2</sup>**  value indicates a larger deviation from the expected, no-effect scenario.

- **p-value**: This value tells us the probability of observing our results (or more extreme) if there were truly no difference in conversion rates between the test and control groups.

  > A low p-value (<0.10 in our case) suggests that the observed differences are statistically significant, meaning it's unlikely they occurred by chance.

**Interpretation**:
- If the p-value is below our predefined threshold (e.g., $\alpha$ is 0.10), we conclude that there is a statistically significant difference in conversion rates between the test and control groups.
- This suggests that the test has had a measurable impact on conversion.
- Conversely, a p-value above $\alpha$ suggests that any observed differences in conversion rates could likely be due to random variation, and we do not have sufficient evidence to say the test affected conversion rates.

### Understanding the u-statistic and p-value for Demand


#### Mann-Whitney U Test: Detailed Explanation

* The Mann-Whitney U test assesses whether one of two groups of independent observations tends to have larger values than the other.
* Unlike the t-test, it does not require the data to follow a normal distribution or have equal variances, making it a robust alternative for ordinal data or non-normally distributed interval data.

#### U-Statistic Explained

The U-statistic quantifies the difference between the distributions of the two groups.
* It's calculated by ranking all observations from both groups together and then summing the ranks for each group.
* The actual U statistic value is the number of times observations in one group precede observations in the other group in this ranking.
* It reflects the magnitude of overlap between the two groups; **smaller values indicate less overlap and a greater difference between the groups**.

#### High U-Statistic and High P-Value: Interpretation

In our case:
1.  A high U-statistic (2910485.0)
2.  A high p-value (0.7567487973086111)

**High U-Statistic:**
  * This indicates a significant overlap in the ranks of the two groups.
  * In practical terms, it suggests that the distributions of the two groups are similar.
  > No group consistently ranks higher than the other across all observations.

**High P-Value:**
  * A high p-value suggests that the observed data are highly compatible with the null hypothesis of no difference between the groups.
  > In this context, the p-value indicates that any observed differences in the central tendencies of the two groups could easily occur by random chance alone.

#### Practical Interpretation

The combination of a high U-statistic and a high p-value in the context of a Mann-Whitney U test suggests that there is no statistically significant difference in the distribution of demand between the test and control groups.

The data do not provide sufficient evidence to reject the null hypothesis of equality in distributions.

The intervention or change tested does not have a statistically significant effect on demand, and any observed differences might be due to chance variation in the sample data.


### Understanding the t-statistic and p-value for Demand

**Demand Analysis**

Looks at whether the average amount spent (demand) per buyer differs significantly between the test and control groups. We utilized the **t-statistic** and **p-value** for assessing this.

- **t-statistic**: This value measures the size of the difference between the group means relative to the variation observed in the groups.
  > A larger absolute value of the t-statistic indicates a greater difference between the groups.
- **p-value**: Similar to the Chi-square test, the p-value here indicates the probability of observing our results (or more extreme) if there were truly no difference in average demand between the test and control groups.
  > A low p-value (<0.10 in our case) indicates that the differences in demand are statistically significant.

**Interpretation**:
-  If the p-value is below our predefined threshold (e.g., $\alpha$ is 0.10), it means there is a statistically significant difference in the average demand between the test and control groups, suggesting that the test has likely influenced buying behavior.
- A p-value above $\alpha$ indicates that any observed differences in average demand could be due to chance, and we do not have enough evidence to claim an effect of the test on demand.


### Visualizing the Underlying Distribution

In [ ]:
%python
import matplotlib.pyplot as plt
import seaborn as sns

# Plotting Demand Distribution
plt.figure(figsize=(12, 6))
sns.histplot(test_group['demand'].dropna(), color="skyblue", label="Test Group Demand", kde=True)
sns.histplot(control_group['demand'].dropna(), color="red", label="Control Group Demand", kde=True)
plt.title('Demand Distribution - Test vs Control')
plt.xlabel('Demand')
plt.ylabel('Density')
plt.legend()
plt.show()


# Plotting the buyer status in both test and control groups.
# Assumes a binary 'buyer' column where 1 indicates a buyer and 0 indicates a non-buyer.
%python
import matplotlib.pyplot as plt
import seaborn as sns

plt.figure(figsize=(12, 6))
plt.subplot(1, 2, 1)
sns.countplot(x='buyer', data=test_group)
plt.title('Buyer Status Counts - Test Group')
plt.xlabel('Buyer Status')
plt.ylabel('Count')

plt.subplot(1, 2, 2)
sns.countplot(x='buyer', data=control_group)
plt.title('Buyer Status Counts - Control Group')
plt.xlabel('Buyer Status')
plt.ylabel('Count')
plt.tight_layout()
plt.show()

# Conversion through Confidence Intervals
%python
import matplotlib.pyplot as plt
import scipy.stats as stats
import numpy as np

# Function to calculate the confidence interval
def proportion_confint(count, nobs, alpha=0.05, method='normal'):
    quantile = stats.norm.ppf(1 - alpha / 2.)
    center = count / nobs
    error = quantile * np.sqrt(center * (1 - center) / nobs)
    return center - error, center + error

# Calculating conversion rate and confidence intervals
test_success = test_group['buyer'].sum()
test_total = test_group['buyer'].count()
test_cr, test_ci_lower, test_ci_upper = test_success / test_total, *proportion_confint(test_success, test_total)

control_success = control_group['buyer'].sum()
control_total = control_group['buyer'].count()
control_cr, control_ci_lower, control_ci_upper = control_success / control_total, *proportion_confint(control_success, control_total)

# Plotting
fig, ax = plt.subplots()
ax.bar(['Test', 'Control'], [test_cr, control_cr], yerr=[[test_cr - test_ci_lower, control_cr - control_ci_lower],[test_ci_upper - test_cr, control_ci_upper - control_cr]], capsize=10)
ax.set_ylabel('Conversion Rate')
ax.set_title('Conversion Rate with 95% Confidence Intervals')
plt.show()


""" The proportion_confint function calculates confidence intervals for each group's conversion rate.
These intervals are then plotted as error bars on a bar chart, giving a clearer picture of the data's variability
and how the groups compare. """

#### Error

KeyError: 'test'

Code: `lift = ((aggregated_metrics.loc['test', metric] - aggregated_metrics.loc['control', metric]) /`

Here is my code:


**Fix:**

*Adjust Access in Step 10: If 'test' and 'control' are indeed column names after transposition, your current loop should work. If they are not, ensure they are correctly set during the transposition.*

In [ ]:
# Assuming aggregated_metrics is transposed correctly and 'test', 'control' are column names
if 'test' in aggregated_metrics.columns and 'control' in aggregated_metrics.columns:
    for metric in aggregated_metrics.index:  # Assuming metrics are now index after transpose
        if metric not in ['Conversion Rate', 'Demand per Buyer']:
            test_val = aggregated_metrics.loc[metric, 'test']
            control_val = aggregated_metrics.loc[metric, 'control']
            lift = ((test_val - control_val) / control_val) * 100 if control_val != 0 else np.nan
            print(metric, lift)  # For demonstration, replace with how you want to store this lift value
else:
    print("Column names 'test' and 'control' not found. Please check DataFrame structure.")


In [ ]:
# Step 0: Import necessary libraries
import numpy as np
import pandas as pd
from scipy import stats

# Assuming 'df' is your DataFrame with necessary columns

# Step 1 through Step 7 remain unchanged as you've described

# Step 8: Perform t-test for 'Conversion Rate' and 'Demand per Buyer'
# This step also remains as you've described, performing t-tests for the specific metrics

# Step 9: Transpose the aggregated metrics for easier handling
# This step remains as you've described

# Integrating Steps 10 and 11 with modifications for Step 12

# Step 10: Calculate lift for all metrics except 'Conversion Rate' and 'Demand per Buyer'
# Adjustments to include the calculation of t_stat and p_value directly in the DataFrame for specific metrics
aggregated_metrics['Lift'] = np.nan
aggregated_metrics.loc['Conversion Rate', 't_stat'] = t_stat_conversion
aggregated_metrics.loc['Conversion Rate', 'p_value'] = p_value_conversion
aggregated_metrics.loc['Demand per Buyer', 't_stat'] = t_stat_demand_per_buyer
aggregated_metrics.loc['Demand per Buyer', 'p_value'] = p_value_demand_per_buyer

# Calculate lift only for metrics other than specified
for metric in aggregated_metrics.index:
    if metric not in ['Conversion Rate', 'Demand per Buyer']:
        control = aggregated_metrics.loc[metric, 'Control']
        test = aggregated_metrics.loc[metric, 'Test']
        if control != 0:  # Prevent division by zero
            aggregated_metrics.loc[metric, 'Lift'] = ((test - control) / control) * 100

# Step 12: Final preparation of the scorecard
# Since 'Lift', 't_stat', and 'p_value' are now part of aggregated_metrics, there's no need for a separate action here

# Step 13: Display the final scorecard with adjustments
# Convert aggregated metrics to a format suitable for display
scorecard = aggregated_metrics[['Test', 'Control', 'Lift', 't_stat', 'p_value']].reset_index().rename(columns={'index': 'Metric'})
scorecard.fillna('', inplace=True)  # Replace NaN with empty strings for cleanliness

print(scorecard)


In [ ]:
# simple calculation to replicate their results in Excel

from scipy.stats import norm

# Given values
z_score = 0.43
alpha = 0.10

# Calculating the p-value for the right-tailed test
p_value = 1 - norm.cdf(z_score)

# Check if the result is statistically significant
is_significant = p_value < alpha

p_value, is_significant

### Statistical Power Analysis

#### Minimum Detectable Effect

In [ ]:
from statsmodels.stats.power import NormalIndPower
from statsmodels.stats.proportion import proportion_effectsize

# Example data points
baseline_conversion_rate = 0.10  # Control group conversion rate
mde = 0.02  # Minimum detectable effect (a 2% increase in conversion rate)
alpha = 0.05  # Significance level
power = 0.8  # Desired power
current_sample_size_per_group = 500  # Number of observations in each group

# Calculate the effect size based on baseline and MDE
effect_size = proportion_effectsize(baseline_conversion_rate, baseline_conversion_rate + mde)

# Initialize the power analysis object for a z-test
power_analysis = NormalIndPower()

# Calculate required sample size
required_sample_size = power_analysis.solve_power(effect_size=effect_size, power=power, alpha=alpha, ratio=1, alternative='larger')

print(f"Required Sample Size per Group: {required_sample_size}")
print(f"Current Sample Size per Group: {current_sample_size_per_group}")

if current_sample_size_per_group >= required_sample_size:
    print("You have collected enough data to reliably detect the MDE with the desired power and significance level.")
else:
    remaining_sample_size = required_sample_size - current_sample_size_per_group
    print(f"More data needed. You need approximately {remaining_sample_size} more observations per group to reach the desired power level.")


Required Sample Size per Group: [10.]
Current Sample Size per Group: 500
You have collected enough data to reliably detect the MDE with the desired power and significance level.


/usr/local/lib/python3.10/dist-packages/statsmodels/stats/power.py:133: RuntimeWarning: invalid value encountered in sqrt
  pow_ = stats.norm.sf(crit - d*np.sqrt(nobs)/sigma)
/usr/local/lib/python3.10/dist-packages/statsmodels/stats/power.py:525: ConvergenceWarning: 
Failed to converge on a solution.

  warnings.warn(convergence_doc, ConvergenceWarning)


#### w/ Confidence Interval

In [ ]:
import numpy as np
import scipy.stats as stats

# Observed means, standard deviations, and sample sizes from your test and control groups
mean_test = 5.1  # Mean of the test group
mean_control = 5.0  # Mean of the control group
std_test = 1.5  # Standard deviation of the test group
std_control = 1.4  # Standard deviation of the control group
n_test = 150  # Sample size of the test group
n_control = 150  # Sample size of the control group

# Calculate pooled standard deviation
std_pooled = np.sqrt(((n_test - 1) * std_test ** 2 + (n_control - 1) * std_control ** 2) / (n_test + n_control - 2))

# Calculate the standard error of the difference in means
se_diff = std_pooled * np.sqrt(1/n_test + 1/n_control)

# Determine the Z-score for a 90% CI
z_score = stats.norm.ppf(0.95)  # This is for a 90% CI, using the 95th percentile

# Calculate the confidence intervals
ci_lower = (mean_test - mean_control) - z_score * se_diff
ci_upper = (mean_test - mean_control) + z_score * se_diff

# Display the confidence interval
print(f"90% Confidence Interval for the difference in means: [{ci_lower:.3f}, {ci_upper:.3f}]")

# Continue with MDE calculations
effect_size = (mean_test - mean_control) / std_pooled
alpha = 0.05  # Significance level for a one-tailed test
power = 0.8  # Desired power

# Calculate the sample size required for the given MDE
sample_size_per_group = stats.norm.ppf(1 - alpha) ** 2 * (std_pooled ** 2) * ((1 / effect_size) ** 2)
print(f"Sample size per group for MDE ({effect_size*100:.2f}% effect size) with 80% power and 5% significance level: {np.ceil(sample_size_per_group)}")


##### Things to look for in the Confidence Interval


1.   **Width of the CI:** The wider the confidence interval, the more uncertainty there is about the precise effect of the treatment.  
> A narrow CI indicates more precision about the treatment's effect size.
2. **If CI Includes 0:** If the confidence interval includes 0 (*e.g., [-0.2, 0.3]*), it means there's a possibility that there is no real difference between the test and control groups.
> The treatment effect could be positive, negative, or non-existent, and we cannot be confident that the treatment had a significant effect.
3. **Effect Size:** Even if a result is statistically significant (e.g., the CI does not include 0), the effect size matters for practical decisions.
> If the CI shows a very small effect size that is not meaningful for our business objectives, we might decide not to implement the change even though it showed a statistically significant difference.



In [ ]:
# prompt: Generate a one-tailed t-test in python

from scipy.stats import ttest_1samp

# Sample data
data = [10, 12, 14, 15, 16, 17, 18, 19, 20]

# Set the null hypothesis mean
null_mean = 15

# Perform the one-tailed t-test
t_statistic, p_value = ttest_1samp(data, null_mean, alternative="greater")

# Print the results
print("t-statistic:", t_statistic)
print("p-value:", p_value)

# Interpret the results
if p_value < 0.05:
    print("Reject the null hypothesis. The sample mean is significantly greater than the null mean.")
else:
    print("Fail to reject the null hypothesis. The sample mean is not significantly greater than the null mean.")


### Test Code for AB Period Measurment w/ Combined Platforms

##### Correct Results



In [ ]:
-----------------------------------------------------------------------------------------------------------------------------------
-- AB Period
-----------------------------------------------------------------------------------------------------------------------------------
WITH

-- use audience file to determine send dates when holdout audience is used
tc_app_sends AS (
    SELECT upm_id
    , CASE WHEN final_exposure_or_holdout like ${hivevar:node_tst_str} THEN 'test'
        -- WHEN final_exposure_or_holdout like ${hivevar:node_ctl_str_2} THEN 'holdout'
        WHEN final_exposure_or_holdout like ${hivevar:node_ctl_str} THEN 'control'
      ELSE 'na' END AS test_control
    , LEFT(timestamp,10) AS campaign_send_dt
    , CAST(LEFT(timestamp,19) AS TIMESTAMP) AS campaign_send_ts
    FROM ${hivevar:aud_table} aud
    ANTI JOIN contam_base c ON aud.upm_id = c.upm_id
    ANTI JOIN gwan13.bot_master_nike_com bot ON aud.upm_id = bot.upm_id
    ANTI JOIN aud_select_workspace.reseller_new slr ON aud.upm_id = slr.upm_user_id
    WHERE CAST(LEFT(timestamp,10) AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND ${hivevar:msmt_end_dt}
)

, tc_first_last_send AS (
  SELECT upm_id
  , MIN(campaign_send_dt) as first_send_dt
  , MAX(campaign_send_dt) as last_send_dt
  FROM tc_app_sends
  GROUP BY 1
)

, test_control_table AS (
    SELECT aud.*
    , fls.first_send_dt
    , fls.last_send_dt
    , LEAD(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt) AS next_send_dt
    , LAG(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt) AS previous_send_dt
    , (int(to_timestamp(campaign_send_dt)) - int(to_timestamp(LEAD(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt))))/86400 AS difference_next_send_days
    , (int(to_timestamp(campaign_send_dt)) - int(to_timestamp(LAG(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt))))/86400 AS difference_previous_send_days
    FROM tc_app_sends aud
    INNER JOIN tc_first_last_send fls ON aud.upm_id = fls.upm_id
)

, view_dol AS (
  SELECT dol.upm_id
  , order_nbr
  , order_dt
  , origl_ordered_qty
  , grd_amt_excl_tax_usd
  , line_of_business_desc
  , univ_div_desc
  , first_send_dt
  , last_send_dt
  FROM awight.na_digital_order_line_snapshot dol
  INNER JOIN tc_first_last_send fls ON dol.upm_id = fls.upm_id
  WHERE rec_excl_ind = 0
    AND ttl_demand_ind = 1
    AND univ_cat_desc <> 'Converse'
    AND dol.upm_id IS NOT NULL
    AND order_dt BETWEEN DATE_SUB(first_send_dt,730) AND DATE_ADD(last_send_dt,7)
    AND (rtn_qty = 0 or rtn_qty IS NULL)
)

-- AA period purchase. dynamically based on when the member receives their first communication. Only returns demand for members in the
, aa_purchase AS (
  SELECT dol.upm_id
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

, demand_bucket AS (
  SELECT tct.upm_id
  , tct.test_control
  , aa.upm_id as upm_id_aa_buyer
  , aa.demand
  , aa.orders
  , aa.units
  , CASE
    WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
    WHEN aa.demand < 50 THEN '000-050'
    WHEN aa.demand < 100 THEN '050-100'
    WHEN aa.demand < 200 THEN '100-200'
    WHEN aa.demand < 500 THEN '200-500'
    WHEN aa.demand < 1000 THEN '500-1000'
    WHEN aa.demand >=1000 THEN '1000+'
    END AS demand_per_buyer_bucket
  , PERCENT_RANK() OVER (
    PARTITION BY test_control
    , CASE
      WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
      WHEN aa.demand < 50 THEN '000-050'
      WHEN aa.demand < 100 THEN '050-100'
      WHEN aa.demand < 200 THEN '100-200'
      WHEN aa.demand < 500 THEN '200-500'
      WHEN aa.demand < 1000 THEN '500-1000'
      WHEN aa.demand >=1000 THEN '1000+'
      END ORDER BY RANDOM(0)
    ) AS percent_rank
  FROM test_control_table tct
  LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
)

, balanced_aud_aa as (
  SELECT DISTINCT upm_id
  FROM demand_bucket db
  -- remove outliers
  WHERE (demand < 2400 OR demand IS NULL)
-- To rebalance, paste text block from template in place of defaults
AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 0.97632 )
OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1 )
OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 0.99509 )
OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1 )
OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 0.90741 )
OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 0.98161 )
OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 0.99998 )
OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 0.98934 )
OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 0.98953 )
OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
))

, mhub as (
 SELECT source_id as upm_id
 , nuid
 , member_id
 , preferred_gender
 FROM MEMBER.MEMBER_HUB
 WHERE ctry_2_cd = 'US'
 AND lower(dpa_status) IN ('activate','reactivate')
)

-- calculate a dynamic member activity window for each communication received by a member
-- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
-- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
, member_activities as (
  SELECT mgd.member_id
  ,SUM(logged_in_visits_count) AS visits
  ,SUM(pdp_favorite_count) AS pdp_favorite_count
  ,SUM(add_to_cart_count) AS add_to_cart_count
  ,SUM(physical_activity_count) AS physical_activity
  ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
  ,SUM(eod_chat_count) AS NEOD_Session
  FROM member_agd mgd --aud_select_workspace.member_agg_member_growth_daily mgd
  INNER JOIN mhub mh ON mgd.member_id = mh.member_id
  INNER JOIN test_control_table tct ON mh.upm_id = tct.upm_id
  WHERE mgd.activity_dt >= tct.campaign_send_dt
    AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
    AND mgd.preferred_retail_geo = 'NA'
    AND mgd.experience_name != 'agnostic'
  GROUP BY 1
)

-- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
, ab_purchase as (
  SELECT dol.upm_id as upm_id_ab_buyer
  , order_dt as order_ts
  , order_nbr
  ,SUM(grd_amt_excl_tax_usd) AS demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
  ,COUNT(DISTINCT order_nbr) AS orders
  ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN last_send_dt AND DATE_ADD(last_send_dt,7)
  GROUP BY 1,2,3
)

-- counting the first order demand after each send
, reset_per_campaign_sent AS (
 SELECT a.*
  ,CONCAT(a.upm_id, ' ', a.campaign_send_dt ) AS touchpoint_per_member
  FROM test_control_table a -- should it be created based on buyers/
  WHERE upm_id IN (SELECT DISTINCT upm_id_ab_buyer FROM ab_purchase )
  )


, aud_order_join AS    (
  SELECT a.*
  , b.*
  ,round(((bigint(to_timestamp(a.campaign_send_ts)))-(bigint(to_timestamp(b.order_ts))))/(60),3) AS order_send_diff_min
  ,RANK() OVER(PARTITION BY  a.touchpoint_per_member  ORDER BY (((bigint(to_timestamp(a.campaign_send_ts)))-(bigint(to_timestamp(b.order_ts))))/(60)) DESC, a.campaign_send_dt ) AS nearest_order
  FROM reset_per_campaign_sent a
  LEFT JOIN ab_purchase b
  ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(CAST(a.`campaign_send_dt` AS DATE), CAST(LEAST(7, COALESCE(a.`difference_next_send_days`, 7)) AS INT)))
  ORDER BY a.upm_id, a.campaign_send_dt ASC
)


-- Aggregate each distinct send, attributing only the nearest order.
-- This prevent double counting of orders when a members receives multiple sends within the measurement window.
, aud_order_agg as (
SELECT upm_id AS upm_id_ab_buyer
  ,SUM(demand) AS demand
  ,SUM(base_FW_demand) AS base_FW_demand
  ,SUM(base_AP_demand) AS base_AP_demand
  ,SUM(base_EQ_demand) AS base_EQ_demand
  ,SUM(clr_FW_demand) AS clr_FW_demand
  ,SUM(clr_AP_demand) AS clr_AP_demand
  ,SUM(clr_EQ_demand) AS clr_EQ_demand
  ,SUM(launch_FW_demand) AS launch_FW_demand
  ,SUM(launch_AP_demand) AS launch_AP_demand
  ,SUM(launch_EQ_demand) AS launch_EQ_demand
  ,SUM(rest_demand) AS rest_demand
  ,SUM(orders) AS orders
  ,SUM(units) AS units
FROM aud_order_join a
WHERE nearest_order = 1
AND demand IS NOT NULL -- additional filter for extraneous records
GROUP BY 1
ORDER BY 1
)

, historical_purchase AS (
  SELECT aoa.upm_id_ab_buyer as upm_id
  , COUNT(DISTINCT order_nbr) as histOrders
  FROM aud_order_agg aoa  -- eligible audience
  INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

/*
--set the correct usage platform variable at the top of the notebook
, platform_usage as (
  SELECT DISTINCT tct.upm_id
  , CASE WHEN CAST(first_${hivevar:usage_platform}_activity_dt AS DATE) BETWEEN first_send_dt AND DATE_ADD(last_send_dt,7) THEN 1
         ELSE 0
    END AS platform_user
  FROM aud_select_workspace.gck_first_and_last_touchpoint flt
  INNER JOIN tc_first_last_send fls ON flt.upm_id = fls.upm_id --need first/last send values as activity window
)
*/
---------------------
-- MAIN QUERY
----------------------
SELECT z.test_control
  -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
  ,COUNT(DISTINCT z.upm_id) AS total_members
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) AS buying_members
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) / COUNT(DISTINCT z.upm_id) AS conversion_rate
  ,AVG(demand) AS avg_demand
  ,stddev(demand) AS std_demand
  -- secondary metrics ON order double click
  ,SUM(demand) / SUM(orders) AS AOV
  ,SUM(demand) / SUM(units) AS AUR
  ,SUM(units) / SUM(orders) AS UPT


  -- secondary metrics ON site/app engagement
  ,AVG(visits) AS avg_site_app_visits
  ,AVG(PDP_FAVORITE_COUNT) AS avg_PDP_FAVORITE_COUNT
  ,AVG(ADD_TO_CART_COUNT) AS avg_ADD_TO_CART_COUNT
  ,AVG(physical_activity) AS avg_physical_activity

  -- secondary metrics ON margin
  ,(AVG(base_FW_demand) * 0.38 + AVG(base_AP_demand) * 0.28 + AVG(base_EQ_demand) * 0.32
    + AVG(clr_FW_demand) * 0.23 + AVG(base_AP_demand) * 0.17 + AVG(clr_EQ_demand) * 0.12
    + AVG(launch_FW_demand) * 0.25 + AVG(launch_AP_demand) * 0.08 + AVG(launch_EQ_demand) * 0.19
    + AVG(rest_demand) * 0.3
  ) AS avg_margin
  ,AVG(base_FW_demand) AS base_FW_demand
  ,AVG(base_AP_demand) AS base_AP_demand
  ,AVG(base_EQ_demand) AS base_EQ_demand
  ,AVG(clr_FW_demand) AS clr_FW_demand
  ,AVG(clr_AP_demand) AS clr_AP_demand
  ,AVG(clr_EQ_demand) AS clr_EQ_demand
  ,AVG(launch_FW_demand) AS launch_FW_demand
  ,AVG(launch_AP_demand) AS launch_AP_demand
  ,AVG(launch_EQ_demand) AS launch_EQ_demand
  ,AVG(rest_demand) AS rest_demand
    -- secondary metrics ON order history
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS DPB_1_prior
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS DPB_2_prior


FROM test_control_table z
INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

INNER JOIN mhub mh ON z.upm_id = mh.upm_id
LEFT JOIN member_activities a ON mh.member_id = a.member_id

LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

--LEFT JOIN platform_usage plt ON z.upm_id = plt.upm_id

WHERE (demand < 600 OR demand IS NULL)
AND test_control != 'na'

GROUP BY 1
ORDER BY 1 DESC;

##### Incorrect Results


In [ ]:
WITH

    -- use audience file to determine send dates when holdout audience is used
    tc_app_sends AS (
      SELECT upm_id
      , mhub.nuid
      , CASE WHEN final_exposure_or_holdout like ${hivevar:node_tst_str} THEN 'test'
            WHEN final_exposure_or_holdout like ${hivevar:node_ctl_str} THEN 'control'
        ELSE 'na' END AS test_control
      , LEFT(timestamp,10) AS campaign_send_dt
      , CAST(LEFT(timestamp,19) AS TIMESTAMP) AS campaign_send_ts
      FROM ${hivevar:aud_table} aud
      INNER JOIN member.member_hub mhub ON aud.upm_id = mhub.source_id
      ANTI JOIN contam_base c ON aud.upm_id = c.upm_id
      ANTI JOIN gwan13.bot_master_nike_com bot ON aud.upm_id = bot.upm_id
      ANTI JOIN aud_select_workspace.reseller_new slr ON aud.upm_id = slr.upm_user_id
      WHERE CAST(LEFT(timestamp,10) AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND ${hivevar:msmt_end_dt}
      GROUP BY 1,2,3,4,5
  )

    , tc_first_last_send AS (
      SELECT upm_id
      , nuid
      , MIN(campaign_send_dt) as first_send_dt
      , MAX(campaign_send_dt) as last_send_dt
      FROM tc_app_sends
      GROUP BY 1,2
    )

    , test_control_table AS (
        SELECT aud.*
        , fls.first_send_dt
        , fls.last_send_dt
        , LEAD(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt) AS next_send_dt
        , LAG(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt) AS previous_send_dt
        , (int(to_timestamp(campaign_send_dt)) - int(to_timestamp(LEAD(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt))))/86400 AS difference_next_send_days
        , (int(to_timestamp(campaign_send_dt)) - int(to_timestamp(LAG(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt))))/86400 AS difference_previous_send_days
        FROM tc_app_sends aud
        INNER JOIN tc_first_last_send fls ON aud.upm_id = fls.upm_id
    )

    , view_dol AS (
      SELECT dol.upm_id
      , order_nbr
      , order_dt
      , origl_ordered_qty
      , grd_amt_excl_tax_usd
      , line_of_business_desc
      , univ_div_desc
      , first_send_dt
      , last_send_dt
      FROM awight.na_digital_order_line_snapshot dol
      INNER JOIN tc_first_last_send fls ON dol.upm_id = fls.upm_id
      WHERE rec_excl_ind = 0
        AND ttl_demand_ind = 1
        AND univ_cat_desc <> 'Converse'
        AND dol.upm_id IS NOT NULL
        AND order_dt BETWEEN DATE_SUB(first_send_dt,730) AND DATE_ADD(last_send_dt,7)
        AND (rtn_qty = 0 or rtn_qty IS NULL)
    )

    -- AA period purchase. dynamically based on when the member receives their first communication. Only returns demand for members in the
    , aa_purchase AS (
      SELECT dol.upm_id
        ,SUM(grd_amt_excl_tax_usd) AS demand
        ,COUNT(DISTINCT order_nbr) AS orders
        ,SUM(origl_ordered_qty) AS units
      FROM view_dol dol
      WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
      GROUP BY 1
    )

    , demand_bucket AS (
      SELECT tct.upm_id
      , tct.test_control
      , aa.upm_id as upm_id_aa_buyer
      , aa.demand
      , aa.orders
      , aa.units
      , CASE
        WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
        WHEN aa.demand < 50 THEN '000-050'
        WHEN aa.demand < 100 THEN '050-100'
        WHEN aa.demand < 200 THEN '100-200'
        WHEN aa.demand < 500 THEN '200-500'
        WHEN aa.demand < 1000 THEN '500-1000'
        WHEN aa.demand >=1000 THEN '1000+'
        END AS demand_per_buyer_bucket
      , PERCENT_RANK() OVER (
        PARTITION BY test_control
        , CASE
          WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
          WHEN aa.demand < 50 THEN '000-050'
          WHEN aa.demand < 100 THEN '050-100'
          WHEN aa.demand < 200 THEN '100-200'
          WHEN aa.demand < 500 THEN '200-500'
          WHEN aa.demand < 1000 THEN '500-1000'
          WHEN aa.demand >=1000 THEN '1000+'
          END ORDER BY RANDOM(0)
        ) AS percent_rank
      FROM test_control_table tct
      LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
    )

    , balanced_aud_aa as (
      SELECT DISTINCT upm_id
      FROM demand_bucket db
      -- remove outliers
      WHERE (demand < 2400 OR demand IS NULL)
    -- To rebalance, paste text block from template in place of defaults
    AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 0.97632 )
    OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1 )
    OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 0.99509 )
    OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1 )
    OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 0.90741 )
    OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 0.98161 )
    OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 0.99998 )
    OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
    OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 0.98934 )
    OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
    OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 0.98953 )
    OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
    OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
    OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
    ))

    , mhub AS (
      SELECT source_id as upm_id
      , nuid
      , member_id
      , preferred_gender
      FROM MEMBER.MEMBER_HUB
      WHERE ctry_2_cd = 'US'
      AND lower(dpa_status) IN ('activate','reactivate')
    )

    -- calculate a dynamic member activity window for each communication received by a member
    -- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
    -- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
    , member_activities as (
      SELECT mgd.member_id
      ,SUM(logged_in_visits_count) AS visits
      ,SUM(pdp_favorite_count) AS pdp_favorite_count
      ,SUM(add_to_cart_count) AS add_to_cart_count
      ,SUM(physical_activity_count) AS physical_activity
      ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
      ,SUM(eod_chat_count) AS NEOD_Session
      FROM member_agd mgd --aud_select_workspace.member_agg_member_growth_daily mgd
      INNER JOIN mhub mh ON mgd.member_id = mh.member_id
      INNER JOIN test_control_table tct ON mh.upm_id = tct.upm_id
      WHERE mgd.activity_dt >= tct.campaign_send_dt
        AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
        AND mgd.preferred_retail_geo = 'NA'
        AND mgd.experience_name != 'agnostic'
      GROUP BY 1
    )

    -- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
    , ab_purchase as (
      SELECT dol.upm_id as upm_id_ab_buyer
      , order_dt as order_ts
      , order_nbr
      ,SUM(grd_amt_excl_tax_usd) AS demand
      ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
      ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
      ,COUNT(DISTINCT order_nbr) AS orders
      ,SUM(origl_ordered_qty) AS units
      FROM view_dol dol
      WHERE CAST(order_dt AS DATE) BETWEEN last_send_dt AND DATE_ADD(last_send_dt,7)
      GROUP BY 1,2,3
    )

    -- counting the first order demand after each send
    , reset_per_campaign_sent AS (
    SELECT a.*
      ,CONCAT(a.upm_id, ' ', a.campaign_send_dt ) AS touchpoint_per_member
      FROM test_control_table a -- should it be created based on buyers/
      WHERE upm_id IN (SELECT DISTINCT upm_id_ab_buyer FROM ab_purchase )
      )


    , aud_order_join AS    (
      SELECT a.*
      , b.*
      ,round(((bigint(to_timestamp(a.campaign_send_ts)))-(bigint(to_timestamp(b.order_ts))))/(60),3) AS order_send_diff_min
      ,RANK() OVER(PARTITION BY  a.touchpoint_per_member  ORDER BY (((bigint(to_timestamp(a.campaign_send_ts)))-(bigint(to_timestamp(b.order_ts))))/(60)) DESC, a.campaign_send_dt ) AS nearest_order
      FROM reset_per_campaign_sent a
      LEFT JOIN ab_purchase b
      ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(CAST(a.`campaign_send_dt` AS DATE), CAST(LEAST(7, COALESCE(a.`difference_next_send_days`, 7)) AS INT)))
      ORDER BY a.upm_id, a.campaign_send_dt ASC
    )


    -- Aggregate each distinct send, attributing only the nearest order.
    -- This prevents double counting of orders when a members receives multiple sends within the measurement window.
    , aud_order_agg as (
    SELECT upm_id AS upm_id_ab_buyer
      ,SUM(demand) AS demand
      ,SUM(base_FW_demand) AS base_FW_demand
      ,SUM(base_AP_demand) AS base_AP_demand
      ,SUM(base_EQ_demand) AS base_EQ_demand
      ,SUM(clr_FW_demand) AS clr_FW_demand
      ,SUM(clr_AP_demand) AS clr_AP_demand
      ,SUM(clr_EQ_demand) AS clr_EQ_demand
      ,SUM(launch_FW_demand) AS launch_FW_demand
      ,SUM(launch_AP_demand) AS launch_AP_demand
      ,SUM(launch_EQ_demand) AS launch_EQ_demand
      ,SUM(rest_demand) AS rest_demand
      ,SUM(orders) AS orders
      ,SUM(units) AS units
    FROM aud_order_join a
    WHERE nearest_order = 1
    AND demand IS NOT NULL -- additional filter for extraneous records
    GROUP BY 1
    ORDER BY 1
    )

    , historical_purchase AS (
      SELECT aoa.upm_id_ab_buyer as upm_id
      , COUNT(DISTINCT order_nbr) as histOrders
      FROM aud_order_agg aoa  -- eligible audience
      INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
      WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
      GROUP BY 1
    )

    -- returns all the instances of the campaign codes for each member along with the action types and revenue if applicable
    , email_engage as (
      SELECT DISTINCT fer.upm_id
      , LEFT(event_dttm,10) AS engage_dt
      , action_type
      , revenue_dollar
      , sent_ind
      , open_ind
      , click_ind
      , bounce_ind
      , optout_ind
      , purchase_ind
      FROM awight.na_email_userengagement_snapshot_1 fer
      INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id
      WHERE record_dt BETWEEN ${hivevar:msmt_start_dt} AND DATE_ADD(${hivevar:msmt_end_dt},7)
      AND action_type IN ('open','click','send','bounce','optout')
      AND comm_id IN (${hivevar:email_commID_str})
      AND CAST(LEFT(event_dttm,10) AS DATE) BETWEEN last_send_dt AND DATE_ADD(last_send_dt,7)
      AND fer.upm_id IS NOT NULL
      GROUP BY 1,2,3,4,5,6,7,8,9,10
      )


    -- calculates the the total number of opens, clicks, purchases and last touch attributed demand for each member for the campaign
    -- responses are only considered if they occur between when the message was sent and 7 days later.
    , email_calc AS (
        SELECT fer.upm_id
        , MAX(sent_ind) AS email_sends
        , MAX(open_ind) AS email_opens
        , MAX(click_ind) AS email_clicks
        , MAX(bounce_ind) AS email_bounces
        , MAX(optout_ind) AS email_optouts
        , MAX(purchase_ind) AS email_purch
        , SUM(CASE WHEN action_type = 'ecommerce-purchase' THEN revenue_dollar ELSE 0 END) AS email_demand
        FROM email_engage fer
        INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id
        GROUP BY 1
    )

    , email_engage_agg as (
      SELECT upm_id
      , SUM(email_sends) AS email_sends
      , SUM(email_opens) AS email_opens
      , SUM(email_clicks) AS email_clicks
      , SUM(email_bounces) AS email_bounces
      , SUM(email_optouts) AS email_optouts
      , SUM(email_purch) AS email_purch
      , SUM(email_demand) AS email_demand
      FROM email_calc
      GROUP BY 1
    )

    , push_engagement AS (
        SELECT -- fpr.upm_id
          fpr.nuid
        , tc.last_send_dt
        , COUNT(DISTINCT CASE WHEN event_dttm IS NOT NULL AND action_type = 'send' THEN cp_cd END) AS pushes
        , COUNT(DISTINCT CASE WHEN event_dttm IS NOT NULL AND action_type = 'open' OR action_type = 'inferred_open' THEN cp_cd
                              -- WHEN evemt_dttm IS NOT NULL AND action_type = 'inferred_open' THEN cp_cd
                          END) AS open_pushes
        FROM awight.na_push_userengagement_snapshot_1 AS fpr
        INNER JOIN test_control_table tc ON tc.nuid = fpr.nuid
        -- INNER JOIN test_control_table tc ON tc.upm_id = fpr.upm_id
        WHERE CAST(event_dttm AS DATE) BETWEEN last_send_dt AND DATE_ADD(last_send_dt,7)
        AND comm_id IN (${hivevar:push_commID_str1}, ${hivevar:push_commID_str2})
        AND action_type IN ('send','open','inferred_open')
        -- AND fpr.upm_id IS NOT NULL
        AND retail_geo = 'NA' -- optional
        GROUP BY 1,2
    )

/*

, app_metrics_feed AS (
    SELECT upm_id
  , SUM(tot_editorial_hub_card_shown_count
                + tot_stream_card_shown_count
                + tot_editorial_card_shown_count
                + tot_mhome_product_marketing_card_shown_count
                + tot_mhome_recommended_product_card_shown_count
        ) as total_card_views
  , 	SUM(tot_editorial_hub_card_clicked_count
                + tot_stream_card_clicked_count
                + tot_editorial_card_clicked_count
                + tot_mhome_product_marketing_card_clicked_count
                + tot_mhome_recommended_product_card_clicked_count
        ) as total_card_taps
    FROM content.agg_user_session_content_activity_daily
    WHERE thread_id IN (${hivevar:feedID_str})
      AND CAST(session_start_date AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND DATE_ADD(${hivevar:msmt_end_dt},7)
      AND app_version IS NOT NULL
      AND card_key != 'UNKNOWN'
    GROUP BY 1
  )


, app_metrics_thread AS (
    SELECT upm_id
    ,SUM(tot_thread_clicks) as thread_taps
    ,SUM(tot_thread_viewed_count) as thread_views
    ,SUM(tot_thread_comment_added_count) as thread_comment_add
    FROM content.agg_user_session_content_activity_daily
    WHERE thread_id IN (${hivevar:threadID_str})
      AND CAST(session_start_date AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND DATE_ADD(${hivevar:msmt_end_dt},7)
      AND app_version IS NOT NULL
      AND card_key != 'UNKNOWN'
    GROUP BY 1
)
*/

    ---------------------
    -- MAIN QUERY
    ----------------------
    SELECT  z.upm_id
          , z.test_control

          -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
          , SUM(demand) AS demand

          -- secondary metrics ON order double click
          , SUM(orders) AS orders
          , SUM(units) AS units

          -- secondary metrics ON emails
          , SUM(email_sends) AS email_sent
          , SUM(email_opens) AS email_opens
          , SUM(email_clicks) AS email_clicks
          , SUM(email_purch) AS email_purch
          , SUM(email_demand) AS email_demand

          -- secondary metrics ON app push/engagement
          , (CASE WHEN SUM(pushes) = 0 THEN 0 ELSE SUM(pushes) END) AS push_sent
          , (CASE WHEN SUM(pushes) = 0 THEN 0 ELSE SUM(open_pushes) END) AS push_opens
          , (CASE WHEN SUM(pushes) = 0 THEN 0 ELSE SUM(open_pushes) / SUM(pushes) END) AS push_open_rate
          /*
          , SUM(total_card_views) as card_views
          , SUM(total_card_taps) as card_taps
          , SUM(thread_views) as thread_views
          , SUM(thread_taps) as thread_taps
          , SUM(thread_comment_add) as thread_comment_add
          */

          -- secondary metrics ON site/app engagement
          , SUM(visits) AS site_app_visits
          , SUM(PDP_FAVORITE_COUNT) AS PDP_FAVORITE_COUNT
          , SUM(ADD_TO_CART_COUNT) AS ADD_TO_CART_COUNT
          , SUM(physical_activity) AS physical_activity

          -- secondary metrics ON margin
          , SUM(base_FW_demand) AS base_FW_demand
          , SUM(base_AP_demand) AS base_AP_demand
          , SUM(base_EQ_demand) AS base_EQ_demand
          , SUM(clr_FW_demand) AS clr_FW_demand
          , SUM(clr_AP_demand) AS clr_AP_demand
          , SUM(clr_EQ_demand) AS clr_EQ_demand
          , SUM(launch_FW_demand) AS launch_FW_demand
          , SUM(launch_AP_demand) AS launch_AP_demand
          , SUM(launch_EQ_demand) AS launch_EQ_demand
          , SUM(rest_demand) AS rest_demand

          -- secondary metrics ON order history
          , SUM(histOrders) AS histOrders
          ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
          ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder

          -- optional first usage kpi
          --  ,SUM(platform_user) AS first_usage

    FROM test_control_table z
    INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

    INNER JOIN mhub mh ON z.upm_id = mh.upm_id
    LEFT JOIN member_activities a ON mh.member_id = a.member_id

    LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
    LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

    LEFT JOIN email_engage_agg cra ON z.upm_id = cra.upm_id
/*
    LEFT JOIN email_delivery_agg cda ON z.upm_id = cda.upm_id
    LEFT JOIN email_response_agg cra ON z.upm_id = cra.upm_id
*/

    LEFT JOIN push_engagement pe ON mh.nuid = pe.nuid
/*
    LEFT JOIN app_metrics_feed amf ON z.upm_id = amf.upm_id
    LEFT JOIN app_metrics_thread amt ON z.upm_id = amt.upm_id
*/
    WHERE (demand < 600 OR demand IS NULL)

    GROUP BY 1,2
    ORDER BY 1 DESC

#### Updated Code

In [ ]:
co
  WITH

    -- use audience file to determine send dates when holdout audience is used
    tc_app_sends AS (
      SELECT upm_id
      , mhub.nuid
      , CASE WHEN final_exposure_or_holdout like ${hivevar:node_tst_str} THEN 'test'
            WHEN final_exposure_or_holdout like ${hivevar:node_ctl_str} THEN 'control'
        ELSE 'na' END AS test_control
      , LEFT(timestamp,10) AS campaign_send_dt
      , CAST(LEFT(timestamp,19) AS TIMESTAMP) AS campaign_send_ts
      FROM ${hivevar:aud_table} aud
      INNER JOIN member.member_hub mhub ON aud.upm_id = mhub.source_id
      ANTI JOIN contam_base c ON aud.upm_id = c.upm_id
      ANTI JOIN gwan13.bot_master_nike_com bot ON aud.upm_id = bot.upm_id
      ANTI JOIN aud_select_workspace.reseller_new slr ON aud.upm_id = slr.upm_user_id
      WHERE CAST(LEFT(timestamp,10) AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND ${hivevar:msmt_end_dt}
      GROUP BY 1,2,3,4,5
  )

    , tc_first_last_send AS (
      SELECT upm_id
      , nuid
      , MIN(campaign_send_dt) as first_send_dt
      , MAX(campaign_send_dt) as last_send_dt
      FROM tc_app_sends
      GROUP BY 1,2
    )

    , test_control_table AS (
        SELECT aud.*
        , fls.first_send_dt
        , fls.last_send_dt
        , LEAD(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt) AS next_send_dt
        , LAG(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt) AS previous_send_dt
        , (int(to_timestamp(campaign_send_dt)) - int(to_timestamp(LEAD(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt))))/86400 AS difference_next_send_days
        , (int(to_timestamp(campaign_send_dt)) - int(to_timestamp(LAG(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt))))/86400 AS difference_previous_send_days
        FROM tc_app_sends aud
        INNER JOIN tc_first_last_send fls ON aud.upm_id = fls.upm_id
    )

    , view_dol AS (
      SELECT dol.upm_id
      , order_nbr
      , order_dt
      , origl_ordered_qty
      , grd_amt_excl_tax_usd
      , line_of_business_desc
      , univ_div_desc
      , first_send_dt
      , last_send_dt
      FROM awight.na_digital_order_line_snapshot dol
      INNER JOIN tc_first_last_send fls ON dol.upm_id = fls.upm_id
      WHERE rec_excl_ind = 0
        AND ttl_demand_ind = 1
        AND univ_cat_desc <> 'Converse'
        AND dol.upm_id IS NOT NULL
        AND order_dt BETWEEN DATE_SUB(first_send_dt,730) AND DATE_ADD(last_send_dt,7)
        AND (rtn_qty = 0 or rtn_qty IS NULL)
    )

    -- AA period purchase. dynamically based on when the member receives their first communication. Only returns demand for members in the
    , aa_purchase AS (
      SELECT dol.upm_id
        ,SUM(grd_amt_excl_tax_usd) AS demand
        ,COUNT(DISTINCT order_nbr) AS orders
        ,SUM(origl_ordered_qty) AS units
      FROM view_dol dol
      WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
      GROUP BY 1
    )

    , demand_bucket AS (
      SELECT tct.upm_id
      , tct.test_control
      , aa.upm_id as upm_id_aa_buyer
      , aa.demand
      , aa.orders
      , aa.units
      , CASE
        WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
        WHEN aa.demand < 50 THEN '000-050'
        WHEN aa.demand < 100 THEN '050-100'
        WHEN aa.demand < 200 THEN '100-200'
        WHEN aa.demand < 500 THEN '200-500'
        WHEN aa.demand < 1000 THEN '500-1000'
        WHEN aa.demand >=1000 THEN '1000+'
        END AS demand_per_buyer_bucket
      , PERCENT_RANK() OVER (
        PARTITION BY test_control
        , CASE
          WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
          WHEN aa.demand < 50 THEN '000-050'
          WHEN aa.demand < 100 THEN '050-100'
          WHEN aa.demand < 200 THEN '100-200'
          WHEN aa.demand < 500 THEN '200-500'
          WHEN aa.demand < 1000 THEN '500-1000'
          WHEN aa.demand >=1000 THEN '1000+'
          END ORDER BY RANDOM(0)
        ) AS percent_rank
      FROM test_control_table tct
      LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
    )

    , balanced_aud_aa as (
      SELECT DISTINCT upm_id
      FROM demand_bucket db
      -- remove outliers
      WHERE (demand < 2400 OR demand IS NULL)
    -- To rebalance, paste text block from template in place of defaults
    AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 0.97632 )
    OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1 )
    OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 0.99509 )
    OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1 )
    OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 0.90741 )
    OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 0.98161 )
    OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 0.99998 )
    OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
    OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 0.98934 )
    OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
    OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 0.98953 )
    OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
    OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
    OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
    ))

    , mhub AS (
      SELECT source_id as upm_id
      , nuid
      , member_id
      , preferred_gender
      FROM MEMBER.MEMBER_HUB
      WHERE ctry_2_cd = 'US'
      AND lower(dpa_status) IN ('activate','reactivate')
    )

    -- calculate a dynamic member activity window for each communication received by a member
    -- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
    -- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
    , member_activities as (
      SELECT mgd.member_id
      ,SUM(logged_in_visits_count) AS visits
      ,SUM(pdp_favorite_count) AS pdp_favorite_count
      ,SUM(add_to_cart_count) AS add_to_cart_count
      ,SUM(physical_activity_count) AS physical_activity
      ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
      ,SUM(eod_chat_count) AS NEOD_Session
      FROM member_agd mgd --aud_select_workspace.member_agg_member_growth_daily mgd
      INNER JOIN mhub mh ON mgd.member_id = mh.member_id
      INNER JOIN test_control_table tct ON mh.upm_id = tct.upm_id
      WHERE mgd.activity_dt >= tct.campaign_send_dt
        AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
        AND mgd.preferred_retail_geo = 'NA'
        AND mgd.experience_name != 'agnostic'
      GROUP BY 1
    )

    -- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
    , ab_purchase as (
      SELECT dol.upm_id as upm_id_ab_buyer
      , order_dt as order_ts
      , order_nbr
      ,SUM(grd_amt_excl_tax_usd) AS demand
      ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
      ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
      ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
      ,COUNT(DISTINCT order_nbr) AS orders
      ,SUM(origl_ordered_qty) AS units
      FROM view_dol dol
      WHERE CAST(order_dt AS DATE) BETWEEN last_send_dt AND DATE_ADD(last_send_dt,7)
      GROUP BY 1,2,3
    )

    -- counting the first order demand after each send
    , reset_per_campaign_sent AS (
    SELECT a.*
      ,CONCAT(a.upm_id, ' ', a.campaign_send_dt ) AS touchpoint_per_member
      FROM test_control_table a -- should it be created based on buyers/
      WHERE upm_id IN (SELECT DISTINCT upm_id_ab_buyer FROM ab_purchase )
      )

    , aud_order_join AS    (
      SELECT a.*
      , b.*
      ,round(((bigint(to_timestamp(a.campaign_send_ts)))-(bigint(to_timestamp(b.order_ts))))/(60),3) AS order_send_diff_min
      ,RANK() OVER(PARTITION BY  a.touchpoint_per_member  ORDER BY (((bigint(to_timestamp(a.campaign_send_ts)))-(bigint(to_timestamp(b.order_ts))))/(60)) DESC, a.campaign_send_dt ) AS nearest_order
      FROM reset_per_campaign_sent a
      LEFT JOIN ab_purchase b
      ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(CAST(a.`campaign_send_dt` AS DATE), CAST(LEAST(7, COALESCE(a.`difference_next_send_days`, 7)) AS INT)))
      ORDER BY a.upm_id, a.campaign_send_dt ASC
    )

    -- Aggregate each distinct send, attributing only the nearest order.
    -- This prevents double counting of orders when a members receives multiple sends within the measurement window.
    , aud_order_agg as (
    SELECT upm_id AS upm_id_ab_buyer
      ,SUM(demand) AS demand
      ,SUM(base_FW_demand) AS base_FW_demand
      ,SUM(base_AP_demand) AS base_AP_demand
      ,SUM(base_EQ_demand) AS base_EQ_demand
      ,SUM(clr_FW_demand) AS clr_FW_demand
      ,SUM(clr_AP_demand) AS clr_AP_demand
      ,SUM(clr_EQ_demand) AS clr_EQ_demand
      ,SUM(launch_FW_demand) AS launch_FW_demand
      ,SUM(launch_AP_demand) AS launch_AP_demand
      ,SUM(launch_EQ_demand) AS launch_EQ_demand
      ,SUM(rest_demand) AS rest_demand
      ,SUM(orders) AS orders
      ,SUM(units) AS units
    FROM aud_order_join a
    WHERE nearest_order = 1
    AND demand IS NOT NULL -- additional filter for extraneous records
    GROUP BY 1
    ORDER BY 1
    )

    , historical_purchase AS (
      SELECT aoa.upm_id_ab_buyer as upm_id
      , COUNT(DISTINCT order_nbr) as histOrders
      FROM aud_order_agg aoa  -- eligible audience
      INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
      WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
      GROUP BY 1
    )

    -- returns all the instances of the campaign codes for each member along with the action types and revenue if applicable
    , email_engage as (
      SELECT DISTINCT fer.upm_id
      , LEFT(event_dttm,10) AS engage_dt
      , action_type
      , revenue_dollar
      , sent_ind
      , open_ind
      , click_ind
      , bounce_ind
      , optout_ind
      , purchase_ind
      FROM awight.na_email_userengagement_snapshot_1 fer
      INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id
      WHERE record_dt BETWEEN ${hivevar:msmt_start_dt} AND DATE_ADD(${hivevar:msmt_end_dt},7)
      AND action_type IN ('open','click','send','bounce','optout')
      AND comm_id IN (${hivevar:email_commID_str})
      AND CAST(LEFT(event_dttm,10) AS DATE) BETWEEN last_send_dt AND DATE_ADD(last_send_dt,7)
      AND fer.upm_id IS NOT NULL
      GROUP BY 1,2,3,4,5,6,7,8,9,10
      )

    -- calculates the the total number of opens, clicks, purchases and last touch attributed demand for each member for the campaign
    -- responses are only considered if they occur between when the message was sent and 7 days later.
    , email_calc AS (
        SELECT fer.upm_id
        , MAX(sent_ind) AS email_sends
        , MAX(open_ind) AS email_opens
        , MAX(click_ind) AS email_clicks
        , MAX(bounce_ind) AS email_bounces
        , MAX(optout_ind) AS email_optouts
        , MAX(purchase_ind) AS email_purch
        , SUM(CASE WHEN action_type = 'ecommerce-purchase' THEN revenue_dollar ELSE 0 END) AS email_demand
        FROM email_engage fer
        INNER JOIN test_control_table tct ON fer.upm_id = tct.upm_id
        GROUP BY 1
    )

    , email_engage_agg as (
      SELECT upm_id
      , SUM(email_sends) AS email_sends
      , SUM(email_opens) AS email_opens
      , SUM(email_clicks) AS email_clicks
      , SUM(email_bounces) AS email_bounces
      , SUM(email_optouts) AS email_optouts
      , SUM(email_purch) AS email_purch
      , SUM(email_demand) AS email_demand
      FROM email_calc
      GROUP BY 1
    )

, push_engagement AS (
    SELECT fpr.upm_id
      , fpr.nuid
      , COUNT(DISTINCT CASE WHEN event_dttm IS NOT NULL AND action_type = 'send' THEN cp_cd END) AS pushes
      , COUNT(DISTINCT CASE WHEN event_dttm IS NOT NULL AND action_type = 'open' OR action_type = 'inferred_open' THEN cp_cd END) AS open_pushes
    FROM awight.na_push_userengagement_snapshot_1 AS fpr
    INNER JOIN test_control_table tc ON tc.nuid = fpr.nuid
    WHERE CAST(event_dttm AS DATE) BETWEEN tc.last_send_dt AND DATE_ADD(tc.last_send_dt,7)
    AND comm_id IN (${hivevar:push_commID_str1}, ${hivevar:push_commID_str2})
    AND action_type IN ('send','open','inferred_open')
    AND retail_geo = 'NA' -- optional
    GROUP BY fpr.upm_id, fpr.nuid
)

/*
    , app_metrics_feed AS (
        SELECT upm_id
      , SUM(tot_editorial_hub_card_shown_count
                    + tot_stream_card_shown_count
                    + tot_editorial_card_shown_count
                    + tot_mhome_product_marketing_card_shown_count
                    + tot_mhome_recommended_product_card_shown_count
            ) as total_card_views
      , 	SUM(tot_editorial_hub_card_clicked_count
                    + tot_stream_card_clicked_count
                    + tot_editorial_card_clicked_count
                    + tot_mhome_product_marketing_card_clicked_count
                    + tot_mhome_recommended_product_card_clicked_count
            ) as total_card_taps
        FROM content.agg_user_session_content_activity_daily
        WHERE thread_id IN (${hivevar:feedID_str})
          AND CAST(session_start_date AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND DATE_ADD(${hivevar:msmt_end_dt},7)
          AND app_version IS NOT NULL
          AND card_key != 'UNKNOWN'
        GROUP BY 1
    )

    , app_metrics_thread AS (
      SELECT upm_id
      ,SUM(tot_thread_clicks) as thread_taps
      ,SUM(tot_thread_viewed_count) as thread_views
      ,SUM(tot_thread_comment_added_count) as thread_comment_add
      FROM content.agg_user_session_content_activity_daily
      WHERE thread_id IN (${hivevar:threadID_str})
        AND CAST(session_start_date AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND DATE_ADD(${hivevar:msmt_end_dt},7)
        AND app_version IS NOT NULL
        AND card_key != 'UNKNOWN'
      GROUP BY 1
    )
*/

, combined_engagement AS (
  SELECT upm_id
      , SUM(email_sends) AS email_sends
      , SUM(email_opens) AS email_opens
      , SUM(email_clicks) AS email_clicks
      , SUM(email_purch) AS email_purch
      , SUM(email_demand) AS email_demand
      , SUM(pushes) AS push_sent
      , SUM(open_pushes) AS push_opens
  FROM (
    SELECT upm_id, email_sends, email_opens, email_clicks, email_purch, email_demand, 0 AS pushes, 0 AS open_pushes
    FROM email_engage_agg
    UNION ALL
    SELECT pe.upm_id, 0 AS email_sends, 0 AS email_opens, 0 AS email_clicks, 0 AS email_purch, 0 AS email_demand, pushes, open_pushes
    FROM push_engagement pe
  ) combined
  GROUP BY upm_id
)

---------------------
-- MAIN QUERY
----------------------
SELECT  z.upm_id
      , z.test_control

      -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
      , SUM(abp.demand) AS demand

      -- secondary metrics ON order double click
      , SUM(abp.orders) AS orders
      , SUM(abp.units) AS units

      -- secondary metrics ON emails
      , SUM(combined_engagement.email_sends) AS email_sends
      , SUM(combined_engagement.email_opens) AS email_opens
      , SUM(combined_engagement.email_clicks) AS email_clicks
      , SUM(combined_engagement.email_purch) AS email_purch
      , SUM(combined_engagement.email_demand) AS email_demand

      -- secondary metrics ON app push/engagement
      , SUM(combined_engagement.push_sent) AS push_sends
      , SUM(combined_engagement.push_opens) AS push_opens
      , (CASE WHEN SUM(combined_engagement.push_sent) = 0 THEN 0 ELSE SUM(combined_engagement.push_opens) / SUM(combined_engagement.push_sent) END) AS push_open_rate

      -- secondary metrics ON site/app engagement
      , SUM(a.visits) AS site_app_visits
      , SUM(a.PDP_FAVORITE_COUNT) AS PDP_FAVORITE_COUNT
      , SUM(a.ADD_TO_CART_COUNT) AS ADD_TO_CART_COUNT
      , SUM(a.physical_activity) AS physical_activity

      -- secondary metrics ON margin
      , SUM(abp.base_FW_demand) AS base_FW_demand
      , SUM(abp.base_AP_demand) AS base_AP_demand
      , SUM(abp.base_EQ_demand) AS base_EQ_demand
      , SUM(abp.clr_FW_demand) AS clr_FW_demand
      , SUM(abp.clr_AP_demand) AS clr_AP_demand
      , SUM(abp.clr_EQ_demand) AS clr_EQ_demand
      , SUM(abp.launch_FW_demand) AS launch_FW_demand
      , SUM(abp.launch_AP_demand) AS launch_AP_demand
      , SUM(abp.launch_EQ_demand) AS launch_EQ_demand
      , SUM(abp.rest_demand) AS rest_demand

      -- secondary metrics ON order history
      , SUM(hp.histOrders) AS histOrders
      , SUM(CASE WHEN hp.histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
      , SUM(CASE WHEN hp.histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder

FROM test_control_table z
INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance
INNER JOIN mhub mh ON z.upm_id = mh.upm_id
LEFT JOIN member_activities a ON mh.member_id = a.member_id
LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id
LEFT JOIN combined_engagement ON z.upm_id = combined_engagement.upm_id

WHERE (abp.demand < 600 OR abp.demand IS NULL)
AND test_control != 'na'

GROUP BY z.upm_id, z.test_control
ORDER BY z.upm_id DESC;



In [ ]:
Whatt

SELECT upm_id
    ,  total_demand
FROM(
      SELECT upm_id
      , SUM (demand) AS total_demand
      FROM ${hivevar:user_db}.${hivevar:to_tbl}_AB_full
      GROUP BY upm_id
) as poo
HAVING COUNT(upm_id) > 1

In [ ]:
 WITH
-- Including the relevant sub-query to identify duplicate order_nbr
, view_dol AS (
  SELECT dol.upm_id
  , order_nbr
  , order_dt
  , origl_ordered_qty
  , grd_amt_excl_tax_usd
  , line_of_business_desc
  , univ_div_desc
  , first_send_dt
  , last_send_dt
  FROM awight.na_digital_order_line_snapshot dol
  INNER JOIN tc_first_last_send fls ON dol.upm_id = fls.upm_id
  WHERE rec_excl_ind = 0
    AND ttl_demand_ind = 1
    AND univ_cat_desc <> 'Converse'
    AND dol.upm_id IS NOT NULL
    AND order_dt BETWEEN DATE_SUB(first_send_dt,730) AND DATE_ADD(last_send_dt,7)
    AND (rtn_qty = 0 or rtn_qty IS NULL)
)

-- Identifies duplicate order numbers

  SELECT order_nbr
  , COUNT(*) AS duplicate_count
  FROM view_dol
  GROUP BY order_nbr
  HAVING COUNT(*) > 1


-- get details of duplicate orders
SELECT vd.upm_id
  , vd.order_nbr
  , vd.order_dt
  , vd.grd_amt_excl_tax_usd
  , vd.line_of_business_desc
  , vd.univ_div_desc
FROM view_dol vd
INNER JOIN duplicate_orders do ON vd.order_nbr = do.order_nbr
ORDER BY vd.order_nbr;

-- Alternatively, if you only need the counts:
SELECT order_nbr, duplicate_count
FROM duplicate_orders
ORDER BY duplicate_count DESC;

#### Adding Holdouts back in

In [ ]:
-----------------------------------------------------------------------------------------------------------------------------------
-- AB Period
-----------------------------------------------------------------------------------------------------------------------------------
WITH

-- use audience file to determine send dates when holdout audience is used
tc_app_sends AS (
    SELECT upm_id
    , CASE WHEN final_exposure_or_holdout like ${hivevar:node_tst_str} THEN 'test'
        WHEN final_exposure_or_holdout like ${hivevar:node_ctl_str_2} THEN 'holdout'
        WHEN final_exposure_or_holdout like ${hivevar:node_ctl_str} THEN 'control'
      ELSE 'na' END AS test_control
    , LEFT(timestamp,10) AS campaign_send_dt
    , CAST(LEFT(timestamp,19) AS TIMESTAMP) AS campaign_send_ts
    FROM ${hivevar:aud_table} aud
    ANTI JOIN contam_base c ON aud.upm_id = c.upm_id
    ANTI JOIN gwan13.bot_master_nike_com bot ON aud.upm_id = bot.upm_id
    ANTI JOIN aud_select_workspace.reseller_new slr ON aud.upm_id = slr.upm_user_id
    WHERE CAST(LEFT(timestamp,10) AS DATE) BETWEEN ${hivevar:msmt_start_dt} AND ${hivevar:msmt_end_dt}
)

, tc_first_last_send AS (
  SELECT upm_id
  , MIN(campaign_send_dt) as first_send_dt
  , MAX(campaign_send_dt) as last_send_dt
  FROM tc_app_sends
  GROUP BY 1
)

, test_control_table AS (
    SELECT aud.*
    , fls.first_send_dt
    , fls.last_send_dt
    , LEAD(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt) AS next_send_dt
    , LAG(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt) AS previous_send_dt
    , (int(to_timestamp(campaign_send_dt)) - int(to_timestamp(LEAD(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt))))/86400 AS difference_next_send_days
    , (int(to_timestamp(campaign_send_dt)) - int(to_timestamp(LAG(campaign_send_dt) OVER (PARTITION BY aud.upm_id ORDER BY campaign_send_dt))))/86400 AS difference_previous_send_days
    FROM tc_app_sends aud
    INNER JOIN tc_first_last_send fls ON aud.upm_id = fls.upm_id
)

, view_dol AS (
  SELECT dol.upm_id
  , order_nbr
  , order_dt
  , origl_ordered_qty
  , grd_amt_excl_tax_usd
  , line_of_business_desc
  , univ_div_desc
  , first_send_dt
  , last_send_dt
  FROM awight.na_digital_order_line_snapshot dol
  INNER JOIN tc_first_last_send fls ON dol.upm_id = fls.upm_id
  WHERE rec_excl_ind = 0
    AND ttl_demand_ind = 1
    AND univ_cat_desc <> 'Converse'
    AND dol.upm_id IS NOT NULL
    AND order_dt BETWEEN DATE_SUB(first_send_dt,730) AND DATE_ADD(last_send_dt,7)
    AND (rtn_qty = 0 or rtn_qty IS NULL)
)

-- AA period purchase. dynamically based on when the member receives their first communication. Only returns demand for members in the
, aa_purchase AS (
  SELECT dol.upm_id
    ,SUM(grd_amt_excl_tax_usd) AS demand
    ,COUNT(DISTINCT order_nbr) AS orders
    ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,31) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

, demand_bucket AS (
  SELECT tct.upm_id
  , tct.test_control
  , aa.upm_id as upm_id_aa_buyer
  , aa.demand
  , aa.orders
  , aa.units
  , CASE
    WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
    WHEN aa.demand < 50 THEN '000-050'
    WHEN aa.demand < 100 THEN '050-100'
    WHEN aa.demand < 200 THEN '100-200'
    WHEN aa.demand < 500 THEN '200-500'
    WHEN aa.demand < 1000 THEN '500-1000'
    WHEN aa.demand >=1000 THEN '1000+'
    END AS demand_per_buyer_bucket
  , PERCENT_RANK() OVER (
    PARTITION BY test_control
    , CASE
      WHEN aa.demand = 0 OR aa.demand IS NULL THEN 'non-buyer'
      WHEN aa.demand < 50 THEN '000-050'
      WHEN aa.demand < 100 THEN '050-100'
      WHEN aa.demand < 200 THEN '100-200'
      WHEN aa.demand < 500 THEN '200-500'
      WHEN aa.demand < 1000 THEN '500-1000'
      WHEN aa.demand >=1000 THEN '1000+'
      END ORDER BY RANDOM(0)
    ) AS percent_rank
  FROM test_control_table tct
  LEFT JOIN aa_purchase aa ON tct.upm_id = aa.upm_id
)

, balanced_aud_aa as (
  SELECT DISTINCT upm_id
  FROM demand_bucket db
  -- remove outliers
  WHERE (demand < 2400 OR demand IS NULL)
-- To rebalance, paste text block from template in place of defaults
AND ((test_control = 'control' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 0.97632 )
OR (test_control = 'control' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1 )
OR (test_control = 'control' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 0.99509 )
OR (test_control = 'control' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1 )
OR (test_control = 'control' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 0.90741 )
OR (test_control = 'control' and (demand > 1000) and PERCENT_RANK <= 0.98161 )
OR (test_control = 'control' and (demand = 0 or demand is null) and PERCENT_RANK <= 0.99998 )
OR (test_control = 'test' and (demand >0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 0.98934 )
OR (test_control = 'test' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 0.98953 )
OR (test_control = 'test' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'test' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
OR (test_control = 'holdout' and (demand > 0 and demand <= 50) and PERCENT_RANK <= 1)
OR (test_control = 'holdout' and (demand > 50 and demand <= 100) and PERCENT_RANK <= 1 )
OR (test_control = 'holdout' and (demand > 100 and demand <= 200) and PERCENT_RANK <= 1)
OR (test_control = 'holdout' and (demand > 200 and demand <= 500) and PERCENT_RANK <= 1)
OR (test_control = 'holdout' and (demand > 500 and demand <= 1000) and PERCENT_RANK <= 1)
OR (test_control = 'holdout' and (demand > 1000) and PERCENT_RANK <= 1)
OR (test_control = 'holdout' and (demand = 0 or demand is null) and PERCENT_RANK <= 1)
))

, mhub as (
 SELECT source_id as upm_id
 , nuid
 , member_id
 , preferred_gender
 FROM MEMBER.MEMBER_HUB
 WHERE ctry_2_cd = 'US'
 AND lower(dpa_status) IN ('activate','reactivate')
)

-- calculate a dynamic member activity window for each communication received by a member
-- activity is calculated on a 7 day window, unless the member receives a communication within that timeframe
-- When multiple messages are received, activity is only calculated up to the next message_send_dt to avoid double counting
, member_activities as (
  SELECT mgd.member_id
  ,SUM(logged_in_visits_count) AS visits
  ,SUM(pdp_favorite_count) AS pdp_favorite_count
  ,SUM(add_to_cart_count) AS add_to_cart_count
  ,SUM(physical_activity_count) AS physical_activity
  ,MAX(CASE WHEN new_user_experience_flag = 'Y' THEN 1 ELSE 0 END) AS new_user_flag
  ,SUM(eod_chat_count) AS NEOD_Session
  FROM member_agd mgd --aud_select_workspace.member_agg_member_growth_daily mgd
  INNER JOIN mhub mh ON mgd.member_id = mh.member_id
  INNER JOIN test_control_table tct ON mh.upm_id = tct.upm_id
  WHERE mgd.activity_dt >= tct.campaign_send_dt
    AND mgd.activity_dt < DATE_ADD(campaign_send_dt, INT(LEAST(7,COALESCE(difference_next_send_days,7))))
    AND mgd.preferred_retail_geo = 'NA'
    AND mgd.experience_name != 'agnostic'
  GROUP BY 1
)

-- for each user, aggregate each order placed within our measurement window. create labels to categorize the type of demand
, ab_purchase as (
  SELECT dol.upm_id as upm_id_ab_buyer
  , order_dt as order_ts
  , order_nbr
  ,SUM(grd_amt_excl_tax_usd) AS demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Base Inline' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS base_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Clearance' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS clr_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Footwear' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_FW_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND univ_div_desc = 'Apparel' THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_AP_demand
  ,SUM(CASE WHEN line_of_business_desc = 'Launch' AND (univ_div_desc = 'Equipment' OR univ_div_desc IS NULL) THEN grd_amt_excl_tax_usd ELSE 0 END) AS launch_EQ_demand
  ,SUM(CASE WHEN line_of_business_desc NOT IN ('Base Inline', 'Clearance', 'Launch') THEN grd_amt_excl_tax_usd ELSE 0 END) AS rest_demand
  ,COUNT(DISTINCT order_nbr) AS orders
  ,SUM(origl_ordered_qty) AS units
  FROM view_dol dol
  WHERE CAST(order_dt AS DATE) BETWEEN last_send_dt AND DATE_ADD(last_send_dt,7)
  GROUP BY 1,2,3
)

-- counting the first order demand after each send
, reset_per_campaign_sent AS (
 SELECT a.*
  ,CONCAT(a.upm_id, ' ', a.campaign_send_dt ) AS touchpoint_per_member
  FROM test_control_table a -- should it be created based on buyers/
  WHERE upm_id IN (SELECT DISTINCT upm_id_ab_buyer FROM ab_purchase )
  )


, aud_order_join AS    (
  SELECT a.*
  , b.*
  ,round(((bigint(to_timestamp(a.campaign_send_ts)))-(bigint(to_timestamp(b.order_ts))))/(60),3) AS order_send_diff_min
  ,RANK() OVER(PARTITION BY  a.touchpoint_per_member  ORDER BY (((bigint(to_timestamp(a.campaign_send_ts)))-(bigint(to_timestamp(b.order_ts))))/(60)) DESC, a.campaign_send_dt ) AS nearest_order
  FROM reset_per_campaign_sent a
  LEFT JOIN ab_purchase b
  ON (a.upm_id = b.upm_id_ab_buyer AND b.order_ts >= a.campaign_send_ts AND b.order_ts <= DATE_ADD(CAST(a.`campaign_send_dt` AS DATE), CAST(LEAST(7, COALESCE(a.`difference_next_send_days`, 7)) AS INT)))
  ORDER BY a.upm_id, a.campaign_send_dt ASC
)


-- Aggregate each distinct send, attributing only the nearest order.
-- This prevent double counting of orders when a members receives multiple sends within the measurement window.
, aud_order_agg as (
SELECT upm_id AS upm_id_ab_buyer
  ,SUM(demand) AS demand
  ,SUM(base_FW_demand) AS base_FW_demand
  ,SUM(base_AP_demand) AS base_AP_demand
  ,SUM(base_EQ_demand) AS base_EQ_demand
  ,SUM(clr_FW_demand) AS clr_FW_demand
  ,SUM(clr_AP_demand) AS clr_AP_demand
  ,SUM(clr_EQ_demand) AS clr_EQ_demand
  ,SUM(launch_FW_demand) AS launch_FW_demand
  ,SUM(launch_AP_demand) AS launch_AP_demand
  ,SUM(launch_EQ_demand) AS launch_EQ_demand
  ,SUM(rest_demand) AS rest_demand
  ,SUM(orders) AS orders
  ,SUM(units) AS units
FROM aud_order_join a
WHERE nearest_order = 1
AND demand IS NOT NULL -- additional filter for extraneous records
GROUP BY 1
ORDER BY 1
)

, historical_purchase AS (
  SELECT aoa.upm_id_ab_buyer as upm_id
  , COUNT(DISTINCT order_nbr) as histOrders
  FROM aud_order_agg aoa  -- eligible audience
  INNER JOIN view_dol dol on aoa.upm_id_ab_buyer = dol.upm_id
  WHERE CAST(order_dt AS DATE) BETWEEN DATE_SUB(first_send_dt,730) AND DATE_SUB(first_send_dt,1)
  GROUP BY 1
)

---------------------
-- MAIN QUERY
----------------------
SELECT z.test_control
  -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
  ,COUNT(DISTINCT z.upm_id) AS total_members
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) AS buying_members
  ,COUNT(DISTINCT abp.upm_id_ab_buyer) / COUNT(DISTINCT z.upm_id) AS conversion_rate
  ,AVG(demand) AS avg_demand
  ,stddev(demand) AS std_demand
  -- secondary metrics ON order double click
  ,SUM(demand) / SUM(orders) AS AOV
  ,SUM(demand) / SUM(units) AS AUR
  ,SUM(units) / SUM(orders) AS UPT


  -- secondary metrics ON site/app engagement

  ,AVG(visits) AS avg_site_app_visits
  ,AVG(PDP_FAVORITE_COUNT) AS avg_PDP_FAVORITE_COUNT
  ,AVG(ADD_TO_CART_COUNT) AS avg_ADD_TO_CART_COUNT
  ,AVG(physical_activity) AS avg_physical_activity

  -- secondary metrics ON margin
  ,(AVG(base_FW_demand) * 0.38 + AVG(base_AP_demand) * 0.28 + AVG(base_EQ_demand) * 0.32
    + AVG(clr_FW_demand) * 0.23 + AVG(base_AP_demand) * 0.17 + AVG(clr_EQ_demand) * 0.12
    + AVG(launch_FW_demand) * 0.25 + AVG(launch_AP_demand) * 0.08 + AVG(launch_EQ_demand) * 0.19
    + AVG(rest_demand) * 0.3
  ) AS avg_margin
  ,AVG(base_FW_demand) AS base_FW_demand
  ,AVG(base_AP_demand) AS base_AP_demand
  ,AVG(base_EQ_demand) AS base_EQ_demand
  ,AVG(clr_FW_demand) AS clr_FW_demand
  ,AVG(clr_AP_demand) AS clr_AP_demand
  ,AVG(clr_EQ_demand) AS clr_EQ_demand
  ,AVG(launch_FW_demand) AS launch_FW_demand
  ,AVG(launch_AP_demand) AS launch_AP_demand
  ,AVG(launch_EQ_demand) AS launch_EQ_demand
  ,AVG(rest_demand) AS rest_demand
    -- secondary metrics ON order history
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_1_priorOrder
  ,SUM(CASE WHEN histOrders = 1 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS DPB_1_prior
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END)/COUNT(DISTINCT z.upm_id) AS pctBuyers_2_priorOrder
  ,SUM(CASE WHEN histOrders = 2 THEN demand ELSE 0 END)/SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS DPB_2_prior

FROM test_control_table z
INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

INNER JOIN mhub mh ON z.upm_id = mh.upm_id
LEFT JOIN member_activities a ON mh.member_id = a.member_id

LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

--LEFT JOIN platform_usage plt ON z.upm_id = plt.upm_id

WHERE (demand < 600 OR demand IS NULL)
AND test_control != 'na'

GROUP BY 1
ORDER BY 1 DESC;


## Pairwise Chi-Square for Conversion

In [ ]:
Here is the structure of my data, where the test z.test_control groupings are: test, control, holdout

SELECT z.upm_id
          , z.test_control

          -- primary metrics ON incremental revenue: conversion rate, average demand per purchaser
          , SUM(demand) AS demand

          -- secondary metrics ON order double click
          , SUM(orders) AS orders
          , SUM(units) AS units

          -- secondary metrics ON site/app engagement
          , SUM(visits) AS site_app_visits
          , SUM(PDP_FAVORITE_COUNT) AS PDP_FAVORITE_COUNT
          , SUM(ADD_TO_CART_COUNT) AS ADD_TO_CART_COUNT
          , SUM(physical_activity) AS physical_activity

          -- secondary metrics ON margin
          , SUM(base_FW_demand) AS base_FW_demand
          , SUM(base_AP_demand) AS base_AP_demand
          , SUM(base_EQ_demand) AS base_EQ_demand
          , SUM(clr_FW_demand) AS clr_FW_demand
          , SUM(clr_AP_demand) AS clr_AP_demand
          , SUM(clr_EQ_demand) AS clr_EQ_demand
          , SUM(launch_FW_demand) AS launch_FW_demand
          , SUM(launch_AP_demand) AS launch_AP_demand
          , SUM(launch_EQ_demand) AS launch_EQ_demand
          , SUM(rest_demand) AS rest_demand

          -- secondary metrics ON order history
          ,SUM(histOrders) AS histOrders
          ,SUM(CASE WHEN histOrders = 1 THEN 1 ELSE 0 END) AS 1_priorOrder
          ,SUM(CASE WHEN histOrders = 2 THEN 1 ELSE 0 END) AS 2_priorOrder

        FROM test_control_table z
        INNER JOIN balanced_aud_aa b ON z.upm_id = b.upm_id  --inner join our balanced audience to remove members that were distorting the AA period balance

        INNER JOIN mhub mh ON z.upm_id = mh.upm_id
        LEFT JOIN member_activities a ON mh.member_id = a.member_id

        LEFT JOIN aud_order_agg abp ON z.upm_id = abp.upm_id_ab_buyer
        LEFT JOIN historical_purchase hp ON z.upm_id = hp.upm_id

        WHERE (demand < 600 OR demand IS NULL)

        GROUP BY 1,2
        ORDER BY 1 DESC


### AB Scorecard

#### For Two Variants

In [ ]:
%python
import numpy as np
import pandas as pd
from scipy import stats

# Assuming 'df' is your DataFrame with necessary columns

# Step 1: Add 'buyer' and 'prior_order' indicator columns
df_ab['buyer'] = (df_ab['orders'] > 0).astype(int)
df_ab['demand_1_prior'] = np.where(df_ab['1_priorOrder'] == 1, df_ab['demand'], np.nan)
df_ab['demand_2_prior'] = np.where(df_ab['2_priorOrder'] == 1, df_ab['demand'], np.nan)
df_ab['member_1_prior'] = np.where(df_ab['1_priorOrder'] == 1, 1, 0)
df_ab['member_2_prior'] = np.where(df_ab['2_priorOrder'] == 1, 1, 0)

# Step 2: Perform temporary calculations for DPB_1_prior and DPB_2_prior
temp_calculations = df_ab.groupby('test_control').agg({
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
}).rename(columns={
    'demand_1_prior': 'Total Demand 1 Prior',
    'demand_2_prior': 'Total Demand 2 Prior',
    'member_1_prior': 'Members 1 Prior',
    'member_2_prior': 'Members 2 Prior',
})

# Step 3: Calculate DPB_1_prior and DPB_2_prior using temporary calculations
dpb_1_prior = temp_calculations['Total Demand 1 Prior'] / temp_calculations['Members 1 Prior']
dpb_2_prior = temp_calculations['Total Demand 2 Prior'] / temp_calculations['Members 2 Prior']

# Step 4: Aggregate metrics
aggregated_metrics = df_ab.groupby('test_control').agg({
    'upm_id': 'nunique',
    'buyer': 'sum',
    'orders': 'sum',
    'units': 'sum',
    'demand': ['sum', 'mean'],
    '1_priorOrder': 'sum',
    '2_priorOrder': 'sum',
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
    # 'email_opens': 'sum',
    # 'email_sends': 'sum',
    # 'email_clicks': 'sum',
    # 'push_sends': 'sum',
    # 'push_opens': 'sum',
    'site_app_visits': 'mean',
    'PDP_FAVORITE_COUNT': 'mean',
    'ADD_TO_CART_COUNT': 'mean',
    'physical_activity': 'mean',
    'base_FW_demand': 'mean',
    'clr_FW_demand': 'mean',
    'launch_FW_demand': 'mean',
    'base_AP_demand': 'mean',
    'clr_AP_demand': 'mean',
    'launch_AP_demand': 'mean',
    'base_EQ_demand': 'mean',
    'clr_EQ_demand': 'mean',
    'launch_EQ_demand': 'mean',
})

# Step 5: Correct the MultiIndex handling by flattening and renaming
aggregated_metrics.columns = ['_'.join(col).strip() for col in aggregated_metrics.columns.values]
aggregated_metrics = aggregated_metrics.rename(columns={
    'upm_id_nunique': 'Total Members',
    'buyer_sum': 'Buying Members',
    'demand_mean': 'Demand per Buyer',
    'demand_sum': 'Total Demand',
    'units_sum': 'Total Units',
    'orders_sum': 'Total Orders',
    # Secondary KPIs - Repeat buyers
    '1_priorOrder_sum': 'Members with 1 previous order',
    '2_priorOrder_sum':'Members with 2 previous orders',
    # Secondary KPIs - Site/app Engagement metrics
    'site_app_visits_mean':'Visits per known member',
    'PDP_FAVORITE_COUNT_mean':'PDP favorite per known member',
    'ADD_TO_CART_COUNT_mean': 'Add to cart per known member',
    'physical_activity_mean': 'Workouts per known member',
})

# Step 6: Calculate DPB_1_prior and DPB_2_prior based on the temporary calculations and integrate directly into the DataFrame
# aggregated_metrics['DPB_1_prior'] = aggregated_metrics['demand_1_prior'] / aggregated_metrics['member_1_prior']
# aggregated_metrics['DPB_2_prior'] = aggregated_metrics['demand_2_prior'] / aggregated_metrics['member_2_prior']

# Step 7: Calculate additional metrics

aggregated_metrics['Conversion Rate'] = aggregated_metrics['Buying Members'] / aggregated_metrics['Total Members']
aggregated_metrics['Demand per Send'] = aggregated_metrics['Demand per Buyer'] * aggregated_metrics['Conversion Rate']
# Secondary KPIs - order deep dive
aggregated_metrics['AOV'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Orders']
aggregated_metrics['AUR'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Units']
aggregated_metrics['UPT'] = aggregated_metrics['Total Units'] / aggregated_metrics['Total Orders']
aggregated_metrics['# Txn per Buyer'] = aggregated_metrics['Demand per Buyer']/aggregated_metrics['AOV']
# Secondary KPIs - Repeat buyers
aggregated_metrics['Conversion rate (1x buyers/email receivers)'] = aggregated_metrics['Members with 1 previous order'] / aggregated_metrics['Total Members']
aggregated_metrics['Conversion rate (2x buyers/email receivers)'] = aggregated_metrics['Members with 2 previous orders'] / aggregated_metrics['Total Members']
aggregated_metrics['Demand per Buyer (1 previous order)'] = aggregated_metrics['demand_1_prior_sum'] / aggregated_metrics.get('member_1_prior_sum', 1)
aggregated_metrics['Demand per Buyer (2 previous order)'] = aggregated_metrics['demand_2_prior_sum'] / aggregated_metrics.get('member_2_prior_sum', 1)

# Secondary KPIs - Email/Push Engagement metrics (all email/push received during measurement windows)
# aggregated_metrics['Email Open Rate (against sends)'] = aggregated_metrics['email_opens_sum'] / aggregated_metrics['email_sends_sum']
# aggregated_metrics['Email Click Rate (against sends)'] = aggregated_metrics['email_clicks_sum'] / aggregated_metrics['email_sends_sum']
# aggregated_metrics['Push Open Rate (against sends)'] = aggregated_metrics['push_opens_sum'] / aggregated_metrics['push_sends_sum']

# Secondary KPIs - product mix
aggregated_metrics['Footwear Demand per Member'] = ( aggregated_metrics['base_FW_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Apparel Demand per Member'] = ( aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Equipment Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['launch_EQ_demand_mean'] ) * aggregated_metrics['Conversion Rate']
# Secondary KPIs - LOB
aggregated_metrics['Regular Product Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['base_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Clearance Demand per Member'] = ( aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Launch Demand per Member'] = ( aggregated_metrics['launch_EQ_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']


# Step 7: Remove temporary calculation columns as they're not needed in final DataFrame
aggregated_metrics.drop(columns=['demand_1_prior_sum', 'demand_2_prior_sum', 'member_1_prior_sum', 'member_2_prior_sum',  'base_FW_demand_mean',
  # 'email_clicks_sum','email_opens_sum','email_sends_sum',
 'clr_FW_demand_mean', 'launch_FW_demand_mean', 'base_AP_demand_mean', 'clr_AP_demand_mean', 'launch_AP_demand_mean', 'base_EQ_demand_mean', 'clr_EQ_demand_mean','launch_EQ_demand_mean'], inplace=True)

# Step 8: Perform t-test for 'Conversion Rate' and 'Demand per Buyer'

# Separate the test and control groups to perform the t-tests correctly.
test_group_ab = df_ab[df_ab['test_control'] == 'test']
control_group_ab = df_ab[df_ab['test_control'] == 'control']

# Conversion Rate Chi-square

# Create a contingency table
conv_contingency_table = pd.crosstab(df_ab['test_control'], df_ab['buyer']) # Count num of buyers and non-buyers in each group

# Perform the Chi-square test
chi2_stat_conversion, p_value_conversion, dof_conversion, expected_cnt_conversion = chi2_contingency(conv_contingency_table)

# Demand per Buyer Wilcox (Mann-Whitney U) test
mw_u_statistic_ab, mw_p_value_ab = stats.mannwhitneyu(test_group_ab['demand'].dropna(), control_group_ab['demand'].dropna(), alternative='two-sided')


# Demand per Buyer T-Test
"""
# 'nan' values imputed with 0
t_stat_demand, p_value_demand = stats.ttest_ind(test_group_ab['demand'].fillna(0), control_group_ab['demand'].fillna(0), equal_var=False)
"""
# 'nan' values omitted
t_stat_demand, p_value_demand = stats.ttest_ind(test_group_ab['demand'], control_group_ab['demand'], equal_var=False, nan_policy='omit')
# Note, results for demand ttest are the same with 'nan' values omitted or imputed to 0

# Step 9: Transpose the table
pd.options.display.float_format = '{:.8f}'.format
aggregated_metrics = aggregated_metrics.T

# Step 10: Calculate lift for all metrics except 'Conversion Rate' and 'Demand per Buyer'
scorecard = pd.DataFrame(index=aggregated_metrics.index, columns=['Metric', 'Test', 'Control', 'Lift','P-Value','Stat'])

# Populate Test and Control values from aggregated_metrics
scorecard['Test'] = aggregated_metrics['test']
scorecard['Control'] = aggregated_metrics['control']

# Calculate Lift
for metric in aggregated_metrics.index:
    if metric not in ['Conversion Rate', 'Demand per Buyer']: # excluding Conversion_Rate and Demand_per_Buyer
        test_val = scorecard.at[metric, 'Test']
        control_val = scorecard.at[metric, 'Control']
        lift = ((test_val - control_val) / control_val) * 100 if control_val != 0 else np.nan
        scorecard.at[metric, 'Lift'] = lift

# Step 11: Add t_stat and p_value for 'Conversion Rate' and 'Demand per Buyer' to dataframe
scorecard.at['Conversion Rate', 'Stat'] = chi2_stat_conversion
scorecard.at['Conversion Rate', 'P-Value'] = p_value_conversion
scorecard.at['Demand per Buyer', 'Stat'] = mw_u_statistic_ab
scorecard.at['Demand per Buyer', 'P-Value'] = mw_p_value_ab

# Step 12: Ensure 'Metric' column is correctly populated
# A byproduct of transposing the dataframe with some calculations only being applied selectively
scorecard['Metric'] = scorecard.index
scorecard.reset_index(drop=False, inplace=True)
scorecard.drop(columns=['index'], inplace=True)

# Step 13: Exclude 'Conversion Rate' and 'Demand per Buyer' from lift calculation
scorecard.loc[scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Lift'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Stat'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'P-Value'] = ''

# Step 14: Display the scorecard
scorecard


#### For Three Variants

In [ ]:
%python
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import chi2_contingency
from statsmodels.stats.multitest import multipletests


# Step 1: Add 'buyer' and 'prior_order' indicator columns
df_ab['buyer'] = (df_ab['orders'] > 0).astype(int)
df_ab['demand_1_prior'] = np.where(df_ab['1_priorOrder'] == 1, df_ab['demand'], np.nan)
df_ab['demand_2_prior'] = np.where(df_ab['2_priorOrder'] == 1, df_ab['demand'], np.nan)
df_ab['member_1_prior'] = np.where(df_ab['1_priorOrder'] == 1, 1, 0)
df_ab['member_2_prior'] = np.where(df_ab['2_priorOrder'] == 1, 1, 0)

# Step 2: Perform temporary calculations for DPB_1_prior and DPB_2_prior
temp_calculations = df_ab.groupby('test_control').agg({
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
}).rename(columns={
    'demand_1_prior': 'Total Demand 1 Prior',
    'demand_2_prior': 'Total Demand 2 Prior',
    'member_1_prior': 'Members 1 Prior',
    'member_2_prior': 'Members 2 Prior',
})

# Step 3: Calculate DPB_1_prior and DPB_2_prior using temporary calculations
dpb_1_prior = temp_calculations['Total Demand 1 Prior'] / temp_calculations['Members 1 Prior']
dpb_2_prior = temp_calculations['Total Demand 2 Prior'] / temp_calculations['Members 2 Prior']

# Step 4: Aggregate metrics
aggregated_metrics = df_ab.groupby('test_control').agg({
    'upm_id': 'nunique',
    'buyer': 'sum',
    'orders': 'sum',
    'units': 'sum',
    'demand': ['sum', 'mean'],
    '1_priorOrder': 'sum',
    '2_priorOrder': 'sum',
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
    # 'email_opens': 'sum',
    # 'email_sends': 'sum',
    # 'email_clicks': 'sum',
    # 'push_sends': 'sum',
    # 'push_opens': 'sum',
    'site_app_visits': 'mean',
    'PDP_FAVORITE_COUNT': 'mean',
    'ADD_TO_CART_COUNT': 'mean',
    'physical_activity': 'mean',
    'base_FW_demand': 'mean',
    'clr_FW_demand': 'mean',
    'launch_FW_demand': 'mean',
    'base_AP_demand': 'mean',
    'clr_AP_demand': 'mean',
    'launch_AP_demand': 'mean',
    'base_EQ_demand': 'mean',
    'clr_EQ_demand': 'mean',
    'launch_EQ_demand': 'mean',
})

# Step 5: Correct the MultiIndex handling by flattening and renaming
aggregated_metrics.columns = ['_'.join(col).strip() for col in aggregated_metrics.columns.values]
aggregated_metrics = aggregated_metrics.rename(columns={
    'upm_id_nunique': 'Total Members',
    'buyer_sum': 'Buying Members',
    'demand_mean': 'Demand per Buyer',
    'demand_sum': 'Total Demand',
    'units_sum': 'Total Units',
    'orders_sum': 'Total Orders',
    # Secondary KPIs - Repeat buyers
    '1_priorOrder_sum': 'Members with 1 previous order',
    '2_priorOrder_sum':'Members with 2 previous orders',
    # Secondary KPIs - Site/app Engagement metrics
    'site_app_visits_mean':'Visits per known member',
    'PDP_FAVORITE_COUNT_mean':'PDP favorite per known member',
    'ADD_TO_CART_COUNT_mean': 'Add to cart per known member',
    'physical_activity_mean': 'Workouts per known member',
})

# Step 6: Calculate DPB_1_prior and DPB_2_prior based on the temporary calculations and integrate directly into the DataFrame
# aggregated_metrics['DPB_1_prior'] = aggregated_metrics['demand_1_prior'] / aggregated_metrics['member_1_prior']
# aggregated_metrics['DPB_2_prior'] = aggregated_metrics['demand_2_prior'] / aggregated_metrics['member_2_prior']

# Step 7: Calculate additional metrics

aggregated_metrics['Conversion Rate'] = aggregated_metrics['Buying Members'] / aggregated_metrics['Total Members']
aggregated_metrics['Demand per Send'] = aggregated_metrics['Demand per Buyer'] * aggregated_metrics['Conversion Rate']
# Secondary KPIs - order deep dive
aggregated_metrics['AOV'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Orders']
aggregated_metrics['AUR'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Units']
aggregated_metrics['UPT'] = aggregated_metrics['Total Units'] / aggregated_metrics['Total Orders']
aggregated_metrics['# Txn per Buyer'] = aggregated_metrics['Demand per Buyer']/aggregated_metrics['AOV']
# Secondary KPIs - Repeat buyers
aggregated_metrics['Conversion rate (1x buyers/email receivers)'] = aggregated_metrics['Members with 1 previous order'] / aggregated_metrics['Total Members']
aggregated_metrics['Conversion rate (2x buyers/email receivers)'] = aggregated_metrics['Members with 2 previous orders'] / aggregated_metrics['Total Members']
aggregated_metrics['Demand per Buyer (1 previous order)'] = aggregated_metrics['demand_1_prior_sum'] / aggregated_metrics.get('member_1_prior_sum', 1)
aggregated_metrics['Demand per Buyer (2 previous order)'] = aggregated_metrics['demand_2_prior_sum'] / aggregated_metrics.get('member_2_prior_sum', 1)

# Secondary KPIs - Email/Push Engagement metrics (all email/push received during measurement windows)
# aggregated_metrics['Email Open Rate (against sends)'] = aggregated_metrics['email_opens_sum'] / aggregated_metrics['email_sends_sum']
# aggregated_metrics['Email Click Rate (against sends)'] = aggregated_metrics['email_clicks_sum'] / aggregated_metrics['email_sends_sum']
# aggregated_metrics['Push Open Rate (against sends)'] = aggregated_metrics['push_opens_sum'] / aggregated_metrics['push_sends_sum']

# Secondary KPIs - product mix
aggregated_metrics['Footwear Demand per Member'] = ( aggregated_metrics['base_FW_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Apparel Demand per Member'] = ( aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Equipment Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['launch_EQ_demand_mean'] ) * aggregated_metrics['Conversion Rate']
# Secondary KPIs - LOB
aggregated_metrics['Regular Product Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['base_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Clearance Demand per Member'] = ( aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Launch Demand per Member'] = ( aggregated_metrics['launch_EQ_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']


# Step 7: Remove temporary calculation columns as they're not needed in final DataFrame
aggregated_metrics.drop(columns=['demand_1_prior_sum', 'demand_2_prior_sum', 'member_1_prior_sum', 'member_2_prior_sum',  'base_FW_demand_mean',
 # 'email_clicks_sum','email_opens_sum','email_sends_sum',  'push_sends_sum', 'push_opens_sum',
 'clr_FW_demand_mean', 'launch_FW_demand_mean', 'base_AP_demand_mean', 'clr_AP_demand_mean', 'launch_AP_demand_mean', 'base_EQ_demand_mean', 'clr_EQ_demand_mean','launch_EQ_demand_mean'], inplace=True)

# Step 8: Perform Chi-Square and Mann-Whitney U tests for 'Conversion Rate' and 'Demand per Buyer'

# Separate the groups for pairwise tests
test_group_ab = df_ab[df_ab['test_control'] == 'test']
control_group_ab = df_ab[df_ab['test_control'] == 'control']
holdout_group_ab = df_ab[df_ab['test_control'] == 'holdout']

# Pairwise Chi-Square tests for Conversion Rate
conv_contingency_table_test_control = pd.crosstab(df_ab[df_ab['test_control'].isin(['test', 'control'])]['test_control'], df_ab[df_ab['test_control'].isin(['test', 'control'])]['buyer'])
conv_contingency_table_test_holdout = pd.crosstab(df_ab[df_ab['test_control'].isin(['test', 'holdout'])]['test_control'], df_ab[df_ab['test_control'].isin(['test', 'holdout'])]['buyer'])
conv_contingency_table_control_holdout = pd.crosstab(df_ab[df_ab['test_control'].isin(['control', 'holdout'])]['test_control'], df_ab[df_ab['test_control'].isin(['control', 'holdout'])]['buyer'])

chi2_test_control, p_test_control, _, _ = chi2_contingency(conv_contingency_table_test_control)
chi2_test_holdout, p_test_holdout, _, _ = chi2_contingency(conv_contingency_table_test_holdout)
chi2_control_holdout, p_control_holdout, _, _ = chi2_contingency(conv_contingency_table_control_holdout)

# Adjust p-values for multiple comparisons
p_values_chi2 = [p_test_control, p_test_holdout, p_control_holdout]
corrected_p_values_chi2 = multipletests(p_values_chi2, method='bonferroni')[1]

# Pairwise Mann-Whitney U tests for Demand per Buyer
u_test_control, p_test_control_mwu = stats.mannwhitneyu(test_group_ab['demand'].dropna(), control_group_ab['demand'].dropna(), alternative='two-sided')
u_test_holdout, p_test_holdout_mwu = stats.mannwhitneyu(test_group_ab['demand'].dropna(), holdout_group_ab['demand'].dropna(), alternative='two-sided')
u_control_holdout, p_control_holdout_mwu = stats.mannwhitneyu(control_group_ab['demand'].dropna(), holdout_group_ab['demand'].dropna(), alternative='two-sided')

# Adjust p-values for multiple comparisons
p_values_mwu = [p_test_control_mwu, p_test_holdout_mwu, p_control_holdout_mwu]
corrected_p_values_mwu = multipletests(p_values_mwu, method='bonferroni')[1]

# Step 9: Transpose the table
pd.options.display.float_format = '{:.8f}'.format
aggregated_metrics = aggregated_metrics.T

# Step 10: Calculate lift for all metrics except 'Conversion Rate' and 'Demand per Buyer'
scorecard = pd.DataFrame(index=aggregated_metrics.index, columns=['Metric', 'Test', 'Control', 'Holdout', 'Lift Test vs Control', 'Lift Test vs Holdout', 'Lift Control vs Holdout', 'P-Value Chi2', 'Stat Chi2', 'P-Value MWU', 'Stat MWU'])

# Populate Test, Control, and Holdout values from aggregated_metrics
scorecard['Test'] = aggregated_metrics['test']
scorecard['Control'] = aggregated_metrics['control']
scorecard['Holdout'] = aggregated_metrics['holdout']

# Calculate Lift
for metric in aggregated_metrics.index:
    if metric not in ['Conversion Rate', 'Demand per Buyer']: # excluding Conversion_Rate and Demand_per_Buyer
        test_val = scorecard.at[metric, 'Test']
        control_val = scorecard.at[metric, 'Control']
        holdout_val = scorecard.at[metric, 'Holdout']
        lift_test_control = ((test_val - control_val) / control_val) * 100 if control_val != 0 else np.nan
        lift_test_holdout = ((test_val - holdout_val) / holdout_val) * 100 if holdout_val != 0 else np.nan
        lift_control_holdout = ((control_val - holdout_val) / holdout_val) * 100 if holdout_val != 0 else np.nan
        scorecard.at[metric, 'Lift Test vs Control'] = lift_test_control
        scorecard.at[metric, 'Lift Test vs Holdout'] = lift_test_holdout
        scorecard.at[metric, 'Lift Control vs Holdout'] = lift_control_holdout

# Step 11: Add Chi2 and MWU test results for 'Conversion Rate' and 'Demand per Buyer' to dataframe
scorecard.at['Conversion Rate', 'Stat Chi2'] = chi2_test_control
scorecard.at['Conversion Rate', 'P-Value Chi2'] = corrected_p_values_chi2[0]
scorecard.at['Demand per Buyer', 'Stat MWU'] = u_test_control
scorecard.at['Demand per Buyer', 'P-Value MWU'] = corrected_p_values_mwu[0]

# Step 12: Ensure 'Metric' column is correctly populated
# A byproduct of transposing the dataframe with some calculations only being applied selectively
scorecard['Metric'] = scorecard.index
scorecard.reset_index(drop=False, inplace=True)
scorecard.drop(columns=['index'], inplace=True)

# Step 13: Exclude 'Conversion Rate' and 'Demand per Buyer' from lift calculation
scorecard.loc[scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Lift Test vs Control'] = ''
scorecard.loc[scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Lift Test vs Holdout'] = ''
scorecard.loc[scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Lift Control vs Holdout'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Stat Chi2'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'P-Value Chi2'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Stat MWU'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'P-Value MWU'] = ''

# Step 14: Display the scorecard
scorecard


#### Updated to reflect fewer p-value and u stat columns



In [ ]:
%python
import numpy as np
import pandas as pd
from scipy import stats
from scipy.stats import chi2_contingency

# Step 1: Add 'buyer' and 'prior_order' indicator columns
df_ab['buyer'] = (df_ab['orders'] > 0).astype(int)
df_ab['demand_1_prior'] = np.where(df_ab['1_priorOrder'] == 1, df_ab['demand'], np.nan)
df_ab['demand_2_prior'] = np.where(df_ab['2_priorOrder'] == 1, df_ab['demand'], np.nan)
df_ab['member_1_prior'] = np.where(df_ab['1_priorOrder'] == 1, 1, 0)
df_ab['member_2_prior'] = np.where(df_ab['2_priorOrder'] == 1, 1, 0)

# Step 2: Perform temporary calculations for DPB_1_prior and DPB_2_prior
temp_calculations = df_ab.groupby('test_control').agg({
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
}).rename(columns={
    'demand_1_prior': 'Total Demand 1 Prior',
    'demand_2_prior': 'Total Demand 2 Prior',
    'member_1_prior': 'Members 1 Prior',
    'member_2_prior': 'Members 2 Prior',
})

# Step 3: Calculate DPB_1_prior and DPB_2_prior using temporary calculations
dpb_1_prior = temp_calculations['Total Demand 1 Prior'] / temp_calculations['Members 1 Prior']
dpb_2_prior = temp_calculations['Total Demand 2 Prior'] / temp_calculations['Members 2 Prior']

# Step 4: Aggregate metrics
aggregated_metrics = df_ab.groupby('test_control').agg({
    'upm_id': 'nunique',
    'buyer': 'sum',
    'orders': 'sum',
    'units': 'sum',
    'demand': ['sum', 'mean'],
    '1_priorOrder': 'sum',
    '2_priorOrder': 'sum',
    'demand_1_prior': 'sum',
    'demand_2_prior': 'sum',
    'member_1_prior': 'sum',
    'member_2_prior': 'sum',
    # 'email_opens': 'sum',
    # 'email_sends': 'sum',
    # 'email_clicks': 'sum',
    # 'push_sends': 'sum',
    # 'push_opens': 'sum',
    'site_app_visits': 'mean',
    'PDP_FAVORITE_COUNT': 'mean',
    'ADD_TO_CART_COUNT': 'mean',
    'physical_activity': 'mean',
    'base_FW_demand': 'mean',
    'clr_FW_demand': 'mean',
    'launch_FW_demand': 'mean',
    'base_AP_demand': 'mean',
    'clr_AP_demand': 'mean',
    'launch_AP_demand': 'mean',
    'base_EQ_demand': 'mean',
    'clr_EQ_demand': 'mean',
    'launch_EQ_demand': 'mean',
})

# Step 5: Correct the MultiIndex handling by flattening and renaming
aggregated_metrics.columns = ['_'.join(col).strip() for col in aggregated_metrics.columns.values]
aggregated_metrics = aggregated_metrics.rename(columns={
    'upm_id_nunique': 'Total Members',
    'buyer_sum': 'Buying Members',
    'demand_mean': 'Demand per Buyer',
    'demand_sum': 'Total Demand',
    'units_sum': 'Total Units',
    'orders_sum': 'Total Orders',
    # Secondary KPIs - Repeat buyers
    '1_priorOrder_sum': 'Members with 1 previous order',
    '2_priorOrder_sum':'Members with 2 previous orders',
    # Secondary KPIs - Site/app Engagement metrics
    'site_app_visits_mean':'Visits per known member',
    'PDP_FAVORITE_COUNT_mean':'PDP favorite per known member',
    'ADD_TO_CART_COUNT_mean': 'Add to cart per known member',
    'physical_activity_mean': 'Workouts per known member',
})

# Step 6: Calculate DPB_1_prior and DPB_2_prior based on the temporary calculations and integrate directly into the DataFrame
# aggregated_metrics['DPB_1_prior'] = aggregated_metrics['demand_1_prior'] / aggregated_metrics['member_1_prior']
# aggregated_metrics['DPB_2_prior'] = aggregated_metrics['demand_2_prior'] / aggregated_metrics['member_2_prior']

# Step 7: Calculate additional metrics

aggregated_metrics['Conversion Rate'] = aggregated_metrics['Buying Members'] / aggregated_metrics['Total Members']
aggregated_metrics['Demand per Send'] = aggregated_metrics['Demand per Buyer'] * aggregated_metrics['Conversion Rate']
# Secondary KPIs - order deep dive
aggregated_metrics['AOV'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Orders']
aggregated_metrics['AUR'] = aggregated_metrics['Total Demand'] / aggregated_metrics['Total Units']
aggregated_metrics['UPT'] = aggregated_metrics['Total Units'] / aggregated_metrics['Total Orders']
aggregated_metrics['# Txn per Buyer'] = aggregated_metrics['Demand per Buyer']/aggregated_metrics['AOV']
# Secondary KPIs - Repeat buyers
aggregated_metrics['Conversion rate (1x buyers/email receivers)'] = aggregated_metrics['Members with 1 previous order'] / aggregated_metrics['Total Members']
aggregated_metrics['Conversion rate (2x buyers/email receivers)'] = aggregated_metrics['Members with 2 previous orders'] / aggregated_metrics['Total Members']
aggregated_metrics['Demand per Buyer (1 previous order)'] = aggregated_metrics['demand_1_prior_sum'] / aggregated_metrics.get('member_1_prior_sum', 1)
aggregated_metrics['Demand per Buyer (2 previous order)'] = aggregated_metrics['demand_2_prior_sum'] / aggregated_metrics.get('member_2_prior_sum', 1)

# Secondary KPIs - Email/Push Engagement metrics (all email/push received during measurement windows)
# aggregated_metrics['Email Open Rate (against sends)'] = aggregated_metrics['email_opens_sum'] / aggregated_metrics['email_sends_sum']
# aggregated_metrics['Email Click Rate (against sends)'] = aggregated_metrics['email_clicks_sum'] / aggregated_metrics['email_sends_sum']
# aggregated_metrics['Push Open Rate (against sends)'] = aggregated_metrics['push_opens_sum'] / aggregated_metrics['push_sends_sum']

# Secondary KPIs - product mix
aggregated_metrics['Footwear Demand per Member'] = ( aggregated_metrics['base_FW_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Apparel Demand per Member'] = ( aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] ) * aggregated_metrics['Conversion Rate']
aggregated_metrics['Equipment Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['launch_EQ_demand_mean'] ) * aggregated_metrics['Conversion Rate']
# Secondary KPIs - LOB
aggregated_metrics['Regular Product Demand per Member'] = ( aggregated_metrics['base_EQ_demand_mean'] + aggregated_metrics['base_AP_demand_mean'] + aggregated_metrics['base_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Clearance Demand per Member'] = ( aggregated_metrics['clr_EQ_demand_mean'] + aggregated_metrics['clr_AP_demand_mean'] + aggregated_metrics['clr_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

aggregated_metrics['Launch Demand per Member'] = ( aggregated_metrics['launch_EQ_demand_mean'] + aggregated_metrics['launch_AP_demand_mean'] + aggregated_metrics['launch_FW_demand_mean'] ) * aggregated_metrics['Conversion Rate']

# Step 7: Remove temporary calculation columns as they're not needed in final DataFrame
aggregated_metrics.drop(columns=['demand_1_prior_sum', 'demand_2_prior_sum', 'member_1_prior_sum', 'member_2_prior_sum',  'base_FW_demand_mean',
   # 'email_clicks_sum','email_opens_sum','email_sends_sum',  'push_sends_sum', 'push_opens_sum',
 'clr_FW_demand_mean', 'launch_FW_demand_mean', 'base_AP_demand_mean', 'clr_AP_demand_mean', 'launch_AP_demand_mean', 'base_EQ_demand_mean', 'clr_EQ_demand_mean','launch_EQ_demand_mean'], inplace=True)

# Step 8: Perform Chi-Square and Mann-Whitney U tests for 'Conversion Rate' and 'Demand per Buyer'

# Separate the groups for pairwise tests
test_group_ab = df_ab[df_ab['test_control'] == 'test']
control_group_ab = df_ab[df_ab['test_control'] == 'control']
holdout_group_ab = df_ab[df_ab['test_control'] == 'holdout']

# Pairwise Chi-Square tests for Conversion Rate
conv_contingency_table_test_control = pd.crosstab(df_ab[df_ab['test_control'].isin(['test', 'control'])]['test_control'], df_ab[df_ab['test_control'].isin(['test', 'control'])]['buyer'])
conv_contingency_table_test_holdout = pd.crosstab(df_ab[df_ab['test_control'].isin(['test', 'holdout'])]['test_control'], df_ab[df_ab['test_control'].isin(['test', 'holdout'])]['buyer'])
conv_contingency_table_control_holdout = pd.crosstab(df_ab[df_ab['test_control'].isin(['control', 'holdout'])]['test_control'], df_ab[df_ab['test_control'].isin(['control', 'holdout'])]['buyer'])

chi2_test_control, p_test_control, _, _ = chi2_contingency(conv_contingency_table_test_control)
chi2_test_holdout, p_test_holdout, _, _ = chi2_contingency(conv_contingency_table_test_holdout)
chi2_control_holdout, p_control_holdout, _, _ = chi2_contingency(conv_contingency_table_control_holdout)

# Pairwise Mann-Whitney U tests for Demand per Buyer
u_test_control, p_test_control_mwu = stats.mannwhitneyu(test_group_ab['demand'].dropna(), control_group_ab['demand'].dropna(), alternative='two-sided')
u_test_holdout, p_test_holdout_mwu = stats.mannwhitneyu(test_group_ab['demand'].dropna(), holdout_group_ab['demand'].dropna(), alternative='two-sided')
u_control_holdout, p_control_holdout_mwu = stats.mannwhitneyu(control_group_ab['demand'].dropna(), holdout_group_ab['demand'].dropna(), alternative='two-sided')

# Step 9: Transpose the table
pd.options.display.float_format = '{:.8f}'.format
aggregated_metrics = aggregated_metrics.T

# Step 10: Calculate lift for all metrics except 'Conversion Rate' and 'Demand per Buyer'
scorecard = pd.DataFrame(index=aggregated_metrics.index, columns=['Metric', 'Test', 'Control', 'Holdout', 'Lift Test vs Control', 'Lift Test vs Holdout', 'Lift Control vs Holdout', 'Stat Test vs Control', 'P-Value Test vs Control', 'Stat Test vs Holdout', 'P-Value Test vs Holdout', 'Stat Control vs Holdout', 'P-Value Control vs Holdout'])

# Populate Test, Control, and Holdout values from aggregated_metrics
scorecard['Test'] = aggregated_metrics['test']
scorecard['Control'] = aggregated_metrics['control']
scorecard['Holdout'] = aggregated_metrics['holdout']

# Calculate Lift
for metric in aggregated_metrics.index:
    if metric not in ['Conversion Rate', 'Demand per Buyer']: # excluding Conversion_Rate and Demand_per_Buyer
        test_val = scorecard.at[metric, 'Test']
        control_val = scorecard.at[metric, 'Control']
        holdout_val = scorecard.at[metric, 'Holdout']
        lift_test_control = ((test_val - control_val) / control_val) * 100 if control_val != 0 else np.nan
        lift_test_holdout = ((test_val - holdout_val) / holdout_val) * 100 if holdout_val != 0 else np.nan
        lift_control_holdout = ((control_val - holdout_val) / holdout_val) * 100 if holdout_val != 0 else np.nan
        scorecard.at[metric, 'Lift Test vs Control'] = lift_test_control
        scorecard.at[metric, 'Lift Test vs Holdout'] = lift_test_holdout
        scorecard.at[metric, 'Lift Control vs Holdout'] = lift_control_holdout

# Step 11: Add test results for 'Conversion Rate' and 'Demand per Buyer' to the dataframe
# Conversion Rate - Chi2 Test results
scorecard.at['Conversion Rate', 'Stat Test vs Control'] = chi2_test_control
scorecard.at['Conversion Rate', 'P-Value Test vs Control'] = p_test_control
scorecard.at['Conversion Rate', 'Stat Test vs Holdout'] = chi2_test_holdout
scorecard.at['Conversion Rate', 'P-Value Test vs Holdout'] = p_test_holdout
scorecard.at['Conversion Rate', 'Stat Control vs Holdout'] = chi2_control_holdout
scorecard.at['Conversion Rate', 'P-Value Control vs Holdout'] = p_control_holdout

# Demand per Buyer - MWU Test results
scorecard.at['Demand per Buyer', 'Stat Test vs Control'] = u_test_control
scorecard.at['Demand per Buyer', 'P-Value Test vs Control'] = p_test_control_mwu
scorecard.at['Demand per Buyer', 'Stat Test vs Holdout'] = u_test_holdout
scorecard.at['Demand per Buyer', 'P-Value Test vs Holdout'] = p_test_holdout_mwu
scorecard.at['Demand per Buyer', 'Stat Control vs Holdout'] = u_control_holdout
scorecard.at['Demand per Buyer', 'P-Value Control vs Holdout'] = p_control_holdout_mwu

# Step 12: Ensure 'Metric' column is correctly populated
# A byproduct of transposing the dataframe with some calculations only being applied selectively
scorecard['Metric'] = scorecard.index
scorecard.reset_index(drop=False, inplace=True)
scorecard.drop(columns=['index'], inplace=True)

# Step 13: Exclude 'Conversion Rate' and 'Demand per Buyer' from lift calculation
scorecard.loc[scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Lift Test vs Control'] = ''
scorecard.loc[scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Lift Test vs Holdout'] = ''
scorecard.loc[scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Lift Control vs Holdout'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Stat Test vs Control'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'P-Value Test vs Control'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Stat Test vs Holdout'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'P-Value Test vs Holdout'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'Stat Control vs Holdout'] = ''
scorecard.loc[~scorecard['Metric'].isin(['Conversion Rate', 'Demand per Buyer']), 'P-Value Control vs Holdout'] = ''

# Step 14: Display the scorecard
scorecard


### Chi Square for Conversion

#### For Two Variants - test & control

In [ ]:
# Two Variants - test and control

%python
from scipy.stats import chi2_contingency

# 'df' contains 'test_control' column indicating group membership
# 'buyer' column indicating conversion (1) or non-conversion (0)

# set alpha
alpha = 0.1

# Creating a contingency table
# Count the number of conversions and non-conversions in each group
conv_contingency_table_ab = pd.crosstab(df_ab['test_control'], df_ab['buyer'])

print("Contingency Table for Conversion:\n")
print(conv_contingency_table_ab)

# Perform Chi-square test
chi2_stat_conversion_ab, p_value_conversion_ab, dof_conversion_ab, expected_cnt_conversion_ab = chi2_contingency(conv_contingency_table_ab)

print(f"\nChi2 Stat: {chi2_stat_conversion_ab}")
print(f"\nP-value: {p_value_conversion_ab}")
print(f"\nDegrees of Freedom: {dof_conversion_ab}")
print(f"\nExpected Counts: {expected_cnt_conversion_ab}")

# Interpretation
if p_value_conversion_ab < alpha:
    print("\nSignificant difference in conversion rates between groups.")
    print(f"\nTherefore, we reject the null hypothesis (that conversion rates are the same between groups).")
else:
    print("\nNo significant difference in conversion rates between groups.")
    print(f"\nTherefore, we do not reject the null hypothesis (that conversion rates are the same between groups).")


#### For Three Variants - test & control & holdout

In [ ]:
# Perform Pairwise Chi-Square Tests: Conduct chi-square tests for each pair of groups separately.

%python
import pandas as pd
from scipy.stats import chi2_contingency
from statsmodels.stats.multitest import multipletests


# Set alpha
alpha = 0.1

# Create contingency tables for each pair of groups

# Test vs Control
conv_contingency_table_test_control = pd.crosstab(df_ab[df_ab['test_control'].isin(['test', 'control'])]['test_control'],
                                                  df_ab[df_ab['test_control'].isin(['test', 'control'])]['buyer'])

# Test vs Holdout
conv_contingency_table_test_holdout = pd.crosstab(df_ab[df_ab['test_control'].isin(['test', 'holdout'])]['test_control'],
                                                  df_ab[df_ab['test_control'].isin(['test', 'holdout'])]['buyer'])

# Control vs Holdout
conv_contingency_table_control_holdout = pd.crosstab(df_ab[df_ab['test_control'].isin(['control', 'holdout'])]['test_control'],
                                                     df_ab[df_ab['test_control'].isin(['control', 'holdout'])]['buyer'])

# Perform Chi-Square tests
chi2_test_control, p_test_control, dof_test_control, expected_test_control = chi2_contingency(conv_contingency_table_test_control)
chi2_test_holdout, p_test_holdout, dof_test_holdout, expected_test_holdout = chi2_contingency(conv_contingency_table_test_holdout)
chi2_control_holdout, p_control_holdout, dof_control_holdout, expected_control_holdout = chi2_contingency(conv_contingency_table_control_holdout)

# Adjust for multiple comparisons using the Bonferroni correction
p_values = [p_test_control, p_test_holdout, p_control_holdout]
corrected_p_values = multipletests(p_values, method='bonferroni')[1]

p_test_control_corrected = corrected_p_values[0]
p_test_holdout_corrected = corrected_p_values[1]
p_control_holdout_corrected = corrected_p_values[2]

# Print results
print("Corrected p-value for Test vs Control:", p_test_control_corrected)
print("Corrected p-value for Test vs Holdout:", p_test_holdout_corrected)
print("Corrected p-value for Control vs Holdout:", p_control_holdout_corrected)

# Interpretation
print("\nInterpretation after correction:")
if p_test_control_corrected < alpha:
    print("Significant difference between Test and Control.")
else:
    print("No significant difference between Test and Control.")

if p_test_holdout_corrected < alpha:
    print("Significant difference between Test and Holdout.")
else:
    print("No significant difference between Test and Holdout.")

if p_control_holdout_corrected < alpha:
    print("Significant difference between Control and Holdout.")
else:
    print("No significant difference between Control and Holdout.")



The **chi2_contingency function** from SciPy returns four values:

1. The chi-squared test statistic (`chi2`).
2. The p-value of the test (`p`).
3. The degrees of freedom (`dof`).
4. The expected frequencies in each category (`expected`).

We are not using degrees of freedom and expected frequenciesin this particular analysis.

### Pairwise Mann Whitney U for Demand

In [ ]:

# Perform Pairwise Mann Whitney U Tests: Conduct Mann Whitney U tests for each pair of groups separately.

%python
from scipy.stats import mannwhitneyu
from statsmodels.stats.multitest import multipletests

# Separate the groups
test_group = df_ab[df_ab['test_control'] == 'test']['demand'].dropna()
control_group = df_ab[df_ab['test_control'] == 'control']['demand'].dropna()
holdout_group = df_ab[df_ab['test_control'] == 'holdout']['demand'].dropna()

# Perform Mann-Whitney U tests for each pair
u_test_control, p_test_control = mannwhitneyu(test_group, control_group, alternative='two-sided')
u_test_holdout, p_test_holdout = mannwhitneyu(test_group, holdout_group, alternative='two-sided')
u_control_holdout, p_control_holdout = mannwhitneyu(control_group, holdout_group, alternative='two-sided')

# Print raw p-values
print("Raw p-values for Mann-Whitney U test:")
print(f"Test vs Control: {p_test_control}")
print(f"Test vs Holdout: {p_test_holdout}")
print(f"Control vs Holdout: {p_control_holdout}")

# Adjust for multiple comparisons using the Bonferroni correction
p_values = [p_test_control, p_test_holdout, p_control_holdout]
corrected_p_values = multipletests(p_values, method='bonferroni')[1]

# Assign corrected p-values to variables
p_test_control_corrected = corrected_p_values[0]
p_test_holdout_corrected = corrected_p_values[1]
p_control_holdout_corrected = corrected_p_values[2]

# Print corrected p-values
print("\nCorrected p-values using Bonferroni correction:")
print(f"Corrected p-value for Test vs Control: {p_test_control_corrected}")
print(f"Corrected p-value for Test vs Holdout: {p_test_holdout_corrected}")
print(f"Corrected p-value for Control vs Holdout: {p_control_holdout_corrected}")

# Interpretation based on corrected p-values
alpha = 0.1 / 3  # Adjusted alpha for Bonferroni correction

print("\nInterpretation after correction:")
if p_test_control_corrected < alpha:
    print("Significant difference between Test and Control.")
else:
    print("No significant difference between Test and Control.")

if p_test_holdout_corrected < alpha:
    print("Significant difference between Test and Holdout.")
else:
    print("No significant difference between Test and Holdout.")

if p_control_holdout_corrected < alpha:
    print("Significant difference between Control and Holdout.")
else:
    print("No significant difference between Control and Holdout.")
